In [ ]:
# inspecionando todos os arquivos: 
import pandas as pd
import json
import xml.etree.ElementTree as ET
import os

# Lista de caminhos dos arquivos que você forneceu
# Use 'r' antes das aspas para garantir que o caminho seja lido corretamente pelo Python
file_paths = [
    r"C:\Users\Leo\Downloads\carteira_52203615000119_20250901.json",
    r"C:\Users\Leo\Downloads\130170-Demonstrativo_Caixa.xlsx",
    r"C:\Users\Leo\Downloads\130166-Carteira.xlsx",
    r"C:\Users\Leo\Downloads\remessas_2025-09-01_completo.json",
    r"C:\Users\Leo\Downloads\remessas_2025-09-01_detalhes_status.csv",
    r"C:\Users\Leo\Downloads\remessas_2025-09-01_lista.csv",
    r"C:\Users\Leo\Downloads\atualizacao_acessos.xlsx",
    r"C:\Users\Leo\Downloads\documentos_52203615000119.json",
    r"C:\Users\Leo\Downloads\78417127.xml",
    r"C:\Users\Leo\Downloads\movimentacao_cotistas_52203615000119_2025-08-20_completo.json",
    r"C:\Users\Leo\Downloads\extrato_52203615000119_2025-08-01_a_2025-08-31_completo.json"
]

def inspect_file(path):
    """
    Função para inspecionar um arquivo, identificando seu tipo e mostrando
    uma prévia do seu conteúdo.
    """
    # Pega apenas o nome do arquivo para exibição
    filename = os.path.basename(path)
    print(f"--- Inspecionando: {filename} ---")
    
    # Verifica se o arquivo realmente existe no caminho especificado
    if not os.path.exists(path):
        print("!!! ERRO: Arquivo não encontrado. Verifique o caminho. !!!\n")
        return

    try:
        # Lógica para arquivos .csv
        if path.endswith('.csv'):
            df = pd.read_csv(path, encoding='utf-8', sep=None, engine='python', nrows=5)
            print("Formato: CSV")
            print("Colunas detectadas:", df.columns.tolist())
            print("Prévia das 5 primeiras linhas:")
            print(df.head(5))

        # Lógica para arquivos .xlsx
        elif path.endswith('.xlsx'):
            xls = pd.ExcelFile(path)
            print(f"Formato: Excel com {len(xls.sheet_names)} aba(s): {xls.sheet_names}")
            # Mostra o início de cada aba (planilha)
            for sheet_name in xls.sheet_names:
                print(f"\n... Lendo aba: '{sheet_name}' ...")
                df = pd.read_excel(path, sheet_name=sheet_name, nrows=60)
                print("Colunas detectadas:", df.columns.tolist())
                print("Prévia das 60 primeiras linhas:")
                print(df.head(60))

        # Lógica para arquivos .json
        elif path.endswith('.json'):
            print("Formato: JSON")
            with open(path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                if isinstance(data, dict):
                    print("Tipo: Dicionário (Objeto JSON)")
                    print("Chaves do JSON (nível raiz):", list(data.keys()))
                elif isinstance(data, list) and data:
                    print("Tipo: Lista de objetos")
                    if isinstance(data[0], dict):
                        print("Chaves do primeiro objeto da lista:", list(data[0].keys()))
                else:
                    print("Conteúdo do JSON não é um dicionário ou lista de objetos.")

        # Lógica para arquivos .xml
        elif path.endswith('.xml'):
            print("Formato: XML")
            tree = ET.parse(path)
            root = tree.getroot()
            print(f"Elemento raiz (root): <{root.tag}>")
            print("Primeiros 5 elementos filhos:")
            for i, child in enumerate(root):
                if i >= 5:
                    break
                print(f"  - <{child.tag}>")

        else:
            print(f"Formato de arquivo não suportado para inspeção automática: {filename}")

    except Exception as e:
        print(f"!!! ERRO ao processar o arquivo: {e} !!!")
    
    # Linha para separar a inspeção de cada arquivo
    print("-" * (len(filename) + 22) + "\n")


# Roda a função de inspeção para cada arquivo da lista
for file_path in file_paths:
    inspect_file(file_path)

print("--- Fim da inspeção ---")

--- Inspecionando: carteira_52203615000119_20250901.json ---
Formato: JSON
Tipo: Lista de objetos
Chaves do primeiro objeto da lista: ['carteiras', 'ativos', 'passivos']
-----------------------------------------------------------

--- Inspecionando: 130170-Demonstrativo_Caixa.xlsx ---
Formato: Excel com 1 aba(s): ['52203615000119-15540']

... Lendo aba: '52203615000119-15540' ...
Colunas detectadas: ['Fundo', 'Carteira', 'Tipo Carteira', 'Posição', 'Emissão', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']
Prévia das 60 primeiras linhas:
                     Fundo                            Carteira Tipo Carteira  \
0   FIDC FCT CONSIGNADO II                               15540          FIDC   
1                      NaN                                 NaN           NaN   
2                   Título                           Histórico     Título CP   
3                      NaN           SALDO FINAL EM 07/07/2025           NaN   
4                C/C VORTX              Certificadora - 584755

In [1]:
import json
import pandas as pd
from datetime import datetime, timedelta

def carregar_dados_carteira(caminho_arquivo):
    """Carrega os dados da carteira a partir de um arquivo JSON."""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        # O JSON é uma lista com um único objeto principal
        dados = json.load(f)[0]
    return dados

def extrair_fluxos_de_caixa(dados):
    """Extrai e organiza todos os eventos de fluxo de caixa do JSON."""
    fluxos = []
    data_base = datetime.strptime(dados['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # 1. Disponibilidades (Entradas)
    # Conta Corrente (D+0)
    disponibilidade = next((ativo['Disponibilidade'] for ativo in dados['ativos'] if 'Disponibilidade' in ativo and ativo['Disponibilidade']), None)
    if disponibilidade:
        saldo_cc = disponibilidade['ativos'][0]['saldo']
        fluxos.append({'data': data_base, 'descricao': 'Conta Corrente (Saldo Inicial)', 'valor': saldo_cc, 'tipo': 'Disponibilidade'})

    # Fundos Investidos (Cotas - Assumindo resgate em D+1)
    cotas = next((ativo['Cotas'] for ativo in dados['ativos'] if 'Cotas' in ativo and ativo['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos.append({'data': data_base + timedelta(days=1), 'descricao': f"Resgate Fundo '{cota['titulo']}'", 'valor': cota['valorBruto'], 'tipo': 'Disponibilidade'})
            
    # Títulos de Renda Fixa
    renda_fixa = next((ativo['RendaFixa'] for ativo in dados['ativos'] if 'RendaFixa' in ativo and ativo['RendaFixa']), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']:
            data_vencimento = datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d')
            fluxos.append({'data': data_vencimento, 'descricao': f"Venc. RF '{titulo['codigoCustodia']}'", 'valor': titulo['mercadoAtual'], 'tipo': 'Disponibilidade'})
    
    # ATENÇÃO: Pagamentos do Estoque (Direitos Creditórios) - SEM DATA NO JSON
    # Esta seção precisaria ser implementada quando a data de vencimento dos direitos creditórios for conhecida.
    # Ex: fluxos.append({'data': data_venc_dc, 'descricao': 'Recebimento Estoque', 'valor': valor_dc, 'tipo': 'Disponibilidade'})


    # 2. Necessidades (Saídas)
    # Despesas (Provisões)
    passivos = dados.get('passivos', [])
    if passivos:
        for despesa in passivos[0].get('ativos', []):
            # Lógica simples para tentar inferir a data. Isso deve ser validado.
            # Se não encontrar data, assume pagamento para o mês seguinte.
            data_pagamento = data_base + timedelta(days=30)
            if 'DEZ25' in despesa['despesa']:
                data_pagamento = datetime(2025, 12, 15) # Exemplo
            
            fluxos.append({'data': data_pagamento, 'descricao': f"Despesa: {despesa['despesa']}", 'valor': despesa['valor'], 'tipo': 'Necessidade'})

    # Amortização de Cotas (Série Sênior)
    carteira_senior = next((c for c in dados['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1 # Valor negativo para saída
        
        # Gera 36 pagamentos mensais a partir de Jan/2026
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            # Regra: Se fim de semana, vai para o próximo dia útil
            while data_amortizacao.weekday() >= 5: # 5=Sábado, 6=Domingo
                data_amortizacao += timedelta(days=1)
            
            fluxos.append({'data': data_amortizacao, 'descricao': f"Amortização Cota Sênior ({i+1}/36)", 'valor': valor_amortizacao, 'tipo': 'Necessidade'})
            
            # Próximo mês
            ano = data_amortizacao.year
            mes = data_amortizacao.month + 1
            if mes > 12:
                mes = 1
                ano += 1
            data_amortizacao = data_amortizacao.replace(year=ano, month=mes)

    return fluxos

def criar_tabela_projecao(fluxos, data_base):
    """Cria a tabela final de projeção de fluxo de caixa."""
    if not fluxos:
        print("Nenhum fluxo de caixa foi extraído.")
        return

    df = pd.DataFrame(fluxos)
    df_fluxo_diario = df.groupby('data')['valor'].sum().reset_index()
    
    # Cria um range de datas do início ao fim para garantir que todos os dias sejam considerados
    idx = pd.date_range(start=data_base, end=df_fluxo_diario['data'].max())
    df_fluxo_diario = df_fluxo_diario.set_index('data').reindex(idx, fill_value=0).reset_index().rename(columns={'index':'data'})
    
    df_fluxo_diario['saldo_projetado'] = df_fluxo_diario['valor'].cumsum()
    df_fluxo_diario = df_fluxo_diario.set_index('data')

    # Define os prazos para a tabela final
    prazos_dias = {'D+0': 0, 'D+1': 1, 'D+5': 5, 'D+10': 10, 'D+21': 21, 'D+63': 63}
    
    # Adiciona datas de amortização para visualização
    datas_amortizacao = sorted([f['data'] for f in fluxos if 'Amortização' in f['descricao']])
    if datas_amortizacao:
        prazos_dias[f"1ª Amort. ({datas_amortizacao[0].strftime('%d/%m/%Y')})"] = (datas_amortizacao[0] - data_base).days
        prazos_dias[f"Últ. Amort. ({datas_amortizacao[-1].strftime('%d/%m/%Y')})"] = (datas_amortizacao[-1] - data_base).days

    projecao = []
    for nome_prazo, dias in prazos_dias.items():
        data_projecao = data_base + timedelta(days=dias)
        saldo = df_fluxo_diario.asof(data_projecao)['saldo_projetado']
        projecao.append({'Prazo': nome_prazo, 'Data': data_projecao.strftime('%d/%m/%Y'), 'Saldo Projetado': f"R$ {saldo:,.2f}"})

    return pd.DataFrame(projecao)


# --- Execução Principal ---
caminho_json = "C:\\Users\\Leo\\Downloads\\carteira_52203615000119_20250901.json"
dados_carteira = carregar_dados_carteira(caminho_json)
lista_fluxos = extrair_fluxos_de_caixa(dados_carteira)

data_referencia = datetime.strptime(dados_carteira['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
tabela_final = criar_tabela_projecao(lista_fluxos, data_referencia)

print("--- Projeção de Fluxo de Caixa ---")
print(tabela_final.to_string())
print("\n--- Avisos Importantes ---")
print("1. Pagamento de Despesas: As datas foram inferidas. É necessário confirmar as datas reais de vencimento.")
print("2. Pagamento de Estoque (Direitos Creditórios): O valor total de R$ 214M a receber NÃO foi incluído na projeção por falta de datas de vencimento no arquivo JSON.")
print("3. Resgate de Fundos: Assumiu-se um prazo de resgate de D+1 para os fundos de cotas.")

--- Projeção de Fluxo de Caixa ---
                      Prazo        Data   Saldo Projetado
0                       D+0  01/09/2025       R$ 5,292.37
1                       D+1  02/09/2025   R$ 1,183,417.67
2                       D+5  06/09/2025   R$ 1,183,417.67
3                      D+10  11/09/2025   R$ 1,183,417.67
4                      D+21  22/09/2025   R$ 1,183,417.67
5                      D+63  03/11/2025  R$ -3,254,323.10
6    1ª Amort. (16/01/2026)  16/01/2026  R$ -6,367,646.81
7  Últ. Amort. (11/01/2029)  11/01/2029  R$ -3,259,933.78

--- Avisos Importantes ---
1. Pagamento de Despesas: As datas foram inferidas. É necessário confirmar as datas reais de vencimento.
2. Pagamento de Estoque (Direitos Creditórios): O valor total de R$ 214M a receber NÃO foi incluído na projeção por falta de datas de vencimento no arquivo JSON.
3. Resgate de Fundos: Assumiu-se um prazo de resgate de D+1 para os fundos de cotas.


In [2]:
import json
import pandas as pd
from datetime import datetime, timedelta

def carregar_dados_carteira(caminho_arquivo):
    """Carrega os dados da carteira a partir de um arquivo JSON."""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        # O JSON é uma lista com um único objeto principal
        dados = json.load(f)[0]
    return dados

def extrair_fluxos_de_caixa(dados):
    """Extrai e organiza todos os eventos de fluxo de caixa do JSON."""
    fluxos = []
    data_base = datetime.strptime(dados['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # 1. Disponibilidades (Entradas)
    # Conta Corrente (D+0)
    disponibilidade = next((ativo['Disponibilidade'] for ativo in dados['ativos'] if 'Disponibilidade' in ativo and ativo['Disponibilidade']), None)
    if disponibilidade:
        saldo_cc = disponibilidade['ativos'][0]['saldo']
        fluxos.append({'data': data_base, 'descricao': 'Conta Corrente (Saldo Inicial)', 'valor': saldo_cc, 'tipo': 'Disponibilidade'})

    # Fundos Investidos (Cotas - Assumindo resgate em D+1)
    cotas = next((ativo['Cotas'] for ativo in dados['ativos'] if 'Cotas' in ativo and ativo['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos.append({'data': data_base + timedelta(days=1), 'descricao': f"Resgate Fundo '{cota['titulo']}'", 'valor': cota['valorBruto'], 'tipo': 'Disponibilidade'})
            
    # Títulos de Renda Fixa
    renda_fixa = next((ativo['RendaFixa'] for ativo in dados['ativos'] if 'RendaFixa' in ativo and ativo['RendaFixa']), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']:
            data_vencimento = datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d')
            fluxos.append({'data': data_vencimento, 'descricao': f"Venc. RF '{titulo['codigoCustodia']}'", 'valor': titulo['mercadoAtual'], 'tipo': 'Disponibilidade'})
    
    # ATENÇÃO: Pagamentos do Estoque (Direitos Creditórios) - SEM DATA NO JSON
    # Esta seção precisaria ser implementada quando a data de vencimento dos direitos creditórios for conhecida.
    # Ex: fluxos.append({'data': data_venc_dc, 'descricao': 'Recebimento Estoque', 'valor': valor_dc, 'tipo': 'Disponibilidade'})


    # 2. Necessidades (Saídas)
    # Despesas (Provisões)
    passivos = dados.get('passivos', [])
    if passivos:
        for despesa in passivos[0].get('ativos', []):
            # Lógica simples para tentar inferir a data. Isso deve ser validado.
            # Se não encontrar data, assume pagamento para o mês seguinte.
            data_pagamento = data_base + timedelta(days=30)
            if 'DEZ25' in despesa['despesa']:
                data_pagamento = datetime(2025, 12, 15) # Exemplo
            
            fluxos.append({'data': data_pagamento, 'descricao': f"Despesa: {despesa['despesa']}", 'valor': despesa['valor'], 'tipo': 'Necessidade'})

    # Amortização de Cotas (Série Sênior)
    carteira_senior = next((c for c in dados['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1 # Valor negativo para saída
        
        # Gera 36 pagamentos mensais a partir de Jan/2026
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            # Regra: Se fim de semana, vai para o próximo dia útil
            while data_amortizacao.weekday() >= 5: # 5=Sábado, 6=Domingo
                data_amortizacao += timedelta(days=1)
            
            fluxos.append({'data': data_amortizacao, 'descricao': f"Amortização Cota Sênior ({i+1}/36)", 'valor': valor_amortizacao, 'tipo': 'Necessidade'})
            
            # Próximo mês
            ano = data_amortizacao.year
            mes = data_amortizacao.month + 1
            if mes > 12:
                mes = 1
                ano += 1
            data_amortizacao = data_amortizacao.replace(year=ano, month=mes)

    return fluxos

def criar_tabela_projecao(fluxos, data_base):
    """Cria a tabela final de projeção de fluxo de caixa."""
    if not fluxos:
        print("Nenhum fluxo de caixa foi extraído.")
        return

    df = pd.DataFrame(fluxos)
    df_fluxo_diario = df.groupby('data')['valor'].sum().reset_index()
    
    # Cria um range de datas do início ao fim para garantir que todos os dias sejam considerados
    idx = pd.date_range(start=data_base, end=df_fluxo_diario['data'].max())
    df_fluxo_diario = df_fluxo_diario.set_index('data').reindex(idx, fill_value=0).reset_index().rename(columns={'index':'data'})
    
    df_fluxo_diario['saldo_projetado'] = df_fluxo_diario['valor'].cumsum()
    df_fluxo_diario = df_fluxo_diario.set_index('data')

    # Define os prazos para a tabela final
    prazos_dias = {'D+0': 0, 'D+1': 1, 'D+5': 5, 'D+10': 10, 'D+21': 21, 'D+63': 63}
    
    # Adiciona datas de amortização para visualização
    datas_amortizacao = sorted([f['data'] for f in fluxos if 'Amortização' in f['descricao']])
    if datas_amortizacao:
        prazos_dias[f"1ª Amort. ({datas_amortizacao[0].strftime('%d/%m/%Y')})"] = (datas_amortizacao[0] - data_base).days
        prazos_dias[f"Últ. Amort. ({datas_amortizacao[-1].strftime('%d/%m/%Y')})"] = (datas_amortizacao[-1] - data_base).days

    projecao = []
    for nome_prazo, dias in prazos_dias.items():
        data_projecao = data_base + timedelta(days=dias)
        saldo = df_fluxo_diario.asof(data_projecao)['saldo_projetado']
        projecao.append({'Prazo': nome_prazo, 'Data': data_projecao.strftime('%d/%m/%Y'), 'Saldo Projetado': f"R$ {saldo:,.2f}"})

    return pd.DataFrame(projecao)


# --- Execução Principal ---
caminho_json = "C:\\Users\\Leo\\Downloads\\carteira_52203615000119_20250901.json"
dados_carteira = carregar_dados_carteira(caminho_json)
lista_fluxos = extrair_fluxos_de_caixa(dados_carteira)

data_referencia = datetime.strptime(dados_carteira['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
tabela_final = criar_tabela_projecao(lista_fluxos, data_referencia)

print("--- Projeção de Fluxo de Caixa ---")
print(tabela_final.to_string())
print("\n--- Avisos Importantes ---")
print("1. Pagamento de Despesas: As datas foram inferidas. É necessário confirmar as datas reais de vencimento.")
print("2. Pagamento de Estoque (Direitos Creditórios): O valor total de R$ 214M a receber NÃO foi incluído na projeção por falta de datas de vencimento no arquivo JSON.")
print("3. Resgate de Fundos: Assumiu-se um prazo de resgate de D+1 para os fundos de cotas.")

--- Projeção de Fluxo de Caixa ---
                      Prazo        Data   Saldo Projetado
0                       D+0  01/09/2025       R$ 5,292.37
1                       D+1  02/09/2025   R$ 1,183,417.67
2                       D+5  06/09/2025   R$ 1,183,417.67
3                      D+10  11/09/2025   R$ 1,183,417.67
4                      D+21  22/09/2025   R$ 1,183,417.67
5                      D+63  03/11/2025  R$ -3,254,323.10
6    1ª Amort. (16/01/2026)  16/01/2026  R$ -6,367,646.81
7  Últ. Amort. (11/01/2029)  11/01/2029  R$ -3,259,933.78

--- Avisos Importantes ---
1. Pagamento de Despesas: As datas foram inferidas. É necessário confirmar as datas reais de vencimento.
2. Pagamento de Estoque (Direitos Creditórios): O valor total de R$ 214M a receber NÃO foi incluído na projeção por falta de datas de vencimento no arquivo JSON.
3. Resgate de Fundos: Assumiu-se um prazo de resgate de D+1 para os fundos de cotas.


In [3]:
# depurar dentro do estoque:
import pandas as pd
import os
import glob

# --- Sua função para processar o estoque ---
# Colocamos ela aqui para ser usada pelo nosso script de depuração.
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"

def processar_dados_estoque(caminho_pasta):
    """Lê e consolida os arquivos de estoque de uma pasta."""
    print("\nCarregando dados de estoque...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv:
        print(f"AVISO: Nenhum arquivo CSV de estoque encontrado em {caminho_pasta}")
        return pd.DataFrame()

    all_dfs = []
    float_cols = [
        'Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido',
        'PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB',
        'Taxa Originador Split', 'Taxa Split FIDC'
    ]
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']

    for file in arquivos_csv:
        try:
            # Sua função original para ler os arquivos de estoque
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()

            if not df.empty:
                for col in float_cols:
                    if col in df.columns:
                        df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
                for col in date_cols:
                    if col in df.columns:
                        df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
                all_dfs.append(df)
        except Exception as e:
            print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
            continue

    if not all_dfs:
        print("Nenhum dado de estoque foi carregado com sucesso.")
        return pd.DataFrame()

    df_final = pd.concat(all_dfs, ignore_index=True)
    print("Dados de estoque consolidados com sucesso.")
    return df_final

# --- Bloco de Inspeção e Depuração ---
if __name__ == "__main__":
    # 1. Executa a sua função para carregar os dados
    df_estoque = processar_dados_estoque(PATH_ESTOQUE)

    # 2. Inicia a análise se os dados foram carregados
    if not df_estoque.empty:
        print("\n--- INICIANDO ANÁLISE DO ESTOQUE CONSOLIDADO ---")

        # Análise 1: Dimensões do DataFrame
        print(f"\n[1] Formato do DataFrame: {df_estoque.shape[0]} linhas e {df_estoque.shape[1]} colunas.")

        # Análise 2: Lista de colunas
        print("\n[2] Colunas encontradas:")
        print(df_estoque.columns.tolist())

        # Análise 3: Tipos de dados (para verificar se as conversões funcionaram)
        print("\n[3] Tipos de dados (info):")
        df_estoque.info()

        # Análise 4: Amostra dos dados
        print("\n[4] Amostra das 5 primeiras linhas:")
        print(df_estoque.head())

        # Análise 5: Foco nas colunas-chave para o fluxo de caixa
        print("\n[5] Análise das colunas-chave para a projeção:")
        
        # Verifica se as colunas essenciais existem
        coluna_data = 'Data Vencimento'
        coluna_valor = 'Valor Presente' # Vamos usar o Valor Presente como referência inicial

        if coluna_data in df_estoque.columns:
            print(f"\n  - Análise da coluna '{coluna_data}':")
            # Remove valores nulos para análise de datas
            datas_validas = df_estoque[coluna_data].dropna()
            if not datas_validas.empty:
                print(f"    - Primeira data de vencimento: {datas_validas.min().strftime('%d/%m/%Y')}")
                print(f"    - Última data de vencimento: {datas_validas.max().strftime('%d/%m/%Y')}")
            else:
                print("    - Nenhuma data de vencimento válida encontrada.")
        else:
            print(f"\n  - ATENÇÃO: Coluna '{coluna_data}' não encontrada!")

        if coluna_valor in df_estoque.columns:
            print(f"\n  - Análise estatística da coluna '{coluna_valor}':")
            print(df_estoque[coluna_valor].describe().apply("{:,.2f}".format))
        else:
            print(f"\n  - ATENÇÃO: Coluna '{coluna_valor}' não encontrada!")

        print("\n--- FIM DA ANÁLISE ---")


Carregando dados de estoque...


C:\Users\Leo\AppData\Local\Temp\ipykernel_26824\779620343.py:37: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)
C:\Users\Leo\AppData\Local\Temp\ipykernel_26824\779620343.py:37: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df[col] = pd.to_datetime(df[col], errors='coerce', dayfirst=True)


Dados de estoque consolidados com sucesso.

--- INICIANDO ANÁLISE DO ESTOQUE CONSOLIDADO ---

[1] Formato do DataFrame: 1644304 linhas e 33 colunas.

[2] Colunas encontradas:
['CCB', 'SEU NUMERO', 'PARCELA', 'Nome Cliente', 'CPF Cliente', 'FIDC', 'Cedente', 'Data Aquisicao', 'Data Vencimento', 'Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido', 'PDD Total', 'Pagamento Parcial', 'Data Referencia', 'Prazo', 'Convênio Formatado', 'Produto', 'Promotora', 'Tipo do Contrato', 'Originador', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB', 'Taxa Originador Split', 'Taxa Split FIDC', 'Data de Nascimento', 'UF', 'CAPAG', 'Status', 'Faixa Vencido', 'Nota PDD']

[3] Tipos de dados (info):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1644304 entries, 0 to 1644303
Data columns (total 33 columns):
 #   Column                   Non-Null Count    Dtype         
---  ------                   --------------    -----         
 0   CCB                      1644304 non-null  int64 

In [4]:
import json
import pandas as pd
from datetime import datetime, timedelta
import os
import glob

# --- FUNÇÕES DE CARREGAMENTO DE DADOS ---

def carregar_dados_carteira(caminho_arquivo):
    """Carrega os dados da carteira a partir de um arquivo JSON."""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        dados = json.load(f)[0]
    return dados

def processar_dados_estoque(caminho_pasta):
    """Lê e consolida os arquivos de estoque de uma pasta (sua função)."""
    print("\nCarregando dados de estoque...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv:
        print(f"AVISO: Nenhum arquivo CSV de estoque encontrado em {caminho_pasta}")
        return pd.DataFrame()

    all_dfs = []
    # ... (O restante da sua função continua aqui, exatamente como antes)
    float_cols = [
        'Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido',
        'PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB',
        'Taxa Originador Split', 'Taxa Split FIDC'
    ]
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']

    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()

            if not df.empty:
                for col in float_cols:
                    if col in df.columns:
                        df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
                for col in date_cols:
                    if col in df.columns:
                        # Adicionando format para ser mais explícito e remover o warning
                        df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception as e:
            print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
            continue

    if not all_dfs:
        print("Nenhum dado de estoque foi carregado com sucesso.")
        return pd.DataFrame()

    df_final = pd.concat(all_dfs, ignore_index=True)
    print("Dados de estoque consolidados com sucesso.")
    return df_final

# --- FUNÇÕES DE PROJEÇÃO ---

def extrair_fluxos_de_caixa(dados_carteira, df_estoque):
    """Extrai e organiza todos os eventos de fluxo de caixa."""
    fluxos = []
    data_base = datetime.strptime(dados_carteira['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # 1. Disponibilidades (Entradas) - DA CARTEIRA JSON
    # ... (extração da Conta Corrente, Fundos Investidos e Renda Fixa como antes)
    disponibilidade = next((ativo['Disponibilidade'] for ativo in dados_carteira['ativos'] if 'Disponibilidade' in ativo and ativo['Disponibilidade']), None)
    if disponibilidade:
        saldo_cc = disponibilidade['ativos'][0]['saldo']
        fluxos.append({'data': data_base, 'descricao': 'Conta Corrente (Saldo Inicial)', 'valor': saldo_cc, 'tipo': 'Disponibilidade'})

    cotas = next((ativo['Cotas'] for ativo in dados_carteira['ativos'] if 'Cotas' in ativo and ativo['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos.append({'data': data_base + timedelta(days=1), 'descricao': f"Resgate Fundo '{cota['titulo']}'", 'valor': cota['valorBruto'], 'tipo': 'Disponibilidade'})
            
    renda_fixa = next((ativo['RendaFixa'] for ativo in dados_carteira['ativos'] if 'RendaFixa' in ativo and ativo['RendaFixa']), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']:
            data_vencimento = datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d')
            fluxos.append({'data': data_vencimento, 'descricao': f"Venc. RF '{titulo['codigoCustodia']}'", 'valor': titulo['mercadoAtual'], 'tipo': 'Disponibilidade'})

    # 2. Disponibilidades (Entradas) - DO ESTOQUE
    if not df_estoque.empty:
        print("Processando e adicionando fluxos do estoque...")
        # FILTRAGEM CRÍTICA
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento'])
        estoque_valido = estoque_valido[estoque_valido['Status'] == 'A vencer']
        
        # Agrupa por dia para obter o total de recebimentos diários
        fluxo_estoque = estoque_valido.groupby('Data Vencimento')['Valor Presente'].sum().reset_index()
        
        for index, row in fluxo_estoque.iterrows():
            fluxos.append({
                'data': row['Data Vencimento'],
                'descricao': 'Recebimento Estoque',
                'valor': row['Valor Presente'],
                'tipo': 'Disponibilidade'
            })
        print(f"{len(fluxo_estoque)} dias com recebimentos do estoque foram adicionados à projeção.")

    # 3. Necessidades (Saídas) - DA CARTEIRA JSON
    # ... (extração das Despesas e Amortização de Cotas como antes)
    passivos = dados_carteira.get('passivos', [])
    if passivos:
        for despesa in passivos[0].get('ativos', []):
            data_pagamento = data_base + timedelta(days=30)
            if 'DEZ25' in despesa['despesa']:
                data_pagamento = datetime(2025, 12, 15)
            fluxos.append({'data': data_pagamento, 'descricao': f"Despesa: {despesa['despesa']}", 'valor': despesa['valor'], 'tipo': 'Necessidade'})

    carteira_senior = next((c for c in dados_carteira['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            while data_amortizacao.weekday() >= 5:
                data_amortizacao += timedelta(days=1)
            fluxos.append({'data': data_amortizacao, 'descricao': f"Amortização Cota Sênior ({i+1}/36)", 'valor': valor_amortizacao, 'tipo': 'Necessidade'})
            ano = data_amortizacao.year
            mes = data_amortizacao.month + 1
            if mes > 12: mes = 1; ano += 1
            data_amortizacao = data_amortizacao.replace(year=ano, month=mes)

    return fluxos


def criar_tabela_projecao(fluxos, data_base):
    # ... (Esta função continua a mesma de antes)
    if not fluxos:
        print("Nenhum fluxo de caixa foi extraído.")
        return

    df = pd.DataFrame(fluxos)
    df_fluxo_diario = df.groupby('data')['valor'].sum().reset_index()
    
    # Garantir que a data máxima considera o vencimento mais longo do estoque
    data_fim_projecao = df_fluxo_diario['data'].max()
    idx = pd.date_range(start=data_base, end=data_fim_projecao)
    df_fluxo_diario = df_fluxo_diario.set_index('data').reindex(idx, fill_value=0).reset_index().rename(columns={'index':'data'})
    
    df_fluxo_diario['saldo_projetado'] = df_fluxo_diario['valor'].cumsum()
    df_fluxo_diario = df_fluxo_diario.set_index('data')

    prazos_dias = {'D+0': 0, 'D+1': 1, 'D+30': 30, 'D+90': 90, 'D+180': 180, 'D+365': 365}
    datas_amortizacao = sorted([f['data'] for f in fluxos if 'Amortização' in f['descricao']])
    if datas_amortizacao:
        prazos_dias[f"1ª Amort."] = (datas_amortizacao[0] - data_base).days
    
    projecao = []
    for nome_prazo, dias in prazos_dias.items():
        data_projecao = data_base + timedelta(days=dias)
        saldo = df_fluxo_diario.asof(data_projecao)['saldo_projetado'] if data_projecao <= data_fim_projecao else df_fluxo_diario.iloc[-1]['saldo_projetado']
        projecao.append({'Prazo': nome_prazo, 'Data': data_projecao.strftime('%d/%m/%Y'), 'Saldo Projetado': f"R$ {saldo:,.2f}"})

    return pd.DataFrame(projecao)


# --- EXECUÇÃO PRINCIPAL ---
PATH_JSON_CARTEIRA = "C:\\Users\\Leo\\Downloads\\carteira_52203615000119_20250901.json"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"

# 1. Carregar todas as fontes de dados
dados_carteira = carregar_dados_carteira(PATH_JSON_CARTEIRA)
df_estoque = processar_dados_estoque(PATH_ESTOQUE)

# 2. Extrair e combinar os fluxos de caixa
lista_fluxos = extrair_fluxos_de_caixa(dados_carteira, df_estoque)

# 3. Gerar a tabela final
data_referencia = datetime.strptime(dados_carteira['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
tabela_final = criar_tabela_projecao(lista_fluxos, data_referencia)

print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (COM ESTOQUE) ---")
print(tabela_final.to_string())


Carregando dados de estoque...
Dados de estoque consolidados com sucesso.
Processando e adicionando fluxos do estoque...
2422 dias com recebimentos do estoque foram adicionados à projeção.

--- PROJEÇÃO DE FLUXO DE CAIXA (COM ESTOQUE) ---
       Prazo        Data   Saldo Projetado
0        D+0  01/09/2025       R$ 5,292.37
1        D+1  02/09/2025   R$ 1,183,417.67
2       D+30  01/10/2025  R$ -3,254,323.10
3       D+90  30/11/2025   R$ 7,420,859.95
4      D+180  28/02/2026  R$ 20,420,395.53
5      D+365  01/09/2026  R$ 35,699,359.88
6  1ª Amort.  16/01/2026  R$ 14,070,295.29


In [1]:
# baixando os json das carteiras: 
import requests
import json
import os
from datetime import date
from dateutil.relativedelta import relativedelta
import time

# --- CONFIGURAÇÕES ---
CNPJS_DOS_FUNDOS = ["52203615000119"]
TOKENS_DE_ACESSO = [
    "96cc2224-0c2f-454f-b785-db8dd204f559",
    "d1bd877a-8b21-4c29-b005-dd5540d38673",
    "ad647ac8-188d-4d50-a6ff-6ef2524ac3ca"
]
MEU_CPF = "10142836982"

# Pasta onde os arquivos JSON serão salvos
PASTA_SAIDA = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"

# --- FIM DAS CONFIGURAÇÕES ---

URL_AUTENTICACAO = "https://apis.vortx.com.br/vxlogin/api/user/AuthUserApi"
URL_BASE_API = "https://apis.vortx.com.br"

def gerar_token_jwt(token_de_acesso, cpf):
    print(f"Tentando gerar token JWT com o token de acesso que termina em: ...{token_de_acesso[-6:]}")
    payload = {"token": token_de_acesso, "login": cpf}
    try:
        response = requests.post(URL_AUTENTICACAO, json=payload, timeout=15)
        if response.status_code == 200:
            jwt_token = response.json().get("token")
            if jwt_token:
                print(">>> Sucesso! Token JWT temporário gerado.")
                return jwt_token
        print(f"FALHA na autenticação. Status: {response.status_code}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"ERRO DE CONEXÃO ao tentar autenticar: {e}")
        return None

def obter_carteira_json(jwt_token, lista_cnpjs, data, pasta_destino):
    print(f"\nBuscando carteira para {', '.join(lista_cnpjs)} na data {data}")
    url_carteira = f"{URL_BASE_API}/carteira-liberada/buscarCarteiraJSON"
    headers = {"Authorization": f"Bearer {jwt_token}"}
    params = {"cnpjFundos[]": lista_cnpjs, "dataCarteira": data}

    try:
        response = requests.get(url_carteira, headers=headers, params=params, timeout=45)
        if response.status_code != 200:
            print(f"FALHA ao obter dados da carteira para {data}. Status: {response.status_code}")
            return False

        dados_carteira = response.json()
        
        # Verifica se a resposta não está vazia
        if not dados_carteira or not dados_carteira[0].get('carteiras'):
            print(f"AVISO: Dados da carteira para {data} vieram vazios. Pulando.")
            return False

        cnpjs_str = "_".join(lista_cnpjs)
        nome_arquivo = f"carteira_{cnpjs_str}_{data.replace('-', '')}.json"
        caminho_completo = os.path.join(pasta_destino, nome_arquivo)

        with open(caminho_completo, 'w', encoding='utf-8') as f:
            json.dump(dados_carteira, f, indent=4, ensure_ascii=False)
        
        print(f"✅ Sucesso! Carteira de {data} salva em: '{caminho_completo}'")
        return True
    except requests.exceptions.RequestException as e:
        print(f"ERRO DE CONEXÃO ao buscar a carteira de {data}: {e}")
        return False

# --- Bloco Principal de Execução ---
if __name__ == "__main__":
    os.makedirs(PASTA_SAIDA, exist_ok=True) # Garante que a pasta de destino exista

    # Gera a lista de datas (primeiro dia de cada mês, do último ano até hoje)
    datas_para_buscar = []
    data_fim = date.today()
    data_inicio = data_fim - relativedelta(years=1)
    
    data_corrente = data_fim
    while data_corrente >= data_inicio:
        # Usamos o primeiro dia do mês para padronizar
        datas_para_buscar.append(data_corrente.replace(day=1).strftime("%Y-%m-%d"))
        data_corrente -= relativedelta(months=1)
    
    print(f"Serão buscadas {len(datas_para_buscar)} carteiras, de {data_inicio.strftime('%Y-%m')} a {data_fim.strftime('%Y-%m')}.")

    jwt_token_gerado = None
    # Tenta obter um token JWT válido primeiro
    for token_acesso in TOKENS_DE_ACESSO:
        jwt_token_gerado = gerar_token_jwt(token_acesso, MEU_CPF)
        if jwt_token_gerado:
            break
    
    if not jwt_token_gerado:
        print("\n❌ Todas as tentativas de autenticação falharam. Saindo do script.")
        exit()

    # Itera sobre as datas e baixa cada carteira
    for data_carteira in datas_para_buscar:
        sucesso_busca = obter_carteira_json(jwt_token_gerado, CNPJS_DOS_FUNDOS, data_carteira, PASTA_SAIDA)
        if not sucesso_busca:
            print(f"Não foi possível obter a carteira para {data_carteira}. Continuando...")
        time.sleep(2) # Pausa de 2 segundos para não sobrecarregar a API

    print("\n--- Download do histórico concluído ---")

Serão buscadas 12 carteiras, de 2024-09 a 2025-09.
Tentando gerar token JWT com o token de acesso que termina em: ...04f559
>>> Sucesso! Token JWT temporário gerado.

Buscando carteira para 52203615000119 na data 2025-09-01
✅ Sucesso! Carteira de 2025-09-01 salva em: 'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras\carteira_52203615000119_20250901.json'

Buscando carteira para 52203615000119 na data 2025-08-01
✅ Sucesso! Carteira de 2025-08-01 salva em: 'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras\carteira_52203615000119_20250801.json'

Buscando carteira para 52203615000119 na data 2025-07-01
✅ Sucesso! Carteira de 2025-07-01 salva em: 'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras\carteira_52203615000119_20250701.json'

Buscando carteira para 52203615000119 na data 2025-06-01
FALHA ao obter dados da carteira para 2025-06-01. Status: 500
Não foi possível obter a carteira para 2025-06-01. Continuan

In [2]:
# tentando projetar para o futuro: 
import json
import pandas as pd
from datetime import datetime, timedelta
import os
import glob
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS DOS DADOS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"


# --- FUNÇÕES DE CARREGAMENTO (como antes) ---
def carregar_dados_carteira(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        dados = json.load(f)[0]
    return dados

def processar_dados_estoque(caminho_pasta):
    # ... (função completa que você já tem)
    print("\nCarregando dados de estoque...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    all_dfs = []
    float_cols = ['Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido','PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB','Taxa Originador Split', 'Taxa Split FIDC']
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                for col in float_cols:
                    if col in df.columns: df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
                for col in date_cols:
                    if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception as e: print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
    if not all_dfs: return pd.DataFrame()
    df_final = pd.concat(all_dfs, ignore_index=True)
    print("Dados de estoque consolidados com sucesso.")
    return df_final


# --- NOVA FUNÇÃO PARA PROJETAR DESPESAS ---
def projetar_despesas_futuras(caminho_pasta_historico, data_base):
    """
    Lê o histórico de carteiras, calcula a média de despesas recorrentes
    e projeta essas despesas para os próximos 12 meses.
    """
    print("Analisando histórico de despesas...")
    arquivos_json = glob.glob(os.path.join(caminho_pasta_historico, "*.json"))
    historico_despesas = []

    for arquivo in arquivos_json:
        dados = carregar_dados_carteira(arquivo)
        data_arquivo = datetime.strptime(dados['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
        passivos = dados.get('passivos', [])
        if passivos and passivos[0].get('ativos'):
            for despesa in passivos[0]['ativos']:
                historico_despesas.append({
                    'data': data_arquivo,
                    'categoria': despesa['despesa'],
                    'valor': despesa['valor']
                })
    
    if not historico_despesas:
        return []

    df_despesas = pd.DataFrame(historico_despesas)
    
    # Identifica despesas recorrentes (ex: que aparecem em mais de 3 meses)
    contagem_meses = df_despesas.groupby('categoria')['data'].nunique()
    categorias_recorrentes = contagem_meses[contagem_meses > 3].index.tolist()
    
    # Calcula a média mensal para despesas recorrentes
    media_recorrentes = df_despesas[df_despesas['categoria'].isin(categorias_recorrentes)].groupby('categoria')['valor'].mean()
    
    fluxos_despesas_projetadas = []
    print("Projetando despesas recorrentes com base na média histórica:")
    print(media_recorrentes)

    # Projeta as despesas recorrentes para os próximos 12 meses
    for i in range(1, 13):
        data_projecao = data_base + relativedelta(months=i)
        for categoria, valor_medio in media_recorrentes.items():
             fluxos_despesas_projetadas.append({
                'data': data_projecao,
                'descricao': f"Projeção Despesa: {categoria}",
                'valor': valor_medio,
                'tipo': 'Necessidade'
            })
            
    # Adiciona despesas pontuais (ex: Auditoria) que ainda não ocorreram
    despesas_pontuais = df_despesas[~df_despesas['categoria'].isin(categorias_recorrentes)]
    if 'AUDIT DEZ25' in despesas_pontuais['categoria'].values:
        fluxos_despesas_projetadas.append({
            'data': datetime(2025, 12, 15),
            'descricao': "Despesa: AUDIT DEZ25",
            'valor': df_despesas[df_despesas['categoria'] == 'AUDIT DEZ25']['valor'].iloc[0],
            'tipo': 'Necessidade'
        })

    return fluxos_despesas_projetadas

# --- FUNÇÃO PRINCIPAL DE FLUXO DE CAIXA (ADAPTADA) ---
def extrair_fluxos_de_caixa(dados_carteira_recente, df_estoque, fluxos_despesas_projetadas):
    """Extrai fluxos da carteira mais recente e combina com projeções."""
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # 1. Ativos da carteira mais recente (Renda Fixa, Cotas, Caixa)
    # ... (lógica idêntica à anterior)
    disponibilidade = next((a['Disponibilidade'] for a in dados_carteira_recente['ativos'] if 'Disponibilidade' in a and a['Disponibilidade']), None)
    if disponibilidade: fluxos.append({'data': data_base, 'descricao': 'Conta Corrente (Saldo Inicial)', 'valor': disponibilidade['ativos'][0]['saldo'], 'tipo': 'Disponibilidade'})
    cotas = next((a['Cotas'] for a in dados_carteira_recente['ativos'] if 'Cotas' in a and a['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']: fluxos.append({'data': data_base + timedelta(days=1), 'descricao': f"Resgate Fundo '{cota['titulo']}'", 'valor': cota['valorBruto'], 'tipo': 'Disponibilidade'})
    renda_fixa = next((a['RendaFixa'] for a in dados_carteira_recente['ativos'] if 'RendaFixa' in a and a['RendaFixa']), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']: fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'descricao': f"Venc. RF '{titulo['codigoCustodia']}'", 'valor': titulo['mercadoAtual'], 'tipo': 'Disponibilidade'})

    # 2. Entradas do Estoque
    # ... (lógica idêntica à anterior)
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento'])
        estoque_valido = estoque_valido[estoque_valido['Status'] == 'A vencer']
        fluxo_estoque = estoque_valido.groupby('Data Vencimento')['Valor Presente'].sum().reset_index()
        for _, row in fluxo_estoque.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'descricao': 'Recebimento Estoque', 'valor': row['Valor Presente'], 'tipo': 'Disponibilidade'})

    # 3. Adiciona as despesas PROJETADAS
    fluxos.extend(fluxos_despesas_projetadas)

    # 4. Saídas de Amortização (da carteira mais recente)
    # ... (lógica idêntica à anterior)
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            while data_amortizacao.weekday() >= 5: data_amortizacao += timedelta(days=1)
            fluxos.append({'data': data_amortizacao, 'descricao': f"Amortização Cota Sênior ({i+1}/36)", 'valor': valor_amortizacao, 'tipo': 'Necessidade'})
            ano, mes = (data_amortizacao.year, data_amortizacao.month + 1)
            if mes > 12: mes, ano = (1, ano + 1)
            data_amortizacao = data_amortizacao.replace(year=ano, month=mes)

    return fluxos

def criar_tabela_projecao(fluxos, data_base):
    # ... (função idêntica à anterior)
    if not fluxos: return pd.DataFrame()
    df = pd.DataFrame(fluxos)
    df_fluxo_diario = df.groupby('data')['valor'].sum().reset_index()
    data_fim_projecao = df_fluxo_diario['data'].max()
    idx = pd.date_range(start=data_base, end=data_fim_projecao)
    df_fluxo_diario = df_fluxo_diario.set_index('data').reindex(idx, fill_value=0).reset_index().rename(columns={'index':'data'})
    df_fluxo_diario['saldo_projetado'] = df_fluxo_diario['valor'].cumsum()
    df_fluxo_diario = df_fluxo_diario.set_index('data')
    prazos_dias = {'D+0': 0, 'D+1': 1, 'D+30': 30, 'D+90': 90, 'D+180': 180, 'D+365': 365}
    datas_amortizacao = sorted([f['data'] for f in fluxos if 'Amortização' in f['descricao']])
    if datas_amortizacao: prazos_dias[f"1ª Amort."] = (datas_amortizacao[0] - data_base).days
    projecao = []
    for nome_prazo, dias in prazos_dias.items():
        data_projecao = data_base + timedelta(days=dias)
        saldo = df_fluxo_diario.asof(data_projecao)['saldo_projetado'] if data_projecao <= data_fim_projecao else np.nan
        projecao.append({'Prazo': nome_prazo, 'Data': data_projecao.strftime('%d/%m/%Y'), 'Saldo Projetado': f"R$ {saldo:,.2f}"})
    return pd.DataFrame(projecao)


# --- EXECUÇÃO PRINCIPAL ---
if __name__ == "__main__":
    # 1. Encontra o arquivo de carteira mais recente
    arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))
    if not arquivos_carteira:
        print(f"ERRO: Nenhum arquivo de carteira encontrado em {PATH_CARTEIRAS_HISTORICO}")
        exit()
    arquivo_recente = max(arquivos_carteira, key=os.path.getctime)
    print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_recente)}")

    # 2. Carregar todas as fontes de dados
    dados_carteira_recente = carregar_dados_carteira(arquivo_recente)
    df_estoque = processar_dados_estoque(PATH_ESTOQUE)
    data_referencia = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    # 3. Projetar despesas com base no histórico
    fluxos_despesas = projetar_despesas_futuras(PATH_CARTEIRAS_HISTORICO, data_referencia)
    
    # 4. Extrair e combinar todos os fluxos de caixa
    lista_fluxos_final = extrair_fluxos_de_caixa(dados_carteira_recente, df_estoque, fluxos_despesas)
    
    # 5. Gerar a tabela final
    tabela_final = criar_tabela_projecao(lista_fluxos_final, data_referencia)

    print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (COM PROJEÇÃO DE DESPESAS HISTÓRICAS) ---")
    print(tabela_final.to_string())

Usando a carteira mais recente para a projeção: carteira_52203615000119_20241101.json

Carregando dados de estoque...
Dados de estoque consolidados com sucesso.
Analisando histórico de despesas...
Projetando despesas recorrentes com base na média histórica:
categoria
AUDIT LASTRO      -8087.224
PEND GESTÃO         -49.690
PROV ADM         -23292.658
PROV GESTÃO      -84061.130
TX ESCRT FIX      -2185.912
VALID          -3198209.798
Name: valor, dtype: float64

--- PROJEÇÃO DE FLUXO DE CAIXA (COM PROJEÇÃO DE DESPESAS HISTÓRICAS) ---
   Prazo        Data   Saldo Projetado
0    D+0  01/11/2024       R$ 1,000.00
1    D+1  02/11/2024  R$ 46,964,837.28
2   D+30  01/12/2024  R$ 43,648,950.87
3   D+90  30/01/2025  R$ 40,333,064.46
4  D+180  30/04/2025  R$ 30,385,405.22
5  D+365  01/11/2025  R$ 11,087,341.57


In [4]:
# vou verificar o demonstrativo de caixa, para ver essas saídas gigantescas: 
import pandas as pd
import numpy as np

# --- CONFIGURAÇÕES ---
# 1. Verifique se o caminho para o seu arquivo está correto.
PATH_DEMONSTRATIVO = r"C:\Users\Leo\Downloads\130170-Demonstrativo_Caixa.xlsx"

# 2. Definimos um "valor relevante" para filtrar apenas grandes pagamentos.
#    Estamos procurando por saídas na casa dos 3.2 milhões.
LIMITE_VALOR_RELEVANTE = 500000  # Meio milhão de reais

# --- FIM DAS CONFIGURAÇÕES ---

try:
    print(f"Lendo o arquivo: {PATH_DEMONSTRATIVO}")
    
    # Etapa 1: Ler o arquivo Excel, pulando as linhas de cabeçalho inúteis.
    # Pelas prévias, os dados reais começam na linha 5 (índice 4).
    # Vamos ler sem cabeçalho e nomear as colunas manualmente.
    df_caixa = pd.read_excel(PATH_DEMONSTRATIVO, header=None, skiprows=4)

    # Etapa 2: Limpar e nomear as colunas com base na estrutura visual.
    df_caixa.columns = [
        'Titulo', 
        'Historico_1', 
        'Historico_2', 
        'Data', 
        'Tipo', 
        'Entrada', 
        'Saida', 
        'Saldo'
    ]

    # Etapa 3: Converter as colunas para os tipos corretos.
    # Converte a coluna 'Saida' de texto (ex: 'R$ 1.413,62') para número.
    def limpar_valor(valor):
        if isinstance(valor, str):
            # Remove 'R$ ', espaços, e troca ',' por '.'
            valor = valor.replace('R$ ', '').strip().replace('.', '').replace(',', '.')
        # Converte para numérico, tratando erros
        return pd.to_numeric(valor, errors='coerce')

    df_caixa['Saida_Num'] = df_caixa['Saida'].apply(limpar_valor)
    
    # Converte a coluna 'Data' para o formato de data
    df_caixa['Data'] = pd.to_datetime(df_caixa['Data'], errors='coerce', dayfirst=True)

    # Remove linhas que não são de transação (ex: linhas totalmente vazias)
    df_caixa.dropna(subset=['Data', 'Saida_Num'], how='all', inplace=True)
    
    # Preenche valores nulos em Saida_Num com 0 para poder filtrar
    df_caixa['Saida_Num'] = df_caixa['Saida_Num'].fillna(0)

    print("\n--- Análise de Saídas de Caixa Relevantes ---")
    print(f"Procurando por pagamentos (saídas) maiores que R$ {LIMITE_VALOR_RELEVANTE:,.2f}")

    # Etapa 4: Filtrar para encontrar apenas as grandes saídas de caixa.
    grandes_saidas = df_caixa[df_caixa['Saida_Num'] > LIMITE_VALOR_RELEVANTE].copy()

    # Etapa 5: Exibir os resultados.
    if grandes_saidas.empty:
        print("\n[CONCLUSÃO PRELIMINAR]")
        print(">>> Nenhuma grande saída de caixa (> R$ 500 mil) foi encontrada no período coberto pelo arquivo.")
        print("Isso fortalece a hipótese de que 'VALID' é uma provisão contábil e não uma saída de caixa mensal.")
    else:
        print("\n[RESULTADO] Encontradas as seguintes saídas de caixa relevantes:")
        # Seleciona e ordena as colunas mais importantes para a nossa análise
        resultado = grandes_saidas[['Data', 'Historico_1', 'Historico_2', 'Saida_Num']].sort_values(by='Saida_Num', ascending=False)
        print(resultado.to_string())
        
        print("\n[CONCLUSÃO PRELIMINAR]")
        print(">>> Analise a tabela acima. Se você NÃO vir pagamentos recorrentes na casa de ~R$ 3.2 milhões, a despesa 'VALID' provavelmente é uma provisão contábil.")
        print("    Se você vir esse padrão, então é uma saída de caixa real.")

except FileNotFoundError:
    print(f"ERRO: Arquivo não encontrado em '{PATH_DEMONSTRATIVO}'. Por favor, verifique o caminho.")
except Exception as e:
    print(f"Ocorreu um erro inesperado ao processar o arquivo: {e}")

Lendo o arquivo: C:\Users\Leo\Downloads\130170-Demonstrativo_Caixa.xlsx

--- Análise de Saídas de Caixa Relevantes ---
Procurando por pagamentos (saídas) maiores que R$ 500,000.00

[RESULTADO] Encontradas as seguintes saídas de caixa relevantes:
          Data                          Historico_1 Historico_2   Saida_Num
281 2025-08-05                         AQUISICAO DC  DC C AQUIS  1266597.82
205 2025-07-29                         AQUISICAO DC  DC C AQUIS   927801.55
192 2025-07-28  Compra Cota CRT 1442 FC FIRF BEYOND  BEYOND FCT   840170.70
49  2025-07-11                         AQUISICAO DC  DC C AQUIS   772639.52
149 2025-07-23                         AQUISICAO DC  DC C AQUIS   703998.73
93  2025-07-16  Compra Cota CRT 1442 FC FIRF BEYOND  BEYOND FCT   629409.33
175 2025-07-25                         AQUISICAO DC  DC C AQUIS   626931.70
83  2025-07-15                         AQUISICAO DC  DC C AQUIS   625809.90
268 2025-08-04  Compra Cota CRT 1442 FC FIRF BEYOND  BEYOND FCT   6200

In [5]:
import json
import pandas as pd
from datetime import datetime, timedelta
import os
import glob
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS DOS DADOS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"


# --- FUNÇÕES DE CARREGAMENTO (sem alterações) ---
def carregar_dados_carteira(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        dados = json.load(f)[0]
    return dados

def processar_dados_estoque(caminho_pasta):
    print("\nCarregando dados de estoque...")
    # ... (função completa sem alterações)
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    all_dfs = []
    float_cols = ['Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido','PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB','Taxa Originador Split', 'Taxa Split FIDC']
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                for col in float_cols:
                    if col in df.columns: df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
                for col in date_cols:
                    if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception as e: print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
    if not all_dfs: return pd.DataFrame()
    df_final = pd.concat(all_dfs, ignore_index=True)
    print("Dados de estoque consolidados com sucesso.")
    return df_final


# --- FUNÇÃO DE PROJEÇÃO DE DESPESAS (COM A CORREÇÃO) ---
def projetar_despesas_futuras(caminho_pasta_historico, data_base):
    print("Analisando histórico de despesas...")
    arquivos_json = glob.glob(os.path.join(caminho_pasta_historico, "*.json"))
    historico_despesas = []

    for arquivo in arquivos_json:
        dados = carregar_dados_carteira(arquivo)
        data_arquivo = datetime.strptime(dados['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
        passivos = dados.get('passivos', [])
        if passivos and passivos[0].get('ativos'):
            for despesa in passivos[0]['ativos']:
                historico_despesas.append({
                    'data': data_arquivo,
                    'categoria': despesa['despesa'],
                    'valor': despesa['valor']
                })
    
    if not historico_despesas:
        return []

    df_despesas = pd.DataFrame(historico_despesas)
    
    contagem_meses = df_despesas.groupby('categoria')['data'].nunique()
    categorias_recorrentes = contagem_meses[contagem_meses > 3].index.tolist()
    
    media_recorrentes = df_despesas[df_despesas['categoria'].isin(categorias_recorrentes)].groupby('categoria')['valor'].mean()
    
    # ####################################################################
    # ## ÚNICA ALTERAÇÃO - AQUI ESTÁ A CORREÇÃO ##
    # Remove a provisão 'VALID' do cálculo de despesas caixa mensais
    media_recorrentes = media_recorrentes.drop('VALID', errors='ignore') 
    # ####################################################################
    
    fluxos_despesas_projetadas = []
    print("\nProjetando despesas de CAIXA recorrentes (média histórica):")
    print(media_recorrentes)

    # Projeta as despesas recorrentes para os próximos 12 meses
    for i in range(1, 13):
        data_projecao = data_base + relativedelta(months=i)
        for categoria, valor_medio in media_recorrentes.items():
             fluxos_despesas_projetadas.append({
                'data': data_projecao,
                'descricao': f"Projeção Despesa: {categoria}",
                'valor': valor_medio,
                'tipo': 'Necessidade'
            })
            
    # Adiciona despesas pontuais que ainda não ocorreram
    # (Esta lógica pode ser melhorada se houver mais despesas pontuais)

    return fluxos_despesas_projetadas


# --- DEMAIS FUNÇÕES (sem alterações) ---
def extrair_fluxos_de_caixa(dados_carteira_recente, df_estoque, fluxos_despesas_projetadas):
    # ... (função idêntica à anterior)
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    disponibilidade = next((a['Disponibilidade'] for a in dados_carteira_recente['ativos'] if 'Disponibilidade' in a and a['Disponibilidade']), None)
    if disponibilidade: fluxos.append({'data': data_base, 'descricao': 'Conta Corrente (Saldo Inicial)', 'valor': disponibilidade['ativos'][0]['saldo'], 'tipo': 'Disponibilidade'})
    cotas = next((a['Cotas'] for a in dados_carteira_recente['ativos'] if 'Cotas' in a and a['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']: fluxos.append({'data': data_base + timedelta(days=1), 'descricao': f"Resgate Fundo '{cota['titulo']}'", 'valor': cota['valorBruto'], 'tipo': 'Disponibilidade'})
    renda_fixa = next((a['RendaFixa'] for a in dados_carteira_recente['ativos'] if 'RendaFixa' in a and a['RendaFixa']), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']: fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'descricao': f"Venc. RF '{titulo['codigoCustodia']}'", 'valor': titulo['mercadoAtual'], 'tipo': 'Disponibilidade'})
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento'])
        estoque_valido = estoque_valido[estoque_valido['Status'] == 'A vencer']
        fluxo_estoque = estoque_valido.groupby('Data Vencimento')['Valor Presente'].sum().reset_index()
        for _, row in fluxo_estoque.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'descricao': 'Recebimento Estoque', 'valor': row['Valor Presente'], 'tipo': 'Disponibilidade'})
    fluxos.extend(fluxos_despesas_projetadas)
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            while data_amortizacao.weekday() >= 5: data_amortizacao += timedelta(days=1)
            fluxos.append({'data': data_amortizacao, 'descricao': f"Amortização Cota Sênior ({i+1}/36)", 'valor': valor_amortizacao, 'tipo': 'Necessidade'})
            ano, mes = (data_amortizacao.year, data_amortizacao.month + 1)
            if mes > 12: mes, ano = (1, ano + 1)
            data_amortizacao = data_amortizacao.replace(year=ano, month=mes)
    return fluxos

def criar_tabela_projecao(fluxos, data_base):
    # ... (função idêntica à anterior)
    if not fluxos: return pd.DataFrame()
    df = pd.DataFrame(fluxos)
    df_fluxo_diario = df.groupby('data')['valor'].sum().reset_index()
    data_fim_projecao = df_fluxo_diario['data'].max() if not df_fluxo_diario.empty else data_base
    idx = pd.date_range(start=data_base, end=data_fim_projecao)
    df_fluxo_diario = df_fluxo_diario.set_index('data').reindex(idx, fill_value=0).reset_index().rename(columns={'index':'data'})
    df_fluxo_diario['saldo_projetado'] = df_fluxo_diario['valor'].cumsum()
    df_fluxo_diario = df_fluxo_diario.set_index('data')
    prazos_dias = {'D+0': 0, 'D+1': 1, 'D+30': 30, 'D+90': 90, 'D+180': 180, 'D+365': 365}
    datas_amortizacao = sorted([f['data'] for f in fluxos if 'Amortização' in f['descricao']])
    if datas_amortizacao: prazos_dias[f"1ª Amort."] = (datas_amortizacao[0] - data_base).days
    projecao = []
    for nome_prazo, dias in prazos_dias.items():
        data_projecao = data_base + timedelta(days=dias)
        saldo = df_fluxo_diario.asof(data_projecao)['saldo_projetado'] if data_projecao <= data_fim_projecao else np.nan
        projecao.append({'Prazo': nome_prazo, 'Data': data_projecao.strftime('%d/%m/%Y'), 'Saldo Projetado': f"R$ {saldo:,.2f}"})
    return pd.DataFrame(projecao)


# --- EXECUÇÃO PRINCIPAL ---
if __name__ == "__main__":
    arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))
    if not arquivos_carteira:
        print(f"ERRO: Nenhum arquivo de carteira encontrado em {PATH_CARTEIRAS_HISTORICO}")
        exit()
    arquivo_recente = max(arquivos_carteira, key=os.path.getctime)
    print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_recente)}")

    dados_carteira_recente = carregar_dados_carteira(arquivo_recente)
    df_estoque = processar_dados_estoque(PATH_ESTOQUE)
    data_referencia = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    fluxos_despesas = projetar_despesas_futuras(PATH_CARTEIRAS_HISTORICO, data_referencia)
    
    lista_fluxos_final = extrair_fluxos_de_caixa(dados_carteira_recente, df_estoque, fluxos_despesas)
    
    tabela_final = criar_tabela_projecao(lista_fluxos_final, data_referencia)

    print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (VERSÃO FINAL CORRIGIDA) ---")
    print(tabela_final.to_string())

Usando a carteira mais recente para a projeção: carteira_52203615000119_20241101.json

Carregando dados de estoque...
Dados de estoque consolidados com sucesso.
Analisando histórico de despesas...

Projetando despesas de CAIXA recorrentes (média histórica):
categoria
AUDIT LASTRO    -8087.224
PEND GESTÃO       -49.690
PROV ADM       -23292.658
PROV GESTÃO    -84061.130
TX ESCRT FIX    -2185.912
Name: valor, dtype: float64

--- PROJEÇÃO DE FLUXO DE CAIXA (VERSÃO FINAL CORRIGIDA) ---
   Prazo        Data   Saldo Projetado
0    D+0  01/11/2024       R$ 1,000.00
1    D+1  02/11/2024  R$ 46,964,837.28
2   D+30  01/12/2024  R$ 46,847,160.67
3   D+90  30/01/2025  R$ 46,729,484.05
4  D+180  30/04/2025  R$ 46,376,454.21
5  D+365  01/11/2025  R$ 49,465,859.14


In [6]:
import pandas as pd
import json
import os
import glob
from datetime import datetime, timedelta
import numpy as np

# --- CAMINHOS DOS DADOS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÕES DE CARREGAMENTO DE DADOS ---

def carregar_dados_carteira(caminho_arquivo):
    """Lê um arquivo JSON de carteira."""
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return json.load(f)[0]

def processar_dados_estoque(caminho_pasta):
    """Lê e consolida os arquivos de estoque."""
    print("\nCarregando dados de estoque...")
    # ... (função completa sem alterações)
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    all_dfs = []
    float_cols = ['Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido','PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB','Taxa Originador Split', 'Taxa Split FIDC']
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                for col in float_cols:
                    if col in df.columns: df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
                for col in date_cols:
                    if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception as e: print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
    if not all_dfs: return pd.DataFrame()
    df_final = pd.concat(all_dfs, ignore_index=True)
    print("Dados de estoque consolidados com sucesso.")
    return df_final

def processar_dados_despesas(caminho_pasta):
    """
    Lê e consolida todos os arquivos 'Despesas_Consolidadas.xlsx' de uma pasta e suas subpastas.
    """
    print("\nCarregando e consolidando arquivos de despesas...")
    # Usa glob para encontrar todos os arquivos com o nome exato, recursivamente
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    
    if not arquivos_excel:
        print(f"AVISO: Nenhum arquivo 'Despesas_Consolidadas.xlsx' encontrado em {caminho_pasta}")
        return pd.DataFrame()

    lista_df_despesas = []
    for arquivo in arquivos_excel:
        try:
            # Lê o arquivo, pulando as 6 primeiras linhas e usando a 7ª como cabeçalho
            df = pd.read_excel(arquivo, header=6) 
            lista_df_despesas.append(df)
        except Exception as e:
            print(f"Erro ao ler o arquivo de despesa {os.path.basename(arquivo)}: {e}")

    if not lista_df_despesas:
        print("Nenhum dado de despesa foi carregado.")
        return pd.DataFrame()

    df_consolidado = pd.concat(lista_df_despesas, ignore_index=True)
    
    # Limpeza e seleção das colunas importantes
    df_consolidado.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'}, inplace=True)
    df_consolidado['Data Pagamento'] = pd.to_datetime(df_consolidado['Data Pagamento'], errors='coerce')
    df_consolidado['Valor'] = pd.to_numeric(df_consolidado['Valor'], errors='coerce')
    
    # Garante que o valor da despesa seja negativo (saída de caixa)
    df_consolidado['Valor'] = -df_consolidado['Valor'].abs()

    # Remove linhas onde as informações essenciais são nulas
    df_final = df_consolidado[['Data Pagamento', 'Categoria', 'Valor']].dropna()
    
    print(f"{len(df_final)} despesas com data de pagamento foram carregadas.")
    return df_final


# --- FUNÇÕES DE PROJEÇÃO (ATUALIZADAS) ---

def extrair_fluxos_de_caixa(dados_carteira_recente, df_estoque, df_despesas_datadas):
    """Combina todos os fluxos de caixa de diferentes fontes."""
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # 1. Ativos da carteira mais recente (Caixa, Fundos, RF)
    # ... (lógica idêntica à anterior)
    disponibilidade = next((a['Disponibilidade'] for a in dados_carteira_recente['ativos'] if 'Disponibilidade' in a and a['Disponibilidade']), None)
    if disponibilidade: fluxos.append({'data': data_base, 'descricao': 'Conta Corrente (Saldo Inicial)', 'valor': disponibilidade['ativos'][0]['saldo'], 'tipo': 'Disponibilidade'})
    cotas = next((a['Cotas'] for a in dados_carteira_recente['ativos'] if 'Cotas' in a and a['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']: fluxos.append({'data': data_base + timedelta(days=1), 'descricao': f"Resgate Fundo '{cota['titulo']}'", 'valor': cota['valorBruto'], 'tipo': 'Disponibilidade'})
    renda_fixa = next((a['RendaFixa'] for a in dados_carteira_recente['ativos'] if 'RendaFixa' in a and a['RendaFixa']), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']: fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'descricao': f"Venc. RF '{titulo['codigoCustodia']}'", 'valor': titulo['mercadoAtual'], 'tipo': 'Disponibilidade'})


    # 2. Entradas do Estoque
    # ... (lógica idêntica à anterior)
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento'])
        estoque_valido = estoque_valido[estoque_valido['Status'] == 'A vencer']
        fluxo_estoque = estoque_valido.groupby('Data Vencimento')['Valor Presente'].sum().reset_index()
        for _, row in fluxo_estoque.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'descricao': 'Recebimento Estoque', 'valor': row['Valor Presente'], 'tipo': 'Disponibilidade'})

    # 3. Saídas de Despesas com Data de Pagamento
    if not df_despesas_datadas.empty:
        for _, despesa in df_despesas_datadas.iterrows():
            fluxos.append({
                'data': despesa['Data Pagamento'],
                'descricao': f"Despesa: {despesa['Categoria']}",
                'valor': despesa['Valor'],
                'tipo': 'Necessidade'
            })

    # 4. Saídas de Amortização
    # ... (lógica idêntica à anterior)
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            while data_amortizacao.weekday() >= 5: data_amortizacao += timedelta(days=1)
            fluxos.append({'data': data_amortizacao, 'descricao': f"Amortização Cota Sênior ({i+1}/36)", 'valor': valor_amortizacao, 'tipo': 'Necessidade'})
            ano, mes = (data_amortizacao.year, data_amortizacao.month + 1)
            if mes > 12: mes, ano = (1, ano + 1)
            data_amortizacao = data_amortizacao.replace(year=ano, month=mes)

    return fluxos

def criar_tabela_projecao(fluxos, data_base):
    # ... (função idêntica à anterior)
    if not fluxos: return pd.DataFrame()
    df = pd.DataFrame(fluxos)
    df_fluxo_diario = df.groupby('data')['valor'].sum().reset_index()
    data_fim_projecao = df_fluxo_diario['data'].max() if not df_fluxo_diario.empty else data_base
    idx = pd.date_range(start=data_base, end=data_fim_projecao)
    df_fluxo_diario = df_fluxo_diario.set_index('data').reindex(idx, fill_value=0).reset_index().rename(columns={'index':'data'})
    df_fluxo_diario['saldo_projetado'] = df_fluxo_diario['valor'].cumsum()
    df_fluxo_diario = df_fluxo_diario.set_index('data')
    prazos_dias = {'D+0': 0, 'D+1': 1, 'D+30': 30, 'D+90': 90, 'D+180': 180, 'D+365': 365}
    datas_amortizacao = sorted([f['data'] for f in fluxos if 'Amortização' in f['descricao']])
    if datas_amortizacao: prazos_dias[f"1ª Amort."] = (datas_amortizacao[0] - data_base).days
    projecao = []
    for nome_prazo, dias in prazos_dias.items():
        data_projecao = data_base + timedelta(days=dias)
        saldo = df_fluxo_diario.asof(data_projecao)['saldo_projetado'] if data_projecao <= data_fim_projecao else np.nan
        projecao.append({'Prazo': nome_prazo, 'Data': data_projecao.strftime('%d/%m/%Y'), 'Saldo Projetado': f"R$ {saldo:,.2f}"})
    return pd.DataFrame(projecao)


# --- EXECUÇÃO PRINCIPAL ---
if __name__ == "__main__":
    # Encontra o arquivo de carteira mais recente
    arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))
    if not arquivos_carteira:
        print(f"ERRO: Nenhum arquivo de carteira encontrado em {PATH_CARTEIRAS_HISTORICO}")
        exit()
    arquivo_recente = max(arquivos_carteira, key=os.path.getctime)
    print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_recente)}")

    # Carrega todas as fontes de dados
    dados_carteira_recente = carregar_dados_carteira(arquivo_recente)
    df_estoque = processar_dados_estoque(PATH_ESTOQUE)
    df_despesas = processar_dados_despesas(PATH_DESPESAS)
    
    data_referencia = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    # Extrai e combina todos os fluxos de caixa
    lista_fluxos_final = extrair_fluxos_de_caixa(dados_carteira_recente, df_estoque, df_despesas)
    
    # Gera a tabela final
    tabela_final = criar_tabela_projecao(lista_fluxos_final, data_referencia)

    print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (COM DATAS DE DESPESAS REAIS) ---")
    print(tabela_final.to_string())

Usando a carteira mais recente para a projeção: carteira_52203615000119_20241101.json

Carregando dados de estoque...
Dados de estoque consolidados com sucesso.

Carregando e consolidando arquivos de despesas...
4 despesas com data de pagamento foram carregadas.

--- PROJEÇÃO DE FLUXO DE CAIXA (COM DATAS DE DESPESAS REAIS) ---
   Prazo        Data   Saldo Projetado
0    D+0  01/11/2024       R$ 1,000.00
1    D+1  02/11/2024  R$ 46,964,837.28
2   D+30  01/12/2024  R$ 46,964,837.28
3   D+90  30/01/2025  R$ 46,964,837.28
4  D+180  30/04/2025  R$ 46,964,837.28
5  D+365  01/11/2025  R$ 50,649,089.25


In [7]:
import pandas as pd
import json
import os
import glob
from datetime import datetime
import numpy as np

# --- CAMINHOS DOS DADOS (sem alterações) ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÕES DE CARREGAMENTO (sem alterações) ---
def carregar_dados_carteira(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return json.load(f)[0]

def processar_dados_estoque(caminho_pasta):
    print("\nCarregando dados de estoque...")
    # ... (função completa sem alterações)
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    all_dfs = []
    float_cols = ['Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido','PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB','Taxa Originador Split', 'Taxa Split FIDC']
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                for col in float_cols:
                    if col in df.columns: df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
                for col in date_cols:
                    if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception as e: print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
    if not all_dfs: return pd.DataFrame()
    df_final = pd.concat(all_dfs, ignore_index=True)
    print("Dados de estoque consolidados com sucesso.")
    return df_final

def processar_dados_despesas(caminho_pasta):
    print("\nCarregando e consolidando arquivos de despesas...")
    # ... (função completa sem alterações)
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame()
    lista_df_despesas = []
    for arquivo in arquivos_excel:
        try:
            df = pd.read_excel(arquivo, header=6) 
            lista_df_despesas.append(df)
        except Exception as e: print(f"Erro ao ler o arquivo de despesa {os.path.basename(arquivo)}: {e}")
    if not lista_df_despesas: return pd.DataFrame()
    df_consolidado = pd.concat(lista_df_despesas, ignore_index=True)
    df_consolidado.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'}, inplace=True)
    df_consolidado['Data Pagamento'] = pd.to_datetime(df_consolidado['Data Pagamento'], errors='coerce')
    df_consolidado['Valor'] = pd.to_numeric(df_consolidado['Valor'], errors='coerce')
    df_consolidado['Valor'] = -df_consolidado['Valor'].abs()
    df_final = df_consolidado[['Data Pagamento', 'Categoria', 'Valor']].dropna()
    print(f"{len(df_final)} despesas com data de pagamento foram carregadas.")
    return df_final


# --- FUNÇÃO DE EXTRAÇÃO DE FLUXOS (COM NOVA CATEGORIZAÇÃO) ---

def extrair_fluxos_de_caixa(dados_carteira_recente, df_estoque, df_despesas_datadas):
    """Combina todos os fluxos de caixa e adiciona uma categoria simplificada para a decomposição."""
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # Saldo Inicial
    disponibilidade = next((a['Disponibilidade'] for a in dados_carteira_recente['ativos'] if 'Disponibilidade' in a and a['Disponibilidade']), None)
    if disponibilidade:
        fluxos.append({'data': data_base, 'categoria': 'Saldo Inicial', 'valor': disponibilidade['ativos'][0]['saldo']})

    # Outras Entradas (Fundos, RF)
    cotas = next((a['Cotas'] for a in dados_carteira_recente['ativos'] if 'Cotas' in a and a['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos.append({'data': data_base, 'categoria': 'Investimentos (Fundos/RF)', 'valor': cota['valorBruto']}) # Simplificado para D+0 na agregação mensal
    renda_fixa = next((a['RendaFixa'] for a in dados_carteira_recente['ativos'] if 'RendaFixa' in a and a['RendaFixa']), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']:
            fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Investimentos (Fundos/RF)', 'valor': titulo['mercadoAtual']})

    # Entradas do Estoque
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento'])
        estoque_valido = estoque_valido[estoque_valido['Status'] == 'A vencer']
        for _, row in estoque_valido.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Recebimento Estoque', 'valor': row['Valor Presente']})

    # Saídas de Despesas
    if not df_despesas_datadas.empty:
        for _, despesa in df_despesas_datadas.iterrows():
            fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas Operacionais', 'valor': despesa['Valor']})

    # Saídas de Amortização
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        # ... (lógica de data da amortização sem alterações)
        data_amortizacao = datetime(2026, 1, 16)
        # ...
        fluxos.append({'data': data_amortizacao, 'categoria': 'Amortização de Cotas', 'valor': valor_amortizacao})


    return pd.DataFrame(fluxos)


# --- NOVA FUNÇÃO PARA GERAR A TABELA DE DECOMPOSIÇÃO ---

def criar_tabela_decomposicao(df_fluxos, data_base):
    """Cria uma tabela mensal detalhada com a decomposição do fluxo de caixa."""
    if df_fluxos.empty:
        return "Nenhum fluxo de caixa para analisar."

    # Filtra fluxos a partir da data base
    df_fluxos = df_fluxos[df_fluxos['data'] >= data_base].copy()
    
    # Prepara o DataFrame para agregação mensal
    df_fluxos['mes'] = df_fluxos['data'].dt.to_period('M')

    # Agrupa por mês e categoria
    decomposicao = df_fluxos.pivot_table(index='mes', columns='categoria', values='valor', aggfunc='sum').fillna(0)
    
    # Garante que as colunas principais existam
    colunas_principais = ['Recebimento Estoque', 'Investimentos (Fundos/RF)', 'Despesas Operacionais', 'Amortização de Cotas']
    for col in colunas_principais:
        if col not in decomposicao.columns:
            decomposicao[col] = 0
            
    # Calcula totais e saldos
    decomposicao['Total Entradas'] = decomposicao['Recebimento Estoque'] + decomposicao['Investimentos (Fundos/RF)']
    decomposicao['Total Saídas'] = decomposicao['Despesas Operacionais'] + decomposicao['Amortização de Cotas']
    decomposicao['Fluxo de Caixa Mês'] = decomposicao['Total Entradas'] + decomposicao['Total Saídas']
    
    # Pega o saldo inicial da data base
    saldo_inicial_valor = df_fluxos[df_fluxos['categoria'] == 'Saldo Inicial']['valor'].sum()
    
    decomposicao['Saldo Final'] = decomposicao['Fluxo de Caixa Mês'].cumsum() + saldo_inicial_valor
    decomposicao['Saldo Inicial'] = decomposicao['Saldo Final'].shift(1).fillna(saldo_inicial_valor)
    
    # Formata para exibição
    colunas_para_mostrar = [
        'Saldo Inicial', 'Total Entradas', 'Recebimento Estoque', 'Total Saídas', 
        'Despesas Operacionais', 'Amortização de Cotas', 'Fluxo de Caixa Mês', 'Saldo Final'
    ]
    
    # Filtra colunas que realmente existem no dataframe final
    colunas_existentes = [col for col in colunas_para_mostrar if col in decomposicao.columns]
    
    # Exibe apenas os primeiros 12 meses da projeção para clareza
    decomposicao_final = decomposicao[colunas_existentes].head(12)
    
    return decomposicao_final.applymap(lambda x: f"{x:,.2f}")


# --- EXECUÇÃO PRINCIPAL ---
if __name__ == "__main__":
    arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))
    if not arquivos_carteira:
        print(f"ERRO: Nenhum arquivo de carteira encontrado em {PATH_CARTEIRAS_HISTORICO}")
        exit()
    arquivo_recente = max(arquivos_carteira, key=os.path.getctime)
    print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_recente)}")

    dados_carteira_recente = carregar_dados_carteira(arquivo_recente)
    df_estoque = processar_dados_estoque(PATH_ESTOQUE)
    df_despesas = processar_dados_despesas(PATH_DESPESAS)
    
    data_referencia = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    df_fluxos_final = extrair_fluxos_de_caixa(dados_carteira_recente, df_estoque, df_despesas)
    
    tabela_decomposicao = criar_tabela_decomposicao(df_fluxos_final, data_referencia)

    print("\n--- DECOMPOSIÇÃO MENSAL DO FLUXO DE CAIXA PROJETADO ---")
    print(tabela_decomposicao.to_string())

Usando a carteira mais recente para a projeção: carteira_52203615000119_20241101.json

Carregando dados de estoque...
Dados de estoque consolidados com sucesso.

Carregando e consolidando arquivos de despesas...
4 despesas com data de pagamento foram carregadas.

--- DECOMPOSIÇÃO MENSAL DO FLUXO DE CAIXA PROJETADO ---
categoria  Saldo Inicial Total Entradas Recebimento Estoque Total Saídas Despesas Operacionais Amortização de Cotas Fluxo de Caixa Mês     Saldo Final
mes                                                                                                                                                   
2024-11         1,000.00  46,963,837.28                0.00         0.00                  0.00                 0.00      46,963,837.28   46,964,837.28
2025-05    46,964,837.28           0.00                0.00  -228,889.26           -228,889.26                 0.00        -228,889.26   46,735,948.02
2025-10    46,735,948.02   3,913,141.23        3,913,141.23         0.00    

C:\Users\Leo\AppData\Local\Temp\ipykernel_22716\766262222.py:158: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return decomposicao_final.applymap(lambda x: f"{x:,.2f}")


In [9]:
import pandas as pd
import json
import os
import glob
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS DOS DADOS (sem alterações) ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÕES DE CARREGAMENTO (sem alterações) ---
def carregar_dados_carteira(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return json.load(f)[0]

def processar_dados_estoque(caminho_pasta):
    print("\nCarregando dados de estoque...")
    # ... (função completa sem alterações)
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    all_dfs = []
    float_cols = ['Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido','PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB','Taxa Originador Split', 'Taxa Split FIDC']
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                for col in float_cols:
                    if col in df.columns: df[col] = df[col].astype(str).str.replace(',', '.').astype(float)
                for col in date_cols:
                    if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception as e: print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
    if not all_dfs: return pd.DataFrame()
    df_final = pd.concat(all_dfs, ignore_index=True)
    print("Dados de estoque consolidados com sucesso.")
    return df_final

def processar_dados_despesas(caminho_pasta):
    print("\nCarregando e consolidando arquivos de despesas...")
    # ... (função completa sem alterações)
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame()
    lista_df_despesas = []
    for arquivo in arquivos_excel:
        try:
            df = pd.read_excel(arquivo, header=6) 
            lista_df_despesas.append(df)
        except Exception as e: print(f"Erro ao ler o arquivo de despesa {os.path.basename(arquivo)}: {e}")
    if not lista_df_despesas: return pd.DataFrame()
    df_consolidado = pd.concat(lista_df_despesas, ignore_index=True)
    df_consolidado.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'}, inplace=True)
    df_consolidado['Data Pagamento'] = pd.to_datetime(df_consolidado['Data Pagamento'], errors='coerce')
    df_consolidado['Valor'] = pd.to_numeric(df_consolidado['Valor'], errors='coerce')
    df_consolidado['Valor'] = -df_consolidado['Valor'].abs()
    df_final = df_consolidado[['Data Pagamento', 'Categoria', 'Valor']].dropna()
    print(f"{len(df_final)} despesas com data de pagamento foram carregadas.")
    return df_final


# --- FUNÇÃO DE EXTRAÇÃO DE FLUXOS (COM LÓGICA DE AMORTIZAÇÃO CORRIGIDA) ---
def extrair_fluxos_de_caixa(dados_carteira_recente, df_estoque, df_despesas_datadas):
    """Combina todos os fluxos de caixa e adiciona uma categoria simplificada."""
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    disponibilidade = next((a['Disponibilidade'] for a in dados_carteira_recente['ativos'] if 'Disponibilidade' in a and a['Disponibilidade']), None)
    if disponibilidade:
        fluxos.append({'data': data_base, 'categoria': 'Saldo Inicial', 'valor': disponibilidade['ativos'][0]['saldo']})

    cotas = next((a['Cotas'] for a in dados_carteira_recente['ativos'] if 'Cotas' in a and a['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos.append({'data': data_base, 'categoria': 'Investimentos (Fundos/RF)', 'valor': cota['valorBruto']})
            
    renda_fixa = next((a['RendaFixa'] for a in dados_carteira_recente['ativos'] if 'RendaFixa' in a and a['RendaFixa']), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']:
            fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Investimentos (Fundos/RF)', 'valor': titulo['mercadoAtual']})

    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento'])
        estoque_valido = estoque_valido[estoque_valido['Status'] == 'A vencer']
        for _, row in estoque_valido.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Recebimento Estoque', 'valor': row['Valor Presente']})

    if not df_despesas_datadas.empty:
        for _, despesa in df_despesas_datadas.iterrows():
            fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas Operacionais', 'valor': despesa['Valor']})

    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5:
                data_pagamento += pd.Timedelta(days=1)
            fluxos.append({'data': data_pagamento, 'categoria': 'Amortização de Cotas', 'valor': valor_amortizacao})

    return pd.DataFrame(fluxos)


# --- FUNÇÃO DE DECOMPOSIÇÃO (COM A CORREÇÃO DO ERRO) ---
def criar_tabela_decomposicao(df_fluxos, data_base):
    if df_fluxos.empty:
        return "Nenhum fluxo de caixa para analisar."

    df_fluxos = df_fluxos[df_fluxos['data'] >= data_base].copy()
    df_fluxos['mes'] = df_fluxos['data'].dt.to_period('M')
    
    decomposicao = df_fluxos.pivot_table(index='mes', columns='categoria', values='valor', aggfunc='sum').fillna(0)
    
    # ### CORREÇÃO DO BUG ESTÁ AQUI ###
    # Converte o 'datetime' do python para 'Timestamp' do pandas antes de usar '.to_period()'
    start_period = pd.Timestamp(data_base).to_period('M')
    end_period = df_fluxos['mes'].max()
    
    # Garante que o end_period seja válido antes de criar o range
    if pd.isna(end_period):
        end_period = start_period

    idx_meses = pd.period_range(start=start_period, end=end_period, freq='M')
    decomposicao = decomposicao.reindex(idx_meses, fill_value=0)
    
    colunas_principais = ['Recebimento Estoque', 'Investimentos (Fundos/RF)', 'Despesas Operacionais', 'Amortização de Cotas']
    for col in colunas_principais:
        if col not in decomposicao.columns:
            decomposicao[col] = 0
            
    decomposicao['Total Entradas'] = decomposicao['Recebimento Estoque'] + decomposicao['Investimentos (Fundos/RF)']
    decomposicao['Total Saídas'] = decomposicao['Despesas Operacionais'] + decomposicao['Amortização de Cotas']
    decomposicao['Fluxo de Caixa Mês'] = decomposicao['Total Entradas'] + decomposicao['Total Saídas']
    
    saldo_inicial_valor = df_fluxos[df_fluxos['categoria'] == 'Saldo Inicial']['valor'].sum()
    
    decomposicao['Saldo Final'] = decomposicao['Fluxo de Caixa Mês'].cumsum() + saldo_inicial_valor
    decomposicao['Saldo Inicial'] = decomposicao['Saldo Final'].shift(1).fillna(saldo_inicial_valor)
    
    colunas_para_mostrar = ['Saldo Inicial', 'Total Entradas', 'Recebimento Estoque', 'Total Saídas', 'Despesas Operacionais', 'Amortização de Cotas', 'Fluxo de Caixa Mês', 'Saldo Final']
    colunas_existentes = [col for col in colunas_para_mostrar if col in decomposicao.columns]
    
    decomposicao_final = decomposicao[colunas_existentes].head(24) # Mostra os primeiros 24 meses
    
    # Formatação final para melhor leitura
    return decomposicao_final.applymap(lambda x: f"{x:,.2f}")


# --- EXECUÇÃO PRINCIPAL ---
if __name__ == "__main__":
    arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))
    if not arquivos_carteira:
        print(f"ERRO: Nenhum arquivo de carteira encontrado em {PATH_CARTEIRAS_HISTORICO}")
        exit()
    arquivo_recente = max(arquivos_carteira, key=os.path.getctime)
    print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_recente)}")

    dados_carteira_recente = carregar_dados_carteira(arquivo_recente)
    df_estoque = processar_dados_estoque(PATH_ESTOQUE)
    df_despesas = processar_dados_despesas(PATH_DESPESAS)
    
    data_referencia = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    df_fluxos_final = extrair_fluxos_de_caixa(dados_carteira_recente, df_estoque, df_despesas)
    
    tabela_decomposicao = criar_tabela_decomposicao(df_fluxos_final, data_referencia)

    print("\n--- DECOMPOSIÇÃO MENSAL DO FLUXO DE CAIXA (VERSÃO CORRIGIDA) ---")
    print(tabela_decomposicao.to_string())

Usando a carteira mais recente para a projeção: carteira_52203615000119_20241101.json

Carregando dados de estoque...


Dados de estoque consolidados com sucesso.

Carregando e consolidando arquivos de despesas...
4 despesas com data de pagamento foram carregadas.

--- DECOMPOSIÇÃO MENSAL DO FLUXO DE CAIXA (VERSÃO CORRIGIDA) ---
categoria   Saldo Inicial Total Entradas Recebimento Estoque Total Saídas Despesas Operacionais Amortização de Cotas Fluxo de Caixa Mês     Saldo Final
2024-11          1,000.00  46,963,837.28                0.00         0.00                  0.00                 0.00      46,963,837.28   46,964,837.28
2024-12     46,964,837.28           0.00                0.00         0.00                  0.00                 0.00               0.00   46,964,837.28
2025-01     46,964,837.28           0.00                0.00         0.00                  0.00                 0.00               0.00   46,964,837.28
2025-02     46,964,837.28           0.00                0.00         0.00                  0.00                 0.00               0.00   46,964,837.28
2025-03     46,964,837.28    

C:\Users\Leo\AppData\Local\Temp\ipykernel_22716\2740654259.py:152: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return decomposicao_final.applymap(lambda x: f"{x:,.2f}")


In [10]:
import pandas as pd
import json
import os
import glob
from datetime import datetime

# --- CONFIGURAÇÕES ---
# Escolha dois arquivos de carteira que você baixou para definir o período da análise
PATH_CARTEIRA_INICIO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras\carteira_52203615000119_20241101.json"
PATH_CARTEIRA_FIM = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras\carteira_52203615000119_20250901.json" # Exemplo, use um arquivo que vc tenha

PATH_DEMONSTRATIVO = r"C:\Users\Leo\Downloads\130170-Demonstrativo_Caixa.xlsx"
# --- FIM DAS CONFIGURAções ---


def extrair_saldo_caixa_json(caminho_arquivo):
    """Extrai o saldo de caixa de um arquivo de carteira JSON."""
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            dados = json.load(f)[0]
        
        data_extracao = datetime.strptime(dados['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
        
        disponibilidade = next((a['Disponibilidade'] for a in dados['ativos'] if 'Disponibilidade' in a and a['Disponibilidade']), None)
        if disponibilidade and disponibilidade.get('ativos'):
            return disponibilidade['ativos'][0]['saldo'], data_extracao
        else: # Se não houver 'Disponibilidade', assume R$ 0.00
            return 0.0, data_extracao

    except (FileNotFoundError, IndexError, KeyError) as e:
        print(f"Erro ao processar o arquivo {os.path.basename(caminho_arquivo)}: {e}")
        return None, None

def calcular_fluxo_real_demonstrativo(caminho_arquivo, data_inicio, data_fim):
    """Lê o demonstrativo e calcula o fluxo de caixa líquido em um período."""
    try:
        df_caixa = pd.read_excel(caminho_arquivo, header=None, skiprows=4)
        df_caixa.columns = ['Titulo', 'Historico_1', 'Historico_2', 'Data', 'Tipo', 'Entrada', 'Saida', 'Saldo']

        def limpar_valor(valor):
            if isinstance(valor, str):
                valor = valor.replace('R$ ', '').strip().replace('.', '').replace(',', '.')
            return pd.to_numeric(valor, errors='coerce')

        df_caixa['Entrada_Num'] = df_caixa['Entrada'].apply(limpar_valor).fillna(0)
        df_caixa['Saida_Num'] = df_caixa['Saida'].apply(limpar_valor).fillna(0)
        df_caixa['Data'] = pd.to_datetime(df_caixa['Data'], errors='coerce', dayfirst=True)
        
        # Filtra o DataFrame para o período de interesse
        df_periodo = df_caixa[(df_caixa['Data'] >= data_inicio) & (df_caixa['Data'] <= data_fim)]
        
        total_entradas = df_periodo['Entrada_Num'].sum()
        total_saidas = df_periodo['Saida_Num'].sum()
        fluxo_liquido = total_entradas - total_saidas
        
        return fluxo_liquido, total_entradas, total_saidas

    except Exception as e:
        print(f"Erro ao processar o demonstrativo de caixa: {e}")
        return None, None, None

# --- EXECUÇÃO DO TESTE DE RECONCILIAÇÃO ---
if __name__ == "__main__":
    saldo_inicial, data_inicial = extrair_saldo_caixa_json(PATH_CARTEIRA_INICIO)
    saldo_final, data_final = extrair_saldo_caixa_json(PATH_CARTEIRA_FIM)

    if saldo_inicial is not None and saldo_final is not None:
        print("\n--- TESTE DE RECONCILIAÇÃO DE CAIXA ---")
        print(f"Analisando o período de {data_inicial.strftime('%d/%m/%Y')} a {data_final.strftime('%d/%m/%Y')}")
        
        # 1. Variação de caixa segundo os relatórios de carteira (JSON)
        variacao_esperada = saldo_final - saldo_inicial
        print("\n[1] Visão da Carteira (JSONs):")
        print(f"   - Saldo de Caixa Inicial...: R$ {saldo_inicial:,.2f}")
        print(f"   - Saldo de Caixa Final.....: R$ {saldo_final:,.2f}")
        print(f"   - VARIAÇÃO ESPERADA........: R$ {variacao_esperada:,.2f}")

        # 2. Fluxo de caixa segundo o demonstrativo (Excel)
        fluxo_real, entradas, saidas = calcular_fluxo_real_demonstrativo(PATH_DEMONSTRATIVO, data_inicial, data_final)
        
        if fluxo_real is not None:
            print("\n[2] Visão do Caixa Real (Demonstrativo Excel):")
            print(f"   - Total de Entradas........: R$ {entradas:,.2f}")
            print(f"   - Total de Saídas..........: R$ {saidas:,.2f}")
            print(f"   - FLUXO DE CAIXA REAL......: R$ {fluxo_real:,.2f}")

            # 3. Comparação Final
            diferenca = fluxo_real - variacao_esperada
            print("\n[3] Resultado da Reconciliação:")
            print(f"   - Diferença (Fluxo Real - Variação Esperada): R$ {diferenca:,.2f}")

            if abs(diferenca) < 1000: # Uma pequena tolerância para arredondamentos
                print("\n✅ CONCLUSÃO: Os dados são consistentes! A diferença é mínima.")
                print("Isso confirma que o saldo de caixa elevado é real, assumindo que o fundo pare de reinvestir.")
            else:
                print("\n⚠️ ALERTA: Há uma diferença relevante entre os relatórios.")
                print("Isso pode indicar que há transações de caixa não capturadas ou desalinhamento entre as datas dos relatórios.")


--- TESTE DE RECONCILIAÇÃO DE CAIXA ---
Analisando o período de 01/11/2024 a 01/09/2025

[1] Visão da Carteira (JSONs):
   - Saldo de Caixa Inicial...: R$ 1,000.00
   - Saldo de Caixa Final.....: R$ 5,292.37
   - VARIAÇÃO ESPERADA........: R$ 4,292.37

[2] Visão do Caixa Real (Demonstrativo Excel):
   - Total de Entradas........: R$ 13,621,819.33
   - Total de Saídas..........: R$ 13,642,601.12
   - FLUXO DE CAIXA REAL......: R$ -20,781.79

[3] Resultado da Reconciliação:
   - Diferença (Fluxo Real - Variação Esperada): R$ -25,074.16

⚠️ ALERTA: Há uma diferença relevante entre os relatórios.
Isso pode indicar que há transações de caixa não capturadas ou desalinhamento entre as datas dos relatórios.


In [11]:
import pandas as pd
import os
import glob
from datetime import datetime

# --- CAMINHOS (verifique se estão corretos) ---
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DEMONSTRATIVO = r"C:\Users\Leo\Downloads\130170-Demonstrativo_Caixa.xlsx"

# --- FUNÇÕES DE CARREGAMENTO (versões simplificadas para esta análise) ---

def processar_dados_estoque(caminho_pasta):
    """Lê e consolida os arquivos de estoque, focando em datas e valores."""
    print("Carregando dados de estoque para análise de entradas...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    
    all_dfs = []
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            
            df['Valor Presente'] = df['Valor Presente'].astype(str).str.replace(',', '.').astype(float)
            df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], errors='coerce', format='%Y-%m-%d', dayfirst=False)
            all_dfs.append(df[['Data Vencimento', 'Valor Presente']])
        except Exception:
            continue
            
    return pd.concat(all_dfs, ignore_index=True)

def processar_demonstrativo_caixa(caminho_arquivo):
    """Lê o demonstrativo e foca nas saídas para aquisição de ativos."""
    print("Carregando demonstrativo de caixa para análise de saídas...")
    try:
        df_caixa = pd.read_excel(caminho_arquivo, header=None, skiprows=4)
        df_caixa.columns = ['Titulo', 'Historico_1', 'Historico_2', 'Data', 'Tipo', 'Entrada', 'Saida', 'Saldo']

        def limpar_valor(valor):
            if isinstance(valor, str):
                valor = valor.replace('R$ ', '').strip().replace('.', '').replace(',', '.')
            return pd.to_numeric(valor, errors='coerce')

        df_caixa['Saida_Num'] = df_caixa['Saida'].apply(limpar_valor).fillna(0)
        df_caixa['Data'] = pd.to_datetime(df_caixa['Data'], errors='coerce', dayfirst=True)
        
        return df_caixa[['Data', 'Historico_2', 'Saida_Num']].dropna(subset=['Data'])
    except Exception:
        return pd.DataFrame()

# --- EXECUÇÃO DA ANÁLISE ---
if __name__ == "__main__":
    # 1. Calcula as ENTRADAS mensais do Estoque
    df_estoque = processar_dados_estoque(PATH_ESTOQUE)
    df_estoque['mes'] = df_estoque['Data Vencimento'].dt.to_period('M')
    entradas_mensais = df_estoque.groupby('mes')['Valor Presente'].sum().reset_index()
    entradas_mensais.rename(columns={'Valor Presente': 'Entradas_Estoque'}, inplace=True)

    # 2. Calcula as SAÍDAS mensais para Aquisição de Ativos
    df_caixa = processar_demonstrativo_caixa(PATH_DEMONSTRATIVO)
    df_caixa['mes'] = df_caixa['Data'].dt.to_period('M')
    
    # Filtra apenas as saídas que são para compra de novos ativos
    saidas_aquisicao = df_caixa[df_caixa['Historico_2'] == 'DC C AQUIS']
    saidas_mensais = saidas_aquisicao.groupby('mes')['Saida_Num'].sum().reset_index()
    saidas_mensais.rename(columns={'Saida_Num': 'Saidas_Aquisicao'}, inplace=True)
    
    # 3. Combina os dados e calcula a taxa
    df_analise = pd.merge(entradas_mensais, saidas_mensais, on='mes', how='inner')
    
    # Evita divisão por zero
    df_analise = df_analise[df_analise['Entradas_Estoque'] > 0]
    
    df_analise['Taxa_Reinvestimento'] = df_analise['Saidas_Aquisicao'] / df_analise['Entradas_Estoque']
    
    print("\n--- ANÁLISE DA TAXA DE REINVESTIMENTO HISTÓRICA ---")
    if df_analise.empty:
        print("Não foi possível encontrar dados sobrepostos de entradas e saídas para calcular a taxa.")
    else:
        # Formata para exibição
        df_analise['Entradas_Estoque'] = df_analise['Entradas_Estoque'].map('{:,.2f}'.format)
        df_analise['Saidas_Aquisicao'] = df_analise['Saidas_Aquisicao'].map('{:,.2f}'.format)
        df_analise['Taxa_Reinvestimento'] = pd.to_numeric(df_analise['Taxa_Reinvestimento'])
        df_analise['Taxa_Reinvestimento_Percentual'] = df_analise['Taxa_Reinvestimento'].map('{:.1%}'.format)
        
        print("Visão mensal (Entradas do Estoque vs. Saídas para Aquisição de novos ativos):")
        print(df_analise[['mes', 'Entradas_Estoque', 'Saidas_Aquisicao', 'Taxa_Reinvestimento_Percentual']].to_string(index=False))
        
        # 4. Calcula a média
        taxa_media = df_analise['Taxa_Reinvestimento'].mean()
        print("\n---------------------------------------------------------")
        print(f">>> Taxa Média de Reinvestimento Histórica: {taxa_media:.1%}")
        print("---------------------------------------------------------")
        print("\nUse este valor na sua projeção para simular o comportamento futuro do gestor.")

Carregando dados de estoque para análise de entradas...
Carregando demonstrativo de caixa para análise de saídas...

--- ANÁLISE DA TAXA DE REINVESTIMENTO HISTÓRICA ---
Visão mensal (Entradas do Estoque vs. Saídas para Aquisição de novos ativos):
    mes Entradas_Estoque Saidas_Aquisicao Taxa_Reinvestimento_Percentual
2025-07     2,968,999.98     8,515,618.97                         286.8%
2025-08     4,742,097.54     2,507,538.35                          52.9%

---------------------------------------------------------
>>> Taxa Média de Reinvestimento Histórica: 169.8%
---------------------------------------------------------

Use este valor na sua projeção para simular o comportamento futuro do gestor.


In [13]:
import pandas as pd
import json
import os
import glob
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS (sem alterações) ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"

# --- Premissa Principal (Ajuste aqui para testar cenários) ---
TAXA_DE_REINVESTIMENTO = 0.529  # 52.9%

# --- FUNÇÕES DE CARREGAMENTO (COM A CORREÇÃO EM processar_dados_estoque) ---

def carregar_dados_carteira(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return json.load(f)[0]

def processar_dados_estoque(caminho_pasta):
    print("\nCarregando dados de estoque...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv:
        print(f"AVISO: Nenhum arquivo CSV de estoque encontrado em {caminho_pasta}")
        return pd.DataFrame()

    all_dfs = []
    float_cols = ['Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido','PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB','Taxa Originador Split', 'Taxa Split FIDC']
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                for col in float_cols:
                    if col in df.columns: df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')
                for col in date_cols:
                    if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                
                # ### CORREÇÃO DO BUG ESTÁ AQUI ###
                # Mantém todas as colunas do DataFrame, em vez de selecionar apenas duas
                all_dfs.append(df)
                
        except Exception as e:
            print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
    
    if not all_dfs:
        print("Nenhum dado de estoque foi carregado.")
        return pd.DataFrame()
        
    df_final = pd.concat(all_dfs, ignore_index=True)
    print("Dados de estoque consolidados com sucesso.")
    return df_final


def processar_dados_despesas(caminho_pasta):
    print("\nCarregando e consolidando arquivos de despesas...")
    # ... (função completa sem alterações)
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame()
    lista_df_despesas = []
    for arquivo in arquivos_excel:
        try:
            df = pd.read_excel(arquivo, header=6) 
            lista_df_despesas.append(df)
        except Exception as e: print(f"Erro ao ler o arquivo de despesa {os.path.basename(arquivo)}: {e}")
    if not lista_df_despesas: return pd.DataFrame()
    df_consolidado = pd.concat(lista_df_despesas, ignore_index=True)
    df_consolidado.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'}, inplace=True)
    df_consolidado['Data Pagamento'] = pd.to_datetime(df_consolidado['Data Pagamento'], errors='coerce')
    df_consolidado['Valor'] = pd.to_numeric(df_consolidado['Valor'], errors='coerce')
    df_consolidado['Valor'] = -df_consolidado['Valor'].abs()
    df_final = df_consolidado[['Data Pagamento', 'Categoria', 'Valor']].dropna()
    print(f"{len(df_final)} despesas com data de pagamento foram carregadas.")
    return df_final


# --- DEMAIS FUNÇÕES (sem alterações) ---

def extrair_fluxos_de_caixa_com_reinvestimento(dados_carteira_recente, df_estoque, df_despesas_datadas, taxa_reinvest):
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    # ... (lógica interna desta função não precisa mudar)
    disponibilidade = next((a['Disponibilidade'] for a in dados_carteira_recente['ativos'] if 'Disponibilidade' in a and a['Disponibilidade']), None)
    if disponibilidade:
        fluxos.append({'data': data_base, 'categoria': 'Saldo Inicial', 'valor': disponibilidade['ativos'][0]['saldo']})
    cotas = next((a['Cotas'] for a in dados_carteira_recente['ativos'] if 'Cotas' in a and a['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos.append({'data': data_base, 'categoria': 'Investimentos (Fundos/RF)', 'valor': cota['valorBruto']})
    renda_fixa = next((a['RendaFixa'] for a in dados_carteira_recente['ativos'] if 'RendaFixa' in a and a['RendaFixa']), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']:
            fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Investimentos (Fundos/RF)', 'valor': titulo['mercadoAtual']})
    if not df_despesas_datadas.empty:
        for _, despesa in df_despesas_datadas.iterrows():
            fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas Operacionais', 'valor': despesa['Valor']})
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            fluxos.append({'data': data_pagamento, 'categoria': 'Amortização de Cotas', 'valor': valor_amortizacao})
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento'])
        estoque_valido = estoque_valido[estoque_valido['Status'] == 'A vencer']
        for _, row in estoque_valido.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Recebimento Estoque', 'valor': row['Valor Presente']})
        entradas_estoque_mensal = estoque_valido.groupby(pd.Grouper(key='Data Vencimento', freq='M'))['Valor Presente'].sum()
        for data_mes, total_entrada_mes in entradas_estoque_mensal.items():
            valor_reinvestimento = total_entrada_mes * taxa_reinvest * -1
            if valor_reinvestimento != 0:
                fluxos.append({'data': data_mes, 'categoria': 'Reinvestimento (Simulado)','valor': valor_reinvestimento})
    return pd.DataFrame(fluxos)

def criar_tabela_decomposicao(df_fluxos, data_base):
    if df_fluxos.empty: return "Nenhum fluxo de caixa para analisar."
    # ... (código da função sem alterações) ...
    df_fluxos = df_fluxos[df_fluxos['data'] >= data_base].copy()
    df_fluxos['mes'] = df_fluxos['data'].dt.to_period('M')
    decomposicao = df_fluxos.pivot_table(index='mes', columns='categoria', values='valor', aggfunc='sum').fillna(0)
    start_period = pd.Timestamp(data_base).to_period('M')
    end_period = df_fluxos['mes'].max()
    if pd.isna(end_period): end_period = start_period
    idx_meses = pd.period_range(start=start_period, end=end_period, freq='M')
    decomposicao = decomposicao.reindex(idx_meses, fill_value=0)
    colunas_principais = ['Recebimento Estoque', 'Investimentos (Fundos/RF)', 'Despesas Operacionais', 'Amortização de Cotas', 'Reinvestimento (Simulado)']
    for col in colunas_principais:
        if col not in decomposicao.columns: decomposicao[col] = 0
    decomposicao['Total Entradas'] = decomposicao['Recebimento Estoque'] + decomposicao['Investimentos (Fundos/RF)']
    decomposicao['Total Saídas'] = decomposicao['Despesas Operacionais'] + decomposicao['Amortização de Cotas'] + decomposicao['Reinvestimento (Simulado)']
    decomposicao['Fluxo de Caixa Mês'] = decomposicao['Total Entradas'] + decomposicao['Total Saídas']
    saldo_inicial_valor = df_fluxos[df_fluxos['categoria'] == 'Saldo Inicial']['valor'].sum()
    decomposicao['Saldo Final'] = decomposicao['Fluxo de Caixa Mês'].cumsum() + saldo_inicial_valor
    decomposicao['Saldo Inicial'] = decomposicao['Saldo Final'].shift(1).fillna(saldo_inicial_valor)
    colunas_para_mostrar = ['Saldo Inicial', 'Total Entradas', 'Recebimento Estoque', 'Total Saídas', 'Reinvestimento (Simulado)', 'Despesas Operacionais', 'Amortização de Cotas', 'Fluxo de Caixa Mês', 'Saldo Final']
    colunas_existentes = [col for col in colunas_para_mostrar if col in decomposicao.columns]
    decomposicao_final = decomposicao[colunas_existentes].head(24)
    return decomposicao_final.applymap(lambda x: f"{x:,.2f}")


# --- EXECUÇÃO PRINCIPAL ---
if __name__ == "__main__":
    # Carrega os dados (pode ser executado em células separadas no Jupyter)
    arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))
    if not arquivos_carteira:
        print(f"ERRO: Nenhum arquivo de carteira encontrado em {PATH_CARTEIRAS_HISTORICO}")
        exit()
    arquivo_recente = max(arquivos_carteira, key=os.path.getctime)
    print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_recente)}")

    dados_carteira_recente = carregar_dados_carteira(arquivo_recente)
    df_estoque = processar_dados_estoque(PATH_ESTOQUE)
    df_despesas = processar_dados_despesas(PATH_DESPESAS)
    
    data_referencia = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    # Gera os fluxos e a tabela final
    df_fluxos_final = extrair_fluxos_de_caixa_com_reinvestimento(dados_carteira_recente, df_estoque, df_despesas, TAXA_DE_REINVESTIMENTO)
    tabela_decomposicao_final = criar_tabela_decomposicao(df_fluxos_final, data_referencia)

    print("\n--- DECOMPOSIÇÃO MENSAL (COM SIMULAÇÃO DE REINVESTIMENTO) ---")
    print(f"Usando uma taxa de reinvestimento de {TAXA_DE_REINVESTIMENTO:.1%}")
    print(tabela_decomposicao_final.to_string())

Usando a carteira mais recente para a projeção: carteira_52203615000119_20241101.json

Carregando dados de estoque...
Dados de estoque consolidados com sucesso.

Carregando e consolidando arquivos de despesas...
4 despesas com data de pagamento foram carregadas.


C:\Users\Leo\AppData\Local\Temp\ipykernel_22716\1147000862.py:115: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  entradas_estoque_mensal = estoque_valido.groupby(pd.Grouper(key='Data Vencimento', freq='M'))['Valor Presente'].sum()



--- DECOMPOSIÇÃO MENSAL (COM SIMULAÇÃO DE REINVESTIMENTO) ---
Usando uma taxa de reinvestimento de 52.9%
categoria  Saldo Inicial Total Entradas Recebimento Estoque   Total Saídas Reinvestimento (Simulado) Despesas Operacionais Amortização de Cotas Fluxo de Caixa Mês    Saldo Final
2024-11         1,000.00  46,963,837.28                0.00           0.00                      0.00                  0.00                 0.00      46,963,837.28  46,964,837.28
2024-12    46,964,837.28           0.00                0.00           0.00                      0.00                  0.00                 0.00               0.00  46,964,837.28
2025-01    46,964,837.28           0.00                0.00           0.00                      0.00                  0.00                 0.00               0.00  46,964,837.28
2025-02    46,964,837.28           0.00                0.00           0.00                      0.00                  0.00                 0.00               0.00  46,964,837.28
2025

C:\Users\Leo\AppData\Local\Temp\ipykernel_22716\1147000862.py:145: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return decomposicao_final.applymap(lambda x: f"{x:,.2f}")


In [16]:
import pandas as pd
import numpy as np

# --- NOVO CAMINHO PARA O ARQUIVO ---
PATH_DEMONSTRATIVO_COMPLETO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Demonstrativos\143414-Demonstrativo_Caixa.xlsx"

# (Assumindo que 'df_estoque' já existe no seu notebook)

# --- Nova função para processar o demonstrativo de caixa ---
# (Criamos uma nova para manter a anterior, se necessário)
def processar_demonstrativo_caixa_completo(caminho_arquivo):
    """Lê o demonstrativo de caixa, focando em datas, históricos e valores."""
    print(f"Carregando demonstrativo de caixa completo de: {caminho_arquivo}")
    try:
        # A estrutura parece ser a mesma, com cabeçalho a ser pulado
        df_caixa = pd.read_excel(caminho_arquivo, header=None, skiprows=4)
        df_caixa.columns = ['Titulo', 'Historico_1', 'Historico_2', 'Data', 'Tipo', 'Entrada', 'Saida', 'Saldo']

        def limpar_valor(valor):
            if isinstance(valor, str):
                valor = valor.replace('R$ ', '').strip().replace('.', '').replace(',', '.')
            return pd.to_numeric(valor, errors='coerce')

        df_caixa['Entrada_Num'] = df_caixa['Entrada'].apply(limpar_valor).fillna(0)
        df_caixa['Saida_Num'] = df_caixa['Saida'].apply(limpar_valor).fillna(0)
        df_caixa['Data'] = pd.to_datetime(df_caixa['Data'], errors='coerce', dayfirst=True)
        
        # Mantém a coluna de Saldo numérico para os cálculos
        df_caixa['Saldo'] = df_caixa['Saldo'].apply(limpar_valor)
        
        return df_caixa.dropna(subset=['Data'])
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado em {caminho_arquivo}")
        return pd.DataFrame()
    except Exception as e:
        print(f"ERRO ao processar o arquivo: {e}")
        return pd.DataFrame()

# A função de análise de Fontes e Usos permanece a mesma
def analise_fontes_e_usos(df_estoque, df_caixa):
    """
    Realiza uma análise de Fontes e Usos de Caixa para entender o comportamento
    de investimento do gestor.
    """
    if df_caixa.empty:
        print("DataFrame de caixa está vazio.")
        return

    # Determina o período de tempo real dos dados de caixa
    data_inicio_real = df_caixa['Data'].min()
    data_fim_real = df_caixa['Data'].max()
    print(f"Analisando o período de {data_inicio_real.strftime('%d/%m/%Y')} a {data_fim_real.strftime('%d/%m/%Y')}\n")

    # Prepara os DataFrames com uma coluna de 'mes'
    df_caixa['mes'] = df_caixa['Data'].dt.to_period('M')
    df_estoque['mes'] = df_estoque['Data Vencimento'].dt.to_period('M')

    meses_analise = sorted(df_caixa['mes'].unique())
    
    resultados = []
    saldo_inicial_mes = None

    for mes in meses_analise:
        df_caixa_mes = df_caixa[df_caixa['mes'] == mes]
        df_estoque_mes = df_estoque[df_estoque['mes'] == mes]

        # Encontra o saldo inicial (último saldo do mês anterior ou o primeiro do mês atual)
        if saldo_inicial_mes is None:
            primeira_transacao_mes = df_caixa_mes.sort_values('Data').iloc[0]
            saldo_inicial_mes = primeira_transacao_mes['Saldo'] - (primeira_transacao_mes['Entrada_Num'] - primeira_transacao_mes['Saida_Num'])
        
        # FONTES DE CAIXA
        entradas_estoque = df_estoque_mes['Valor Presente'].sum()
        saidas_aquisicao = df_caixa_mes[df_caixa_mes['Historico_2'] == 'DC C AQUIS']['Saida_Num'].sum()
        
        # Todas as outras entradas e saídas do extrato
        outras_entradas = df_caixa_mes[df_caixa_mes['Historico_2'] != 'DC C AQUIS']['Entrada_Num'].sum()
        outras_saidas = df_caixa_mes[df_caixa_mes['Historico_2'] != 'DC C AQUIS']['Saida_Num'].sum()

        saldo_final_mes = df_caixa_mes.sort_values('Data').iloc[-1]['Saldo']

        resultados.append({
            'Mês': mes,
            'Saldo Inicial': saldo_inicial_mes,
            'Entradas Estoque': entradas_estoque,
            'Outras Entradas': outras_entradas,
            'Total Fontes': saldo_inicial_mes + entradas_estoque + outras_entradas,
            'Saídas Aquisição': saidas_aquisicao,
            'Outras Saídas': outras_saidas,
            'Saldo Final': saldo_final_mes,
        })
        
        saldo_inicial_mes = saldo_final_mes

    df_resultado = pd.DataFrame(resultados).set_index('Mês')
    
    print("--- ANÁLISE DE FONTES E USOS DE CAIXA (DADOS COMPLETOS) ---")
    print(df_resultado.applymap('{:,.2f}'.format).to_string())
    
    print("\n--- CONCLUSÃO DA ANÁLISE ---")
    print("Observe o comportamento mensal. Agora podemos ver com mais clareza como o fundo utiliza seu caixa.")


# --- BLOCO DE EXECUÇÃO ---
# Carrega o NOVO demonstrativo de caixa
df_caixa_completo = processar_demonstrativo_caixa_completo(PATH_DEMONSTRATIVO_COMPLETO)

# Executa a análise com o novo arquivo
analise_fontes_e_usos(df_estoque, df_caixa_completo)

Carregando demonstrativo de caixa completo de: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Demonstrativos\143414-Demonstrativo_Caixa.xlsx
Analisando o período de 31/07/2024 a 30/08/2024

--- ANÁLISE DE FONTES E USOS DE CAIXA (DADOS COMPLETOS) ---
        Saldo Inicial Entradas Estoque Outras Entradas   Total Fontes Saídas Aquisição Outras Saídas Saldo Final
Mês                                                                                                             
2024-07          0.00        18,890.91    2,397,087.42   2,415,978.33     1,198,249.93  1,198,249.93      293.78
2024-08        293.78        64,623.75   26,781,069.93  26,845,987.46    20,698,693.91  6,082,669.80        0.00

--- CONCLUSÃO DA ANÁLISE ---
Observe o comportamento mensal. Agora podemos ver com mais clareza como o fundo utiliza seu caixa.


C:\Users\Leo\AppData\Local\Temp\ipykernel_22716\4257945184.py:98: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(df_resultado.applymap('{:,.2f}'.format).to_string())


In [17]:
import pandas as pd
import numpy as np
import os
import glob

# --- CAMINHO PARA A PASTA COM TODOS OS DEMONSTRATIVOS ---
PATH_PASTA_DEMONSTRATIVOS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Demonstrativos"

# (Assumindo que 'df_estoque' já existe no seu notebook)

# --- Nova função para ler e consolidar TODOS os demonstrativos de uma pasta ---
def processar_demonstrativos_pasta(caminho_pasta):
    """
    Lê todos os arquivos Excel de uma pasta, os consolida em um único DataFrame
    e o prepara para a análise.
    """
    print(f"Lendo todos os arquivos de demonstrativo da pasta: {caminho_pasta}")
    
    # Pega todos os arquivos .xlsx na pasta
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, "*.xlsx"))
    
    if not arquivos_excel:
        print(f"ERRO: Nenhum arquivo Excel (.xlsx) encontrado em {caminho_pasta}")
        return pd.DataFrame()

    lista_dfs_mensais = []
    for arquivo in arquivos_excel:
        try:
            df_mes = pd.read_excel(arquivo, header=None, skiprows=4)
            lista_dfs_mensais.append(df_mes)
        except Exception as e:
            print(f"AVISO: Não foi possível ler o arquivo '{os.path.basename(arquivo)}'. Erro: {e}")

    if not lista_dfs_mensais:
        print("ERRO: Nenhum arquivo de demonstrativo pôde ser lido com sucesso.")
        return pd.DataFrame()

    # Consolida todos os dataframes em um só
    df_consolidado = pd.concat(lista_dfs_mensais, ignore_index=True)
    
    # Limpeza e preparação (mesma lógica de antes)
    df_consolidado.columns = ['Titulo', 'Historico_1', 'Historico_2', 'Data', 'Tipo', 'Entrada', 'Saida', 'Saldo']

    def limpar_valor(valor):
        if isinstance(valor, str):
            valor = valor.replace('R$ ', '').strip().replace('.', '').replace(',', '.')
        return pd.to_numeric(valor, errors='coerce')

    df_consolidado['Entrada_Num'] = df_consolidado['Entrada'].apply(limpar_valor).fillna(0)
    df_consolidado['Saida_Num'] = df_consolidado['Saida'].apply(limpar_valor).fillna(0)
    df_consolidado['Data'] = pd.to_datetime(df_consolidado['Data'], errors='coerce', dayfirst=True)
    df_consolidado['Saldo'] = df_consolidado['Saldo'].apply(limpar_valor)
    
    df_final = df_consolidado.dropna(subset=['Data']).sort_values('Data').reset_index(drop=True)
    print(f"Foram consolidados {len(arquivos_excel)} arquivos de demonstrativo, resultando em {len(df_final)} transações.")
    return df_final


# A função de análise de Fontes e Usos permanece a mesma
def analise_fontes_e_usos(df_estoque, df_caixa):
    if df_caixa.empty:
        print("DataFrame de caixa está vazio.")
        return

    data_inicio_real = df_caixa['Data'].min()
    data_fim_real = df_caixa['Data'].max()
    print(f"Analisando o período consolidado de {data_inicio_real.strftime('%d/%m/%Y')} a {data_fim_real.strftime('%d/%m/%Y')}\n")

    df_caixa['mes'] = df_caixa['Data'].dt.to_period('M')
    df_estoque['mes'] = df_estoque['Data Vencimento'].dt.to_period('M')

    meses_analise = sorted(df_caixa['mes'].unique())
    resultados = []
    saldo_inicial_mes = None

    for mes in meses_analise:
        df_caixa_mes = df_caixa[df_caixa['mes'] == mes]
        df_estoque_mes = df_estoque[df_estoque['mes'] == mes]

        if saldo_inicial_mes is None:
            primeira_transacao = df_caixa.iloc[0]
            saldo_inicial_mes = primeira_transacao['Saldo'] - (primeira_transacao['Entrada_Num'] - primeira_transacao['Saida_Num'])
        
        entradas_estoque = df_estoque_mes['Valor Presente'].sum()
        saidas_aquisicao = df_caixa_mes[df_caixa_mes['Historico_2'] == 'DC C AQUIS']['Saida_Num'].sum()
        outras_entradas = df_caixa_mes[df_caixa_mes['Historico_2'] != 'DC C AQUIS']['Entrada_Num'].sum()
        outras_saidas = df_caixa_mes[df_caixa_mes['Historico_2'] != 'DC C AQUIS']['Saida_Num'].sum()
        saldo_final_mes = df_caixa_mes.sort_values('Data').iloc[-1]['Saldo']

        resultados.append({
            'Mês': mes,
            'Saldo Inicial': saldo_inicial_mes,
            'Entradas Estoque': entradas_estoque,
            'Outras Entradas': outras_entradas,
            'Saídas Aquisição': saidas_aquisicao,
            'Outras Saídas': outras_saidas,
            'Saldo Final': saldo_final_mes,
        })
        saldo_inicial_mes = saldo_final_mes

    df_resultado = pd.DataFrame(resultados).set_index('Mês')
    
    print("--- ANÁLISE DE FONTES E USOS DE CAIXA (HISTÓRICO CONSOLIDADO) ---")
    print(df_resultado.applymap('{:,.2f}'.format).to_string())


# --- BLOCO DE EXECUÇÃO ---
# 1. Carrega e consolida TODOS os demonstrativos da pasta
df_caixa_consolidado = processar_demonstrativos_pasta(PATH_PASTA_DEMONSTRATIVOS)

# 2. Executa a análise com o DataFrame consolidado
analise_fontes_e_usos(df_estoque, df_caixa_consolidado)

Lendo todos os arquivos de demonstrativo da pasta: C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Demonstrativos
Foram consolidados 12 arquivos de demonstrativo, resultando em 2187 transações.
Analisando o período consolidado de 31/07/2024 a 31/07/2025

--- ANÁLISE DE FONTES E USOS DE CAIXA (HISTÓRICO CONSOLIDADO) ---
        Saldo Inicial Entradas Estoque Outras Entradas Saídas Aquisição   Outras Saídas  Saldo Final
Mês                                                                                                 
2024-07          0.00        18,890.91    2,397,087.42     1,198,249.93    1,198,249.93  -357,221.15
2024-08   -357,221.15        64,623.75   26,781,069.93    20,698,693.91    6,082,669.80         0.00
2024-09          0.00        73,910.95   79,481,234.21    18,825,962.15   60,654,272.06         0.00
2024-10          0.00        78,902.67  142,306,785.29    24,915,228.73  117,389,556.56   104,917.88
2024-11    104,917.88       140,920.39  118,643,613.46 

C:\Users\Leo\AppData\Local\Temp\ipykernel_22716\1484511809.py:104: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  print(df_resultado.applymap('{:,.2f}'.format).to_string())


In [20]:
import pandas as pd
import json
import os
import glob
from datetime import datetime
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS E PREMISSAS FINAIS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"

# Premissa final baseada na análise histórica: o saldo de caixa que o gestor tenta manter.
SALDO_CAIXA_ALVO = 10000.00


def carregar_dados_carteira(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return json.load(f)[0]

def processar_dados_estoque(caminho_pasta):
    print("\nCarregando dados de estoque...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv:
        print(f"AVISO: Nenhum arquivo CSV de estoque encontrado em {caminho_pasta}")
        return pd.DataFrame()

    all_dfs = []
    float_cols = ['Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido','PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB','Taxa Originador Split', 'Taxa Split FIDC']
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                for col in float_cols:
                    if col in df.columns: df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')
                for col in date_cols:
                    if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                
                # ### CORREÇÃO DO BUG ESTÁ AQUI ###
                # Mantém todas as colunas do DataFrame, em vez de selecionar apenas duas
                all_dfs.append(df)
                
        except Exception as e:
            print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
    
    if not all_dfs:
        print("Nenhum dado de estoque foi carregado.")
        return pd.DataFrame()
        
    df_final = pd.concat(all_dfs, ignore_index=True)
    print("Dados de estoque consolidados com sucesso.")
    return df_final


def processar_dados_despesas(caminho_pasta):
    print("\nCarregando e consolidando arquivos de despesas...")
    # ... (função completa sem alterações)
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame()
    lista_df_despesas = []
    for arquivo in arquivos_excel:
        try:
            df = pd.read_excel(arquivo, header=6) 
            lista_df_despesas.append(df)
        except Exception as e: print(f"Erro ao ler o arquivo de despesa {os.path.basename(arquivo)}: {e}")
    if not lista_df_despesas: return pd.DataFrame()
    df_consolidado = pd.concat(lista_df_despesas, ignore_index=True)
    df_consolidado.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'}, inplace=True)
    df_consolidado['Data Pagamento'] = pd.to_datetime(df_consolidado['Data Pagamento'], errors='coerce')
    df_consolidado['Valor'] = pd.to_numeric(df_consolidado['Valor'], errors='coerce')
    df_consolidado['Valor'] = -df_consolidado['Valor'].abs()
    df_final = df_consolidado[['Data Pagamento', 'Categoria', 'Valor']].dropna()
    print(f"{len(df_final)} despesas com data de pagamento foram carregadas.")
    return df_final

# --- FUNÇÃO DE PROJEÇÃO FINAL (COM LÓGICA DE SALDO ALVO) ---
def extrair_fluxos_de_caixa_final(dados_carteira_recente, df_estoque, df_despesas_datadas, saldo_alvo):
    """
    Gera a lista completa de fluxos de caixa, simulando o reinvestimento
    para manter um saldo de caixa alvo.
    """
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    # Adiciona todos os fluxos, exceto o reinvestimento, a uma lista temporária
    fluxos_pre_reinvest = []
    
    disponibilidade = next((a['Disponibilidade'] for a in dados_carteira_recente['ativos'] if 'Disponibilidade' in a and a['Disponibilidade']), None)
    if disponibilidade:
        fluxos.append({'data': data_base, 'categoria': 'Saldo Inicial', 'valor': disponibilidade['ativos'][0]['saldo']})

    cotas = next((a['Cotas'] for a in dados_carteira_recente['ativos'] if 'Cotas' in a and a['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos_pre_reinvest.append({'data': data_base, 'categoria': 'Investimentos (Fundos/RF)', 'valor': cota['valorBruto']})
            
    # ... (Restante da extração de RF, Estoque, Despesas, Amortização como antes) ...
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento'])
        estoque_valido = estoque_valido[estoque_valido['Status'] == 'A vencer']
        for _, row in estoque_valido.iterrows():
            fluxos_pre_reinvest.append({'data': row['Data Vencimento'], 'categoria': 'Recebimento Estoque', 'valor': row['Valor Presente']})
    
    if not df_despesas_datadas.empty:
        for _, despesa in df_despesas_datadas.iterrows():
            fluxos_pre_reinvest.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas Operacionais', 'valor': despesa['Valor']})

    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            fluxos_pre_reinvest.append({'data': data_pagamento, 'categoria': 'Amortização de Cotas', 'valor': valor_amortizacao})


    df_pre_reinvest = pd.DataFrame(fluxos_pre_reinvest)
    df_pre_reinvest['mes'] = df_pre_reinvest['data'].dt.to_period('M')
    
    # Calcula o fluxo de caixa operacional de cada mês
    fluxo_op_mensal = df_pre_reinvest.groupby('mes')['valor'].sum()
    
    # Adiciona os fluxos originais à lista final
    fluxos.extend(fluxos_pre_reinvest)
    
    # Calcula e adiciona as saídas de reinvestimento
    saldo_acumulado = 0 # Começa do zero pois o saldo inicial já está na lista principal
    for mes, fluxo_do_mes in fluxo_op_mensal.items():
        caixa_disponivel_antes = saldo_acumulado + fluxo_do_mes
        excedente = caixa_disponivel_antes - saldo_alvo
        
        if excedente > 0:
            valor_reinvestimento = -excedente
            fluxos.append({
                'data': mes.to_timestamp(how='end'), # Adiciona no final do mês
                'categoria': 'Reinvestimento (Simulado)',
                'valor': valor_reinvestimento
            })
            saldo_acumulado += fluxo_do_mes + valor_reinvestimento
        else:
            saldo_acumulado += fluxo_do_mes
            
    return pd.DataFrame(fluxos)

# A função criar_tabela_decomposicao permanece a mesma
def criar_tabela_decomposicao(df_fluxos, data_base):
    if df_fluxos.empty: return "Nenhum fluxo de caixa para analisar."
    # ... (código da função sem alterações) ...
    df_fluxos = df_fluxos[df_fluxos['data'] >= data_base].copy()
    df_fluxos['mes'] = df_fluxos['data'].dt.to_period('M')
    decomposicao = df_fluxos.pivot_table(index='mes', columns='categoria', values='valor', aggfunc='sum').fillna(0)
    start_period = pd.Timestamp(data_base).to_period('M')
    end_period = df_fluxos['mes'].max()
    if pd.isna(end_period): end_period = start_period
    idx_meses = pd.period_range(start=start_period, end=end_period, freq='M')
    decomposicao = decomposicao.reindex(idx_meses, fill_value=0)
    colunas_principais = ['Recebimento Estoque', 'Investimentos (Fundos/RF)', 'Despesas Operacionais', 'Amortização de Cotas', 'Reinvestimento (Simulado)']
    for col in colunas_principais:
        if col not in decomposicao.columns: decomposicao[col] = 0
    decomposicao['Total Entradas'] = decomposicao['Recebimento Estoque'] + decomposicao['Investimentos (Fundos/RF)']
    decomposicao['Total Saídas'] = decomposicao['Despesas Operacionais'] + decomposicao['Amortização de Cotas'] + decomposicao['Reinvestimento (Simulado)']
    decomposicao['Fluxo de Caixa Mês'] = decomposicao['Total Entradas'] + decomposicao['Total Saídas']
    saldo_inicial_valor = df_fluxos[df_fluxos['categoria'] == 'Saldo Inicial']['valor'].sum()
    decomposicao['Saldo Final'] = decomposicao['Fluxo de Caixa Mês'].cumsum() + saldo_inicial_valor
    decomposicao['Saldo Inicial'] = decomposicao['Saldo Final'].shift(1).fillna(saldo_inicial_valor)
    colunas_para_mostrar = ['Saldo Inicial', 'Total Entradas', 'Recebimento Estoque', 'Total Saídas', 'Reinvestimento (Simulado)', 'Despesas Operacionais', 'Amortização de Cotas', 'Fluxo de Caixa Mês', 'Saldo Final']
    colunas_existentes = [col for col in colunas_para_mostrar if col in decomposicao.columns]
    decomposicao_final = decomposicao[colunas_existentes].head(24)
    return decomposicao_final.applymap(lambda x: f"{x:,.2f}")


# --- BLOCO DE EXECUÇÃO ---
# (Assumindo que os dataframes já foram carregados)

df_fluxos_final = extrair_fluxos_de_caixa_final(dados_carteira_recente, df_estoque, df_despesas, SALDO_CAIXA_ALVO)
tabela_final = criar_tabela_decomposicao(df_fluxos_final, data_referencia)
print(tabela_final)

categoria Saldo Inicial Total Entradas Recebimento Estoque    Total Saídas  \
2024-11        1,000.00  46,963,837.28                0.00  -46,953,837.28   
2024-12       11,000.00           0.00                0.00            0.00   
2025-01       11,000.00           0.00                0.00            0.00   
2025-02       11,000.00           0.00                0.00            0.00   
2025-03       11,000.00           0.00                0.00            0.00   
2025-04       11,000.00           0.00                0.00            0.00   
2025-05       11,000.00           0.00                0.00     -228,889.26   
2025-06     -217,889.26           0.00                0.00            0.00   
2025-07     -217,889.26           0.00                0.00            0.00   
2025-08     -217,889.26           0.00                0.00            0.00   
2025-09     -217,889.26           0.00                0.00            0.00   
2025-10     -217,889.26   3,913,141.23        3,913,141.23   -3,

C:\Users\Leo\AppData\Local\Temp\ipykernel_22716\527954461.py:174: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return decomposicao_final.applymap(lambda x: f"{x:,.2f}")


In [1]:
#%%
import pandas as pd
import json
import os
import glob
from datetime import datetime
from dateutil.relativedelta import relativedelta

#%%
#! paths !################################################################################
path_carteiras = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
path_estoque = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
path_despesas = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"
#!#######################################################################################
# possibilidade de estimar com uma taxa constante de reinvestimento
taxa_reinvest = 0.00 

#%%
#? funcoes de carregar os dados 

def load_carteira(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)[0]

def load_estoque(path):
    print("\ncarregando dados de estoque...")
    csv_files = glob.glob(os.path.join(path, "*.csv"))
    
    dfs = []
    float_cols = ['Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido','PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB','Taxa Originador Split', 'Taxa Split FIDC']
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']
    
    for file in csv_files:
        df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
        df.columns = df.columns.str.strip()
        
        for col in float_cols:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')
        for col in date_cols:
            df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
        
        dfs.append(df)
        
    df = pd.concat(dfs, ignore_index=True)
    print("dados de estoque consolidados.")
    return df

def load_despesas(path):
    print("\ncarregando despesas...")
    excel_files = glob.glob(os.path.join(path, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    
    dfs = []
    for file in excel_files:
        df = pd.read_excel(file, header=6) 
        dfs.append(df)
        
    df = pd.concat(dfs, ignore_index=True)
    df.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'}, inplace=True)
    
    df['Data Pagamento'] = pd.to_datetime(df['Data Pagamento'], errors='coerce')
    df['Valor'] = pd.to_numeric(df['Valor'], errors='coerce')
    df['Valor'] = -df['Valor'].abs() # valor fica negativo pois eh uma saida
    
    df = df[['Data Pagamento', 'Categoria', 'Valor']].dropna()
    print(f"{len(df)} despesas carregadas.")
    return df

#%%
#? funcoes de calc

def get_fluxos(carteira_data, estoque, despesas, reinvest_rate):
    fluxos = []
    base_date = datetime.strptime(carteira_data['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    # busca os saldos/investimentos iniciais
    disp = next((a['Disponibilidade'] for a in carteira_data['ativos'] if 'Disponibilidade' in a), None)
    if disp:
        fluxos.append({'data': base_date, 'categoria': 'Saldo Inicial', 'valor': disp['ativos'][0]['saldo']})

    cotas = next((a['Cotas'] for a in carteira_data['ativos'] if 'Cotas' in a), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos.append({'data': base_date, 'categoria': 'Investimentos (Fundos/RF)', 'valor': cota['valorBruto']})
            
    rf = next((a['RendaFixa'] for a in carteira_data['ativos'] if 'RendaFixa' in a), None)
    if rf:
        for titulo in rf['ativos']:
            venc = datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d')
            fluxos.append({'data': venc, 'categoria': 'Investimentos (Fundos/RF)', 'valor': titulo['mercadoAtual']})
            
    # add despesas operacionais
    for _, row in despesas.iterrows():
        fluxos.append({'data': row['Data Pagamento'], 'categoria': 'Despesas Operacionais', 'valor': row['Valor']})

    # calcula amortizacao da cota senior
    sr_carteira = next((c for c in carteira_data['carteiras'] if 'SR2' in c['nome']), None)
    if sr_carteira:
        pl_sr = sr_carteira['pl']
        amort_val = (pl_sr / 36) * -1
        amort_date = datetime(2026, 1, 16)
        for i in range(36):
            pay_date = amort_date + relativedelta(months=i)
            while pay_date.weekday() >= 5: pay_date += pd.Timedelta(days=1)
            fluxos.append({'data': pay_date, 'categoria': 'Amortização de Cotas', 'valor': amort_val})
    
    # adiciona recebimentos e reinvestimentos do estoque
    estoque = estoque.dropna(subset=['Data Vencimento'])
    estoque = estoque[estoque['Status'] == 'A vencer']
    for _, row in estoque.iterrows():
        fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Recebimento Estoque', 'valor': row['Valor Presente']})
        
    entradas_mes = estoque.groupby(pd.Grouper(key='Data Vencimento', freq='M'))['Valor Presente'].sum()
    for data, total in entradas_mes.items():
        reinvest_val = total * reinvest_rate * -1
        if reinvest_val != 0:
            fluxos.append({'data': data, 'categoria': 'Reinvestimento (Simulado)','valor': reinvest_val})

    return pd.DataFrame(fluxos)

def make_dec_table(fluxos, base_date):
    fluxos = fluxos[fluxos['data'] >= base_date].copy()
    fluxos['mes'] = fluxos['data'].dt.to_period('M')
    
    dec = fluxos.pivot_table(index='mes', columns='categoria', values='valor', aggfunc='sum').fillna(0)
    
    start_p = pd.Timestamp(base_date).to_period('M')
    end_p = fluxos['mes'].max()
    idx_meses = pd.period_range(start=start_p, end=end_p, freq='M')
    dec = dec.reindex(idx_meses, fill_value=0)
    
    # garante que as colunas principais existem
    main_cols = ['Recebimento Estoque', 'Investimentos (Fundos/RF)', 'Despesas Operacionais', 'Amortização de Cotas', 'Reinvestimento (Simulado)']
    for col in main_cols:
        if col not in dec.columns: dec[col] = 0

    dec['Total Entradas'] = dec['Recebimento Estoque'] + dec.get('Investimentos (Fundos/RF)', 0)
    dec['Total Saídas'] = dec['Despesas Operacionais'] + dec['Amortização de Cotas'] + dec['Reinvestimento (Simulado)']
    dec['Fluxo de Caixa Mês'] = dec['Total Entradas'] + dec['Total Saídas']
    
    saldo_ini_val = fluxos[fluxos['categoria'] == 'Saldo Inicial']['valor'].sum()
    dec['Saldo Final'] = dec['Fluxo de Caixa Mês'].cumsum() + saldo_ini_val
    dec['Saldo Inicial'] = dec['Saldo Final'].shift(1).fillna(saldo_ini_val)
    
    cols_ordem = ['Saldo Inicial', 'Total Entradas', 'Recebimento Estoque', 'Total Saídas', 'Reinvestimento (Simulado)', 'Despesas Operacionais', 'Amortização de Cotas', 'Fluxo de Caixa Mês', 'Saldo Final']
    cols_final = [col for col in cols_ordem if col in dec.columns]
    
    dec_final = dec[cols_final].head(24)
    return dec_final.applymap(lambda x: f"{x:,.2f}")

#%%
# main >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

# carrega os dados
carteira_files = glob.glob(os.path.join(path_carteiras, "*.json"))
latest_file = max(carteira_files, key=os.path.getctime)
print(f"usando carteira: {os.path.basename(latest_file)}")

carteira_data = load_carteira(latest_file)
estoque = load_estoque(path_estoque)
despesas = load_despesas(path_despesas)

# gero os fluxos e a tabela final ############################
ref_date = datetime.strptime(carteira_data['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
fluxos = get_fluxos(carteira_data, estoque, despesas, taxa_reinvest)
dec_table = make_dec_table(fluxos, ref_date)

# imprime o resultado
print("\n--- DECOMPOSICAO MENSAL  ---")
print(f"taxa de reinvestimento: {taxa_reinvest:.1%}")
print(dec_table.to_string())

usando carteira: carteira_52203615000119_20241101.json

carregando dados de estoque...
dados de estoque consolidados.

carregando despesas...
4 despesas carregadas.


C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\1717109862.py:112: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  entradas_mes = estoque.groupby(pd.Grouper(key='Data Vencimento', freq='M'))['Valor Presente'].sum()



--- DECOMPOSICAO MENSAL (COM SIMULACAO DE REINVESTIMENTO) ---
taxa de reinvestimento: 52.9%
categoria  Saldo Inicial Total Entradas Recebimento Estoque   Total Saídas Reinvestimento (Simulado) Despesas Operacionais Amortização de Cotas Fluxo de Caixa Mês    Saldo Final
2024-11         1,000.00  46,963,837.28                0.00           0.00                      0.00                  0.00                 0.00      46,963,837.28  46,964,837.28
2024-12    46,964,837.28           0.00                0.00           0.00                      0.00                  0.00                 0.00               0.00  46,964,837.28
2025-01    46,964,837.28           0.00                0.00           0.00                      0.00                  0.00                 0.00               0.00  46,964,837.28
2025-02    46,964,837.28           0.00                0.00           0.00                      0.00                  0.00                 0.00               0.00  46,964,837.28
2025-03    46,964

C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\1717109862.py:148: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return dec_final.applymap(lambda x: f"{x:,.2f}")


In [8]:
import pandas as pd
import json
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS PARA AS FONTES DE DADOS FINAIS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÕES DE CARREGAMENTO (sem alterações) ---
# (As funções carregar_dados_carteira, processar_dados_estoque, e
# processar_dados_despesas que já usamos antes entram aqui. Omitidas para brevidade.)

# --- FUNÇÕES DE CARREGAMENTO (COM A CORREÇÃO EM processar_dados_estoque) ---

def carregar_dados_carteira(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return json.load(f)[0]

def processar_dados_estoque(caminho_pasta):
    print("\nCarregando dados de estoque...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv:
        print(f"AVISO: Nenhum arquivo CSV de estoque encontrado em {caminho_pasta}")
        return pd.DataFrame()

    all_dfs = []
    float_cols = ['Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido','PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB','Taxa Originador Split', 'Taxa Split FIDC']
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                for col in float_cols:
                    if col in df.columns: df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')
                for col in date_cols:
                    if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                
                # ### CORREÇÃO DO BUG ESTÁ AQUI ###
                # Mantém todas as colunas do DataFrame, em vez de selecionar apenas duas
                all_dfs.append(df)
                
        except Exception as e:
            print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
    
    if not all_dfs:
        print("Nenhum dado de estoque foi carregado.")
        return pd.DataFrame()
        
    df_final = pd.concat(all_dfs, ignore_index=True)
    print("Dados de estoque consolidados com sucesso.")
    return df_final


def processar_dados_despesas(caminho_pasta):
    print("\nCarregando e consolidando arquivos de despesas...")
    # ... (função completa sem alterações)
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame()
    lista_df_despesas = []
    for arquivo in arquivos_excel:
        try:
            df = pd.read_excel(arquivo, header=6) 
            lista_df_despesas.append(df)
        except Exception as e: print(f"Erro ao ler o arquivo de despesa {os.path.basename(arquivo)}: {e}")
    if not lista_df_despesas: return pd.DataFrame()
    df_consolidado = pd.concat(lista_df_despesas, ignore_index=True)
    df_consolidado.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'}, inplace=True)
    df_consolidado['Data Pagamento'] = pd.to_datetime(df_consolidado['Data Pagamento'], errors='coerce')
    df_consolidado['Valor'] = pd.to_numeric(df_consolidado['Valor'], errors='coerce')
    df_consolidado['Valor'] = -df_consolidado['Valor'].abs()
    df_final = df_consolidado[['Data Pagamento', 'Categoria', 'Valor']].dropna()
    print(f"{len(df_final)} despesas com data de pagamento foram carregadas.")
    return df_final

# --- NOVA LÓGICA DE GERAÇÃO DO RELATÓRIO ---

def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    """
    Cria uma lista de TODOS os eventos de fluxo de caixa futuros,
    categorizados exatamente como na tabela do chefe.
    """
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # Disponibilidades
    disponibilidade = next((a['Disponibilidade'] for a in dados_carteira_recente['ativos'] if 'Disponibilidade' in a and a['Disponibilidade']), None)
    if disponibilidade:
        fluxos.append({'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo']})

    cotas = next((a['Cotas'] for a in dados_carteira_recente['ativos'] if 'Cotas' in a and a['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']:
            # Assumindo liquidez D+1 para fundos
            fluxos.append({'data': data_base + timedelta(days=1), 'categoria': 'Fundo inv.', 'valor': cota['valorBruto']})
            
    renda_fixa = next((a['RendaFixa'] for a in dados_carteira_recente['ativos'] if 'RendaFixa' in a and a['RendaFixa']), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']:
            fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Títulos Renda Fixa', 'valor': titulo['mercadoAtual']})

    # Pagamentos do Estoque
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Presente'])
        estoque_valido = estoque_valido[estoque_valido['Status'] == 'A vencer']
        for _, row in estoque_valido.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Pagamentos Estoque', 'valor': row['Valor Presente']})

    # Necessidades
    if not df_despesas_datadas.empty:
        for _, despesa in df_despesas_datadas.iterrows():
            fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas', 'valor': despesa['Valor']})

    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5:
                data_pagamento += pd.Timedelta(days=1)
            fluxos.append({'data': data_pagamento, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao})

    return pd.DataFrame(fluxos)

def gerar_relatorio_d_mais(df_fluxos, data_base):
    """
    Gera a tabela final no formato D+N solicitado pelo chefe.
    """
    prazos = [0, 1, 5, 10, 21, 63]
    colunas_relatorio = ['Conta Corr.', 'Fundo inv.', 'Títulos Renda Fixa', 'Pagamentos Estoque', 'Despesas', 'Amort./ Resgat. Cota']
    
    linhas_relatorio = []

    for dias in prazos:
        data_limite = data_base + timedelta(days=dias)
        
        # Filtra todos os eventos de caixa que ocorrem ATÉ a data limite
        fluxos_ate_data = df_fluxos[df_fluxos['data'] <= data_limite]
        
        # Agrupa por categoria e soma os valores acumulados
        saldos_acumulados = fluxos_ate_data.groupby('categoria')['valor'].sum()
        
        # Cria a linha do relatório
        linha = {'Prazo': f"D+{dias}"}
        for col in colunas_relatorio:
            linha[col] = saldos_acumulados.get(col, 0.0)
        
        linhas_relatorio.append(linha)
        
    df_relatorio = pd.DataFrame(linhas_relatorio).set_index('Prazo')
    
    # Adiciona as colunas de totais e saldo final
    df_relatorio['Disponibilidades'] = df_relatorio['Conta Corr.'] + df_relatorio['Fundo inv.'] + df_relatorio['Títulos Renda Fixa'] + df_relatorio['Pagamentos Estoque']
    df_relatorio['Necessidades'] = df_relatorio['Despesas'] + df_relatorio['Amort./ Resgat. Cota']
    df_relatorio['Caixa Projetado'] = df_relatorio['Disponibilidades'] + df_relatorio['Necessidades'] # Necessidades já são negativas
    
    return df_relatorio.applymap('{:,.2f}'.format)


# --- BLOCO DE EXECUÇÃO ---
if __name__ == "__main__":
    # Carrega os dados (substitua os placeholders pelas funções completas)
    #df_estoque = processar_dados_estoque(PATH_ESTOQUE)
    #df_despesas = processar_dados_despesas(PATH_DESPESAS)
    
    # Encontra o arquivo de carteira mais recente
    arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))
    if not arquivos_carteira:
        print(f"ERRO: Nenhum arquivo de carteira encontrado.")
        exit()
    arquivo_recente = max(arquivos_carteira, key=os.path.getctime)
    print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_recente)}")

    dados_carteira_recente = carregar_dados_carteira(arquivo_recente)
    data_referencia = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    # Prepara a lista de todos os eventos de fluxo de caixa futuros
    # (Use seus DataFrames já carregados aqui)
    df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)
    
    # Gera o relatório no formato D+N
    relatorio_final = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia)

    print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (FORMATO SOLICITADO) ---")
    print(relatorio_final.to_string())
    

Usando a carteira mais recente para a projeção: carteira_52203615000119_20250818.json

--- PROJEÇÃO DE FLUXO DE CAIXA (FORMATO SOLICITADO) ---
      Conta Corr.  Fundo inv. Títulos Renda Fixa Pagamentos Estoque     Despesas Amort./ Resgat. Cota Disponibilidades Necessidades Caixa Projetado
Prazo                                                                                                                                              
D+0      1,000.00        0.00               0.00               0.00  -228,889.26                 0.00         1,000.00  -228,889.26     -227,889.26
D+1      1,000.00  661,422.32               0.00               0.00  -228,889.26                 0.00       662,422.32  -228,889.26      433,533.06
D+5      1,000.00  661,422.32               0.00               0.00  -228,889.26                 0.00       662,422.32  -228,889.26      433,533.06
D+10     1,000.00  661,422.32               0.00               0.00  -228,889.26                 0.00       662,422.3

C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\1784773212.py:165: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_relatorio.applymap('{:,.2f}'.format)


In [7]:
#%%
# imports
import requests
import json
import os
from datetime import date, timedelta
import time

#%%
# --- configuracoes ---
cnpjs = ["52203615000119"]
tokens = [
    "96cc2224-0c2f-454f-b785-db8dd204f559",
    "d1bd877a-8b21-4c29-b005-dd5540d38673",
    "ad647ac8-188d-4d50-a6ff-6ef2524ac3ca"
]
cpf = "10142836982"
pasta_saida = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
url_auth = "https://apis.vortx.com.br/vxlogin/api/user/AuthUserApi"
url_base = "https://apis.vortx.com.br"

#%%
# --- funcoes ---

def gerar_jwt(token_acesso, cpf_usuario):
    """tenta gerar um token de autenticacao temporario."""
    print(f"gerando token jwt com o final: ...{token_acesso[-6:]}")
    payload = {"token": token_acesso, "login": cpf_usuario}
    try:
        r = requests.post(url_auth, json=payload, timeout=15)
        if r.status_code == 200:
            jwt = r.json().get("token")
            if jwt:
                print(">>> token jwt gerado com sucesso.")
                return jwt
        print(f"falha na autenticacao. status: {r.status_code}")
        return None
    except requests.exceptions.RequestException as e:
        print(f"erro de conexao ao autenticar: {e}")
        return None

def obter_carteira(jwt, cnpjs_lista, data_str, pasta):
    """busca e salva o arquivo json da carteira para uma data especifica."""
    print(f"\nbuscando carteira para {data_str}")
    url = f"{url_base}/carteira-liberada/buscarCarteiraJSON"
    headers = {"Authorization": f"Bearer {jwt}"}
    params = {"cnpjFundos[]": cnpjs_lista, "dataCarteira": data_str}

    try:
        r = requests.get(url, headers=headers, params=params, timeout=45)
        if r.status_code != 200:
            print(f"falha ao obter carteira de {data_str}. status: {r.status_code}")
            return False

        dados = r.json()
        
        if not dados or not dados[0].get('carteiras'):
            print(f"aviso: dados da carteira para {data_str} vieram vazios. pulando.")
            return False

        cnpjs_str = "_".join(cnpjs_lista)
        nome_arquivo = f"carteira_{cnpjs_str}_{data_str.replace('-', '')}.json"
        caminho = os.path.join(pasta, nome_arquivo)

        with open(caminho, 'w', encoding='utf-8') as f:
            json.dump(dados, f, indent=4, ensure_ascii=False)
        
        print(f"-> carteira de {data_str} salva em: '{caminho}'")
        return True
    except requests.exceptions.RequestException as e:
        print(f"erro de conexao ao buscar a carteira de {data_str}: {e}")
        return False

#%%
# --- execucao ---
if __name__ == "__main__":
    os.makedirs(pasta_saida, exist_ok=True) 

    # --- ALTERACAO PRINCIPAL AQUI ---
    # gera a lista de datas para um periodo de 2 semanas terminando em 29/08/2025
    datas = []
    data_final = date(2025, 8, 29) # data final fixa, conforme solicitado
    # comecamos 13 dias antes para ter um periodo total de 14 dias
    data_inicial = data_final - timedelta(days=13) 
    
    d = data_inicial
    while d <= data_final:
        datas.append(d.strftime("%Y-%m-%d"))
        d += timedelta(days=1)
    
    # invertemos para comecar pela data mais recente
    datas.reverse()
    
    print(f"serao buscadas as carteiras de {len(datas)} dias.")
    print(f"periodo: de {datas[-1]} (16/08/2025) a {datas[0]} (29/08/2025)")

    jwt = None
    # tenta obter um token jwt valido
    for token in tokens:
        jwt = gerar_jwt(token, cpf)
        if jwt:
            break
    
    if not jwt:
        print("\n-> todas as tentativas de autenticacao falharam. fim do script.")
        exit()

    # itera sobre as datas e baixa cada carteira
    for data_carteira in datas:
        ok = obter_carteira(jwt, cnpjs, data_carteira, pasta_saida)
        if not ok:
            print(f"problema ao obter a carteira para {data_carteira}. continuando...")
        time.sleep(2) # pausa de 2s para nao sobrecarregar a api

    print("\n--- downloads concluidos ---")

serao buscadas as carteiras de 14 dias.
periodo: de 2025-08-16 (16/08/2025) a 2025-08-29 (29/08/2025)
gerando token jwt com o final: ...04f559
>>> token jwt gerado com sucesso.

buscando carteira para 2025-08-29
-> carteira de 2025-08-29 salva em: 'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras\carteira_52203615000119_20250829.json'

buscando carteira para 2025-08-28
-> carteira de 2025-08-28 salva em: 'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras\carteira_52203615000119_20250828.json'

buscando carteira para 2025-08-27
-> carteira de 2025-08-27 salva em: 'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras\carteira_52203615000119_20250827.json'

buscando carteira para 2025-08-26
-> carteira de 2025-08-26 salva em: 'C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras\carteira_52203615000119_20250826.json'

buscando carteira para 2025-08-25
-> carteira de 2025-08-25 salva em: 'C:\

In [9]:
import pandas as pd
import json
import os
import glob
from datetime import datetime

# --- CAMINHOS (sem alterações) ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"

# --- FUNÇÕES DE CARREGAMENTO (copiadas do seu script para garantir consistência) ---
def carregar_dados_carteira(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return json.load(f)[0]

def processar_dados_estoque(caminho_pasta):
    print("\n--- INSPECIONANDO DADOS DE ESTOQUE ---")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv:
        print("Nenhum arquivo CSV de estoque encontrado.")
        return pd.DataFrame()
    # ... (código da função omitido para brevidade, a lógica é a mesma)
    all_dfs = []
    float_cols = ['Valor Aquisicao', 'Valor Nominal', 'Valor Presente', 'PDD Vencido','PDD Total', 'Taxa Operada Originador', 'CET Mensal', 'Taxa CCB','Taxa Originador Split', 'Taxa Split FIDC']
    date_cols = ['Data Aquisicao', 'Data Vencimento', 'Data Referencia', 'Data de Nascimento']
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                for col in float_cols:
                    if col in df.columns: df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', '.'), errors='coerce')
                for col in date_cols:
                    if col in df.columns: df[col] = pd.to_datetime(df[col], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception as e:
            print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
    if not all_dfs: return pd.DataFrame()
    df_final = pd.concat(all_dfs, ignore_index=True)
    return df_final


def processar_dados_despesas(caminho_pasta):
    print("\n--- INSPECIONANDO DADOS DE DESPESAS ---")
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel:
        print("Nenhum arquivo 'Despesas_Consolidadas.xlsx' encontrado.")
        return pd.DataFrame()
    # ... (código da função omitido para brevidade, a lógica é a mesma)
    lista_df_despesas = []
    for arquivo in arquivos_excel:
        try:
            df = pd.read_excel(arquivo, header=6) 
            lista_df_despesas.append(df)
        except Exception as e: print(f"Erro ao ler o arquivo de despesa {os.path.basename(arquivo)}: {e}")
    if not lista_df_despesas: return pd.DataFrame()
    df_consolidado = pd.concat(lista_df_despesas, ignore_index=True)
    df_consolidado.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'}, inplace=True)
    df_consolidado['Data Pagamento'] = pd.to_datetime(df_consolidado['Data Pagamento'], errors='coerce')
    df_consolidado['Valor'] = pd.to_numeric(df_consolidado['Valor'], errors='coerce')
    df_consolidado['Valor'] = -df_consolidado['Valor'].abs()
    df_final = df_consolidado[['Data Pagamento', 'Categoria', 'Valor']].dropna()
    return df_final


# --- BLOCO DE EXECUÇÃO DA DEPURAÇÃO ---
if __name__ == "__main__":
    
    # 1. Depuração da Carteira
    print("--- INSPECIONANDO DADOS DA CARTEIRA ---")
    arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))
    if not arquivos_carteira:
        print("Nenhum arquivo de carteira encontrado.")
    else:
        arquivo_recente = max(arquivos_carteira, key=os.path.getctime)
        dados_carteira = carregar_dados_carteira(arquivo_recente)
        data_referencia = datetime.strptime(dados_carteira['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
        print(f"Arquivo de carteira mais recente: {os.path.basename(arquivo_recente)}")
        print(f"DATA DE REFERÊNCIA PARA A PROJEÇÃO (D+0): {data_referencia.strftime('%d/%m/%Y')}")
    
    print("-" * 50)
    
    # 2. Depuração do Estoque
    df_estoque_debug = processar_dados_estoque(PATH_ESTOQUE)
    if not df_estoque_debug.empty:
        print(f"\nFormato do Estoque: {df_estoque_debug.shape[0]} linhas, {df_estoque_debug.shape[1]} colunas.")
        print("\nAmostra dos dados de Estoque:")
        print(df_estoque_debug[['Data Vencimento', 'Valor Presente', 'Status']].head())
        print("\nIntervalo de datas no Estoque:")
        print(f"  - Data de Vencimento Mais Antiga: {df_estoque_debug['Data Vencimento'].min().strftime('%d/%m/%Y')}")
        print(f"  - Data de Vencimento Mais Recente: {df_estoque_debug['Data Vencimento'].max().strftime('%d/%m/%Y')}")
    
    print("-" * 50)

    # 3. Depuração das Despesas
    df_despesas_debug = processar_dados_despesas(PATH_DESPESAS)
    if not df_despesas_debug.empty:
        print(f"\nFormato das Despesas: {df_despesas_debug.shape[0]} linhas, {df_despesas_debug.shape[1]} colunas.")
        print("\nAmostra dos dados de Despesas:")
        print(df_despesas_debug.head())
        print("\nIntervalo de datas nas Despesas:")
        print(f"  - Data de Pagamento Mais Antiga: {df_despesas_debug['Data Pagamento'].min().strftime('%d/%m/%Y')}")
        print(f"  - Data de Pagamento Mais Recente: {df_despesas_debug['Data Pagamento'].max().strftime('%d/%m/%Y')}")

    print("\n" + "="*50)
    print("ANÁLISE PRELIMINAR:")
    print("Compare os intervalos de datas acima com a 'DATA DE REFERÊNCIA'.")
    print("Se as datas de vencimento/pagamento estiverem muito distantes da data de referência, elas não aparecerão na projeção de curto prazo (D+0 a D+63).")
    print("="*50)

--- INSPECIONANDO DADOS DA CARTEIRA ---
Arquivo de carteira mais recente: carteira_52203615000119_20250818.json
DATA DE REFERÊNCIA PARA A PROJEÇÃO (D+0): 18/08/2025
--------------------------------------------------

--- INSPECIONANDO DADOS DE ESTOQUE ---

Formato do Estoque: 1644304 linhas, 33 colunas.

Amostra dos dados de Estoque:
  Data Vencimento  Valor Presente    Status
0      2027-05-04        424.0903  A vencer
1      2026-07-04        504.5277  A vencer
2      2029-09-04        315.1219  A vencer
3      2026-01-04        122.1097  A vencer
4      2026-06-04        111.3383  A vencer

Intervalo de datas no Estoque:
  - Data de Vencimento Mais Antiga: 02/03/2024
  - Data de Vencimento Mais Recente: 26/10/2033
--------------------------------------------------

--- INSPECIONANDO DADOS DE DESPESAS ---

Formato das Despesas: 146 linhas, 3 colunas.

Amostra dos dados de Despesas:
  Data Pagamento     Categoria      Valor
0     2025-05-09  PROV ADM      -26221.07
1     2025-05-09  P

In [10]:
import pandas as pd
import json
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS (sem alterações) ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- BLOCO DE EXECUÇÃO DA DEPURAÇÃO FINAL ---
# (Assumindo que você já tem as funções de carregamento definidas na sua sessão)

# 1. Carrega todos os dados#
df_estoque_completo = df_estoque
df_despesas_completo = df_despesas
arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))
arquivo_recente = max(arquivos_carteira, key=os.path.getctime)
dados_carteira_recente = carregar_dados_carteira(arquivo_recente)
data_referencia = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

# 2. Gera a "Lista Mestra" de fluxos de caixa
df_fluxos_detalhados = preparar_fluxos_detalhados(
    dados_carteira_recente, 
    df_estoque_completo, 
    df_despesas_completo
)

print("\n" + "="*50)
print("--- DEPURAÇÃO FINAL: INSPECIONANDO A 'LISTA MESTRA' DE FLUXOS DE CAIXA ---")
print("="*50)

if df_fluxos_detalhados.empty:
    print("\nAVISO: A lista mestra de fluxos de caixa está vazia. Nenhum evento foi adicionado.")
else:
    print(f"\nTotal de eventos de caixa encontrados: {len(df_fluxos_detalhados)}")
    
    print("\n--- Análise por Categoria ---")
    
    # Verifica cada categoria e mostra uma amostra
    categorias = df_fluxos_detalhados['categoria'].unique()
    
    print(f"\nCategorias encontradas na lista: {list(categorias)}")
    
    for cat in ['Conta Corr.', 'Fundo inv.', 'Pagamentos Estoque', 'Despesas', 'Amort./ Resgat. Cota']:
        print("-" * 20)
        print(f"Analisando Categoria: '{cat}'")
        df_cat = df_fluxos_detalhados[df_fluxos_detalhados['categoria'] == cat]
        
        if df_cat.empty:
            print(">>> Nenhum evento encontrado para esta categoria.")
        else:
            print(f">>> {len(df_cat)} eventos encontrados.")
            print("Amostra:")
            print(df_cat.head())
            print(f"Intervalo de Datas: de {df_cat['data'].min().strftime('%d/%m/%Y')} a {df_cat['data'].max().strftime('%d/%m/%Y')}")

# Análise específica do filtro de Status do Estoque
print("\n" + "="*50)
print("--- INVESTIGAÇÃO EXTRA: FILTRO DE STATUS DO ESTOQUE ---")
print("="*50)
if not df_estoque_completo.empty:
    data_limite_curto_prazo = data_referencia + timedelta(days=63)
    
    estoque_curto_prazo = df_estoque_completo[
        (df_estoque_completo['Data Vencimento'] >= data_referencia) &
        (df_estoque_completo['Data Vencimento'] <= data_limite_curto_prazo)
    ]
    
    if estoque_curto_prazo.empty:
        print("Nenhum pagamento de estoque programado para os próximos 63 dias.")
    else:
        print("Status das parcelas de estoque que vencem nos próximos 63 dias:")
        print(estoque_curto_prazo['Status'].value_counts())


--- DEPURAÇÃO FINAL: INSPECIONANDO A 'LISTA MESTRA' DE FLUXOS DE CAIXA ---

Total de eventos de caixa encontrados: 1552972

--- Análise por Categoria ---

Categorias encontradas na lista: ['Conta Corr.', 'Fundo inv.', 'Títulos Renda Fixa', 'Pagamentos Estoque', 'Despesas', 'Amort./ Resgat. Cota']
--------------------
Analisando Categoria: 'Conta Corr.'
>>> 1 eventos encontrados.
Amostra:
        data    categoria   valor
0 2025-08-18  Conta Corr.  1000.0
Intervalo de Datas: de 18/08/2025 a 18/08/2025
--------------------
Analisando Categoria: 'Fundo inv.'
>>> 1 eventos encontrados.
Amostra:
        data   categoria      valor
1 2025-08-19  Fundo inv.  661422.32
Intervalo de Datas: de 19/08/2025 a 19/08/2025
--------------------
Analisando Categoria: 'Pagamentos Estoque'
>>> 1552929 eventos encontrados.
Amostra:
        data           categoria     valor
3 2027-05-04  Pagamentos Estoque  424.0903
4 2026-07-04  Pagamentos Estoque  504.5277
5 2029-09-04  Pagamentos Estoque  315.1219
6 20

In [13]:
import pandas as pd
import json
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS PARA AS FONTES DE DADOS FINAIS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÕES DE CARREGAMENTO (Completas) ---
def carregar_dados_carteira(caminho_arquivo):
    with open(caminho_arquivo, 'r', encoding='utf-8') as f:
        return json.load(f)[0]

def processar_dados_estoque(caminho_pasta):
    print("Carregando dados de estoque...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    all_dfs = []
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                df['Valor Presente'] = pd.to_numeric(df['Valor Presente'].astype(str).str.replace(',', '.'), errors='coerce')
                df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception as e: print(f"Erro ao processar o arquivo de estoque {os.path.basename(file)}: {e}")
    if not all_dfs: return pd.DataFrame()
    return pd.concat(all_dfs, ignore_index=True)

def processar_dados_despesas(caminho_pasta):
    print("Carregando e consolidando arquivos de despesas...")
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame()
    lista_df_despesas = []
    for arquivo in arquivos_excel:
        try:
            df = pd.read_excel(arquivo, header=6) 
            lista_df_despesas.append(df)
        except Exception as e: print(f"Erro ao ler o arquivo de despesa {os.path.basename(arquivo)}: {e}")
    if not lista_df_despesas: return pd.DataFrame()
    df_consolidado = pd.concat(lista_df_despesas, ignore_index=True)
    df_consolidado.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'}, inplace=True)
    df_consolidado['Data Pagamento'] = pd.to_datetime(df_consolidado['Data Pagamento'], errors='coerce')
    df_consolidado['Valor'] = pd.to_numeric(df_consolidado['Valor'], errors='coerce')
    df_consolidado['Valor'] = -df_consolidado['Valor'].abs()
    return df_consolidado[['Data Pagamento', 'Categoria', 'Valor']].dropna()

# --- FUNÇÃO DE PREPARAÇÃO DE FLUXOS (COM FILTRO CORRIGIDO) ---
def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # Disponibilidades
    disponibilidade = next((a['Disponibilidade'] for a in dados_carteira_recente['ativos'] if 'Disponibilidade' in a and a['Disponibilidade']), None)
    if disponibilidade:
        fluxos.append({'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo']})

    cotas = next((a['Cotas'] for a in dados_carteira_recente['ativos'] if 'Cotas' in a and a['Cotas']), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos.append({'data': data_base + timedelta(days=1), 'categoria': 'Fundo inv.', 'valor': cota['valorBruto']})
            
    # Pagamentos do Estoque (COM FILTRO CORRIGIDO)
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Presente', 'Status'])
        
        # ### A CORREÇÃO ESTÁ AQUI ###
        # Inclui todos os status que representam uma entrada de caixa esperada
        status_a_incluir = ['A vencer', 'Previsto', 'Vencido']
        estoque_filtrado = estoque_valido[estoque_valido['Status'].isin(status_a_incluir)]
        
        for _, row in estoque_filtrado.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Pagamentos Estoque', 'valor': row['Valor Presente']})

    # Necessidades (apenas as que vencem no futuro)
    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base]
        for _, despesa in despesas_futuras.iterrows():
            fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas', 'valor': despesa['Valor']})
            
    # Amortização de cotas (não muda)
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            fluxos.append({'data': data_pagamento, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao})
            
    return pd.DataFrame(fluxos)

# --- FUNÇÃO DE GERAÇÃO DO RELATÓRIO (sem alterações) ---
def gerar_relatorio_d_mais(df_fluxos, data_base):
    """
    Gera a tabela final no formato D+N solicitado pelo chefe.
    """
    # ### ALTERAÇÃO AQUI: Adicione um prazo longo para o teste ###
    prazos = [0, 1, 5, 10, 21, 63, 1200] # Adicionamos 1200 dias
    # ###########################################################
    
    colunas_relatorio = ['Conta Corr.', 'Fundo inv.', 'Títulos Renda Fixa', 'Pagamentos Estoque', 'Despesas', 'Amort./ Resgat. Cota']
    
    linhas_relatorio = []

    for dias in prazos:
        data_limite = data_base + timedelta(days=dias)
        
        fluxos_ate_data = df_fluxos[df_fluxos['data'] <= data_limite]
        saldos_acumulados = fluxos_ate_data.groupby('categoria')['valor'].sum()
        
        linha = {'Prazo': f"D+{dias}"}
        for col in colunas_relatorio:
            linha[col] = saldos_acumulados.get(col, 0.0)
        
        linhas_relatorio.append(linha)
        
    df_relatorio = pd.DataFrame(linhas_relatorio).set_index('Prazo')
    
    df_relatorio['Disponibilidades'] = df_relatorio['Conta Corr.'] + df_relatorio['Fundo inv.'] + df_relatorio['Títulos Renda Fixa'] + df_relatorio['Pagamentos Estoque']
    df_relatorio['Necessidades'] = df_relatorio['Despesas'] + df_relatorio['Amort./ Resgat. Cota']
    df_relatorio['Caixa Projetado'] = df_relatorio['Disponibilidades'] + df_relatorio['Necessidades']
    
    return df_relatorio.applymap('{:,.2f}'.format)
# --- BLOCO DE EXECUÇÃO FINAL ---
if __name__ == "__main__":
    #df_estoque = processar_dados_estoque(PATH_ESTOQUE)
    #df_despesas = processar_dados_despesas(PATH_DESPESAS)
    
    arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))
    if not arquivos_carteira:
        print("ERRO: Nenhum arquivo de carteira encontrado.")
        exit()
    arquivo_recente = max(arquivos_carteira, key=os.path.getctime)
    print(f"\nUsando a carteira mais recente para a projeção: {os.path.basename(arquivo_recente)}")

    dados_carteira_recente = carregar_dados_carteira(arquivo_recente)
    data_referencia = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)
    
    relatorio_final = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia)

    print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (VERSÃO FINAL CORRIGIDA) ---")
    print(relatorio_final.to_string())


Usando a carteira mais recente para a projeção: carteira_52203615000119_20250818.json

--- PROJEÇÃO DE FLUXO DE CAIXA (VERSÃO FINAL CORRIGIDA) ---
       Conta Corr.  Fundo inv. Títulos Renda Fixa Pagamentos Estoque Despesas Amort./ Resgat. Cota Disponibilidades     Necessidades Caixa Projetado
Prazo                                                                                                                                               
D+0       1,000.00        0.00               0.00      11,038,307.04     0.00                 0.00    11,039,307.04             0.00   11,039,307.04
D+1       1,000.00  661,422.32               0.00      11,059,373.74     0.00                 0.00    11,721,796.06             0.00   11,721,796.06
D+5       1,000.00  661,422.32               0.00      12,759,984.37     0.00                 0.00    13,422,406.69             0.00   13,422,406.69
D+10      1,000.00  661,422.32               0.00      13,406,135.03     0.00                 0.00    14,06

C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\1338505837.py:132: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_relatorio.applymap('{:,.2f}'.format)


In [15]:
import json
import os
import glob
from datetime import datetime

# --- CAMINHOS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"

# --- BLOCO DE DEPURAÇÃO CORRIGIDO ---

print("--- DEPURAÇÃO CORRIGIDA: Encontrando o arquivo de carteira pela data interna ---")

# 1. Encontra todos os arquivos .json na pasta
arquivos_carteira = glob.glob(os.path.join(PATH_CARTEIRAS_HISTORICO, "*.json"))

if not arquivos_carteira:
    print("ERRO: Nenhum arquivo de carteira (.json) encontrado na pasta.")
else:
    arquivo_mais_recente = None
    data_mais_recente = None

    # 2. Itera sobre cada arquivo para encontrar a data mais recente
    print(f"Analisando {len(arquivos_carteira)} arquivos para encontrar o mais recente...")
    for arquivo in arquivos_carteira:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                # Carrega o JSON corretamente
                dados = json.load(f)
                # Acessa a data interna do relatório
                data_atual_str = dados[0]['carteiras'][0]['dataAtual']
                data_atual_obj = datetime.strptime(data_atual_str.split('T')[0], '%Y-%m-%d')

                # Compara com a data mais recente encontrada até agora
                if data_mais_recente is None or data_atual_obj > data_mais_recente:
                    data_mais_recente = data_atual_obj
                    arquivo_mais_recente = arquivo
        except (json.JSONDecodeError, IndexError, KeyError, TypeError) as e:
            print(f"AVISO: Não foi possível ler a data do arquivo '{os.path.basename(arquivo)}'. Erro: {e}")

    if not arquivo_mais_recente:
        print("ERRO: Não foi possível determinar o arquivo de carteira mais recente.")
    else:
        # 3. Agora que temos o arquivo correto, fazemos a inspeção
        print("\n------------------------------------------------------------------")
        print(f"✅ Arquivo mais recente encontrado (pela data interna): {os.path.basename(arquivo_mais_recente)}")
        print(f"Data do Relatório (D+0): {data_mais_recente.strftime('%d/%m/%Y')}")
        print("------------------------------------------------------------------")
        
        print("\n--- Inspecionando o conteúdo deste arquivo para 'RendaFixa' ---")
        
        with open(arquivo_mais_recente, 'r', encoding='utf-8') as f:
            dados_carteira_recente = json.load(f)[0]

        # Tenta extrair a seção "RendaFixa"
        renda_fixa_bloco = next(
            (ativo.get('RendaFixa') for ativo in dados_carteira_recente.get('ativos', []) if 'RendaFixa' in ativo and ativo.get('RendaFixa')),
            None
        )

        if renda_fixa_bloco:
            print("\n✅ SUCESSO: A seção 'RendaFixa' foi encontrada no arquivo!")
            print("Conteúdo:")
            print(json.dumps(renda_fixa_bloco, indent=4, ensure_ascii=False))
        else:
            print("\n❌ FALHA: A seção 'RendaFixa' NÃO foi encontrada ou está vazia (null) no arquivo de carteira deste dia.")
            print("Isso confirma que a coluna no relatório final está corretamente zerada para esta data de referência.")
        print("------------------------------------------------------------------")

--- DEPURAÇÃO CORRIGIDA: Encontrando o arquivo de carteira pela data interna ---
Analisando 10 arquivos para encontrar o mais recente...

------------------------------------------------------------------
✅ Arquivo mais recente encontrado (pela data interna): carteira_52203615000119_20250829.json
Data do Relatório (D+0): 29/08/2025
------------------------------------------------------------------

--- Inspecionando o conteúdo deste arquivo para 'RendaFixa' ---

✅ SUCESSO: A seção 'RendaFixa' foi encontrada no arquivo!
Conteúdo:
{
    "tipo": "Renda Fixa",
    "plTotal": 111802909,
    "ativos": [
        {
            "carteira": "15543",
            "nome": "FIDC FCT II SR2",
            "nomeMercado": "CETIP",
            "tipo": "Outros",
            "codigoCustodia": "09H00003977-",
            "emissor": "******",
            "dataCompra": "2024-12-16",
            "dataEmissao": "2024-12-16",
            "dataVencimento": "2028-12-18",
            "indexador": "DI1",
           

In [16]:
import pandas as pd
import json
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS PARA AS FONTES DE DADOS FINAIS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÕES DE CARREGAMENTO FINAIS ---
def encontrar_e_carregar_carteira_recente(caminho_pasta):
    """
    Encontra o arquivo de carteira mais recente pela data interna e carrega seus dados.
    """
    print("Procurando o arquivo de carteira mais recente...")
    arquivos_json = glob.glob(os.path.join(caminho_pasta, "*.json"))
    if not arquivos_json:
        return None, None

    arquivo_mais_recente = None
    data_mais_recente = None

    for arquivo in arquivos_json:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)
                data_atual_str = dados[0]['carteiras'][0]['dataAtual']
                data_atual_obj = datetime.strptime(data_atual_str.split('T')[0], '%Y-%m-%d')
                if data_mais_recente is None or data_atual_obj > data_mais_recente:
                    data_mais_recente = data_atual_obj
                    arquivo_mais_recente = arquivo
        except Exception:
            continue
    
    if arquivo_mais_recente:
        print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_mais_recente)}")
        with open(arquivo_mais_recente, 'r', encoding='utf-8') as f:
            dados_carteira = json.load(f)[0]
        return dados_carteira, data_mais_recente
    
    return None, None


def processar_dados_estoque(caminho_pasta):
    # (função completa que já usamos, sem alterações)
    print("Carregando dados de estoque...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    all_dfs = []
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                df['Valor Presente'] = pd.to_numeric(df['Valor Presente'].astype(str).str.replace(',', '.'), errors='coerce')
                df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception:
            continue
    if not all_dfs: return pd.DataFrame()
    return pd.concat(all_dfs, ignore_index=True)

def processar_dados_despesas(caminho_pasta):
    # (função completa que já usamos, sem alterações)
    print("Carregando e consolidando arquivos de despesas...")
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame()
    lista_df_despesas = []
    for arquivo in arquivos_excel:
        try:
            df = pd.read_excel(arquivo, header=6) 
            lista_df_despesas.append(df)
        except Exception:
            continue
    if not lista_df_despesas: return pd.DataFrame()
    df_consolidado = pd.concat(lista_df_despesas, ignore_index=True)
    df_consolidado.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'}, inplace=True)
    df_consolidado['Data Pagamento'] = pd.to_datetime(df_consolidado['Data Pagamento'], errors='coerce')
    df_consolidado['Valor'] = pd.to_numeric(df_consolidado['Valor'], errors='coerce')
    df_consolidado['Valor'] = -df_consolidado['Valor'].abs()
    return df_consolidado[['Data Pagamento', 'Categoria', 'Valor']].dropna()


# --- FUNÇÃO DE PREPARAÇÃO DE FLUXOS (FINAL) ---
def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # ... (lógica completa e corrigida que já usamos) ...
    disponibilidade = next((a.get('Disponibilidade') for a in dados_carteira_recente['ativos'] if a.get('Disponibilidade')), None)
    if disponibilidade:
        fluxos.append({'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo']})

    cotas = next((a.get('Cotas') for a in dados_carteira_recente['ativos'] if a.get('Cotas')), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos.append({'data': data_base + timedelta(days=1), 'categoria': 'Fundo inv.', 'valor': cota['valorBruto']})
            
    renda_fixa = next((a.get('RendaFixa') for a in dados_carteira_recente['ativos'] if a.get('RendaFixa')), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']:
            fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Títulos Renda Fixa', 'valor': titulo['mercadoAtual']})

    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Presente', 'Status'])
        status_a_incluir = ['A vencer', 'Previsto', 'Vencido']
        estoque_filtrado = estoque_valido[estoque_valido['Status'].isin(status_a_incluir)]
        for _, row in estoque_filtrado.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Pagamentos Estoque', 'valor': row['Valor Presente']})

    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base]
        for _, despesa in despesas_futuras.iterrows():
            fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas', 'valor': despesa['Valor']})
            
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            fluxos.append({'data': data_pagamento, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao})
            
    return pd.DataFrame(fluxos)

# --- FUNÇÃO DE GERAÇÃO DO RELATÓRIO (FINAL) ---
def gerar_relatorio_d_mais(df_fluxos, data_base):
    # Adicionando prazos longos para ver todos os fluxos
    prazos = [0, 1, 5, 10, 21, 63, 365, 730, 1200]
    colunas_relatorio = ['Conta Corr.', 'Fundo inv.', 'Títulos Renda Fixa', 'Pagamentos Estoque', 'Despesas', 'Amort./ Resgat. Cota']
    
    linhas_relatorio = []
    for dias in prazos:
        data_limite = data_base + timedelta(days=dias)
        fluxos_ate_data = df_fluxos[df_fluxos['data'] <= data_limite]
        saldos_acumulados = fluxos_ate_data.groupby('categoria')['valor'].sum()
        
        linha = {'Prazo': f"D+{dias}"}
        for col in colunas_relatorio:
            linha[col] = saldos_acumulados.get(col, 0.0)
        
        linhas_relatorio.append(linha)
        
    df_relatorio = pd.DataFrame(linhas_relatorio).set_index('Prazo')
    
    df_relatorio['Disponibilidades'] = df_relatorio[colunas_relatorio[:4]].sum(axis=1)
    df_relatorio['Necessidades'] = df_relatorio[colunas_relatorio[4:]].sum(axis=1)
    df_relatorio['Caixa Projetado'] = df_relatorio['Disponibilidades'] + df_relatorio['Necessidades']
    
    return df_relatorio.applymap('{:,.2f}'.format)

# --- BLOCO DE EXECUÇÃO FINAL ---
if __name__ == "__main__":
    dados_carteira_recente, data_referencia = encontrar_e_carregar_carteira_recente(PATH_CARTEIRAS_HISTORICO)
    
    if dados_carteira_recente:
        df_estoque = processar_dados_estoque(PATH_ESTOQUE)
        df_despesas = processar_dados_despesas(PATH_DESPESAS)
        
        df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)
        
        relatorio_final = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia)

        print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (VERSÃO DEFINITIVA) ---")
        print(relatorio_final.to_string())

Procurando o arquivo de carteira mais recente...
Usando a carteira mais recente para a projeção: carteira_52203615000119_20250829.json
Carregando dados de estoque...
Carregando e consolidando arquivos de despesas...

--- PROJEÇÃO DE FLUXO DE CAIXA (VERSÃO DEFINITIVA) ---
       Conta Corr.  Fundo inv. Títulos Renda Fixa Pagamentos Estoque Despesas Amort./ Resgat. Cota Disponibilidades     Necessidades Caixa Projetado
Prazo                                                                                                                                               
D+0       1,000.00        0.00               0.00      13,448,387.14     0.00                 0.00    13,449,387.14             0.00   13,449,387.14
D+1       1,000.00  440,304.72               0.00      13,500,473.05     0.00                 0.00    13,941,777.77             0.00   13,941,777.77
D+5       1,000.00  440,304.72               0.00      13,728,677.28     0.00                 0.00    14,169,982.00             0.00

C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\175266382.py:157: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_relatorio.applymap('{:,.2f}'.format)


In [17]:
import pandas as pd
import os
import glob
from datetime import datetime

# --- CAMINHO PARA A PASTA COM OS ARQUIVOS DE DESPESAS ---
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"

# --- BLOCO DE DEPURAÇÃO ---

print("--- AUDITORIA DE DADOS: Verificando arquivos de Despesas ---")

# 1. Encontra todos os arquivos de despesas na pasta e subpastas
arquivos_excel = glob.glob(os.path.join(PATH_DESPESAS, '**/Despesas_Consolidadas.xlsx'), recursive=True)

if not arquivos_excel:
    print("\nERRO CRÍTICO: Nenhum arquivo 'Despesas_Consolidadas.xlsx' foi encontrado na pasta especificada.")
else:
    print(f"Encontrados {len(arquivos_excel)} arquivos de despesas para análise:")
    
    lista_de_datas = []
    detalhes_arquivos = []

    # 2. Itera sobre cada arquivo para extrair seu intervalo de datas
    for arquivo in arquivos_excel:
        try:
            # Lê o arquivo, pulando as 6 primeiras linhas
            df = pd.read_excel(arquivo, header=6)
            
            # Converte a coluna 'Data Pagamento' para o formato de data
            df['Data Pagamento'] = pd.to_datetime(df['Data Pagamento'], errors='coerce')
            
            # Remove linhas onde a data não pôde ser lida
            df.dropna(subset=['Data Pagamento'], inplace=True)
            
            if not df.empty:
                data_min = df['Data Pagamento'].min()
                data_max = df['Data Pagamento'].max()
                
                detalhes_arquivos.append({
                    "Arquivo": os.path.basename(os.path.dirname(arquivo)), # Pega o nome da pasta pai
                    "Data Mais Antiga": data_min.strftime('%d/%m/%Y'),
                    "Data Mais Recente": data_max.strftime('%d/%m/%Y')
                })
                # Adiciona todas as datas à lista geral
                lista_de_datas.extend(df['Data Pagamento'].tolist())
            else:
                detalhes_arquivos.append({
                    "Arquivo": os.path.basename(os.path.dirname(arquivo)),
                    "Data Mais Antiga": "N/A",
                    "Data Mais Recente": "N/A"
                })

        except Exception as e:
            print(f"\nAVISO: Falha ao processar o arquivo '{os.path.basename(arquivo)}'. Erro: {e}")

    # 3. Apresenta os resultados da auditoria
    print("\n--- Detalhamento por Arquivo ---")
    df_detalhes = pd.DataFrame(detalhes_arquivos)
    print(df_detalhes.to_string(index=False))

    if lista_de_datas:
        data_geral_mais_recente = max(lista_de_datas)
        print("\n------------------------------------------------------------------")
        print(">>> CONCLUSÃO DA AUDITORIA:")
        print(f">>> A DATA DE PAGAMENTO MAIS RECENTE encontrada em TODOS os arquivos é: {data_geral_mais_recente.strftime('%d/%m/%Y')}")
        print("------------------------------------------------------------------")
        print("\nCompare esta data com a data de referência da sua projeção (D+0).")
        print("Se esta data for anterior à data de referência, o resultado 'zero' no relatório está correto, pois não há despesas futuras nos dados.")
    else:
        print("\nNenhuma data de pagamento válida foi encontrada em nenhum dos arquivos.")

--- AUDITORIA DE DADOS: Verificando arquivos de Despesas ---
Encontrados 47 arquivos de despesas para análise:

--- Detalhamento por Arquivo ---
                                Arquivo Data Mais Antiga Data Mais Recente
    143063-Despesas_Consolidadas_143063       09/05/2025        09/05/2025
    143489-Despesas_Consolidadas_143489       09/05/2025        09/05/2025
    143490-Despesas_Consolidadas_143490       09/05/2025        09/05/2025
    143491-Despesas_Consolidadas_143491       09/05/2025        09/05/2025
    143492-Despesas_Consolidadas_143492       09/05/2025        09/05/2025
    143493-Despesas_Consolidadas_143493       09/05/2025        09/05/2025
    143494-Despesas_Consolidadas_143494       09/05/2025        09/05/2025
    143495-Despesas_Consolidadas_143495       09/05/2025        09/05/2025
    143496-Despesas_Consolidadas_143496       09/05/2025        09/05/2025
    143497-Despesas_Consolidadas_143497       09/05/2025        09/05/2025
143497-Despesas_Consolidadas_1

In [29]:
import pandas as pd
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

# --- CAMINHOS ---
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"
# Defina a data de referência da sua projeção para o teste
DATA_BASE_PROJECAO_TESTE = datetime(2025, 8, 29)


def depurar_processamento_despesas(caminho_pasta, data_base_projecao, meses_a_projetar=12):
    """
    Função de depuração para inspecionar o processamento de despesas passo a passo.
    """
    print("--- INICIANDO DEPURAÇÃO DO PROCESSAMENTO DE DESPESAS ---")
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel:
        print("1. Nenhum arquivo de despesa encontrado.")
        return pd.DataFrame()

    df_mais_recente = None
    data_relatorio_recente = None
    arquivo_usado = None

    # 1. Encontra o arquivo com a 'Data' (data de geração do relatório) mais recente
    for arquivo in arquivos_excel:
        try:
            df_temp = pd.read_excel(arquivo, header=6)
            df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce')
            data_atual = df_temp['Data'].max()
            if data_relatorio_recente is None or data_atual > data_relatorio_recente:
                data_relatorio_recente = data_atual
                df_mais_recente = df_temp
                arquivo_usado = os.path.basename(os.path.dirname(arquivo)) # Pega o nome da pasta
        except Exception:
            continue
            
    if df_mais_recente is None:
        print("1. Nenhum arquivo de despesa válido pôde ser lido.")
        return pd.DataFrame()

    print(f"1. Arquivo de despesas mais recente identificado: '{arquivo_usado}' (gerado em {data_relatorio_recente.strftime('%d/%m/%Y')})")
    
    # 2. Limpa o DataFrame mais recente para usá-lo como "molde"
    df_base = df_mais_recente.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'})
    df_base['Valor'] = -pd.to_numeric(df_base['Valor'], errors='coerce').abs()
    df_base = df_base[['Categoria', 'Valor']].dropna()
    
    print("\n2. Despesas 'molde' extraídas deste arquivo para a projeção:")
    print(df_base.to_string())
    
    despesas_finais = []
    
    # 3. Projeta as despesas para os meses FUTUROS
    print(f"\n3. Projetando despesas para os próximos {meses_a_projetar} meses, a partir de {data_base_projecao.strftime('%d/%m/%Y')}")
    
    mes_inicial_projecao = data_base_projecao.replace(day=1)

    for i in range(meses_a_projetar):
        mes_atual = mes_inicial_projecao + relativedelta(months=i)
        data_pagamento_proj = mes_atual.replace(day=5)
        
        while data_pagamento_proj.weekday() >= 5:
            data_pagamento_proj += timedelta(days=1)
        
        # A projeção só deve incluir datas futuras
        if data_pagamento_proj >= data_base_projecao:
            for _, row in df_base.iterrows():
                despesas_finais.append({
                    'Data Pagamento': data_pagamento_proj,
                    'Categoria': row['Categoria'],
                    'Valor': row['Valor']
                })

    df_final = pd.DataFrame(despesas_finais)
    
    if df_final.empty:
        print("\n4. Nenhuma despesa futura foi gerada. A projeção de despesas está vazia.")
    else:
        print("\n4. Tabela final de despesas futuras geradas:")
        print(df_final.head(10).to_string()) # Mostra as 10 primeiras despesas projetadas
    
    print("\n--- FIM DA DEPURAÇÃO ---")
    return df_final

# --- BLOCO DE EXECUÇÃO DA DEPURAÇÃO ---
df_despesas_depuradas = depurar_processamento_despesas(PATH_DESPESAS, DATA_BASE_PROJECAO_TESTE)

--- INICIANDO DEPURAÇÃO DO PROCESSAMENTO DE DESPESAS ---


C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\2373365679.py:31: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce')
C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\2373365679.py:31: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce')
C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\2373365679.py:31: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce')
C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\2373365679.py:31: UserWarning: Parsing dates in %d/%m/%Y format 

1. Arquivo de despesas mais recente identificado: '143501-Despesas_Consolidadas_143501' (gerado em 08/12/2025)

2. Despesas 'molde' extraídas deste arquivo para a projeção:
      Categoria     Valor
0  PROV ADM      -9305.14
1  PROV GESTÃO  -31017.12
2  TX ESCRT FIX   -794.32

3. Projetando despesas para os próximos 12 meses, a partir de 29/08/2025

4. Tabela final de despesas futuras geradas:
  Data Pagamento     Categoria     Valor
0     2025-09-05  PROV ADM      -9305.14
1     2025-09-05  PROV GESTÃO  -31017.12
2     2025-09-05  TX ESCRT FIX   -794.32
3     2025-10-06  PROV ADM      -9305.14
4     2025-10-06  PROV GESTÃO  -31017.12
5     2025-10-06  TX ESCRT FIX   -794.32
6     2025-11-05  PROV ADM      -9305.14
7     2025-11-05  PROV GESTÃO  -31017.12
8     2025-11-05  TX ESCRT FIX   -794.32
9     2025-12-05  PROV ADM      -9305.14

--- FIM DA DEPURAÇÃO ---


C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\2373365679.py:31: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce')


In [27]:
import pandas as pd
import json
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS PARA AS FONTES DE DADOS FINAIS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÃO DE PROCESSAMENTO DE DESPESAS (VERSÃO FINAL E INTELIGENTE) ---


def processar_dados_despesas(caminho_pasta, data_base_projecao, meses_a_projetar=12):
    """
    Encontra o relatório de despesas mais recente e projeta suas provisões para o futuro.
    """
    print("\nCarregando despesas com a lógica final...")
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel:
        return pd.DataFrame()

    df_mais_recente = None
    data_relatorio_recente = None

    # 1. Encontra o arquivo com a 'Data' (data de geração) mais recente
    for arquivo in arquivos_excel:
        try:
            df_temp = pd.read_excel(arquivo, header=6)
            df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce')
            data_atual = df_temp['Data'].max()
            if data_relatorio_recente is None or data_atual > data_relatorio_recente:
                data_relatorio_recente = data_atual
                df_mais_recente = df_temp
        except Exception:
            continue
            
    if df_mais_recente is None:
        print("Nenhum arquivo de despesa válido encontrado.")
        return pd.DataFrame()

    print(f"Usando o relatório de despesas gerado em: {data_relatorio_recente.strftime('%d/%m/%Y')}")
    
    # 2. Limpa o DataFrame mais recente e o usa como base
    df_base = df_mais_recente.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor', 'Data Pagamento': 'Data_Pagamento'})
    df_base['Valor'] = -pd.to_numeric(df_base['Valor'], errors='coerce').abs()
    df_base = df_base[['Categoria', 'Valor']].dropna()
    
    despesas_finais = []
    
    # 3. Projeta as despesas para os próximos meses
    print(f"Projetando {len(df_base)} categorias de despesas para os próximos {meses_a_projetar} meses.")
    
    # Começa a projeção no primeiro mês após ou igual à data de referência
    mes_inicial_projecao = data_base_projecao.replace(day=1)

    for i in range(meses_a_projetar):
        mes_atual = mes_inicial_projecao + relativedelta(months=i)
        # Define a data de pagamento para o 5º dia
        data_pagamento_proj = mes_atual.replace(day=5)
        
        # Ajusta para o próximo dia útil
        while data_pagamento_proj.weekday() >= 5: # 5=Sábado, 6=Domingo
            data_pagamento_proj += timedelta(days=1)

        for _, row in df_base.iterrows():
            despesas_finais.append({
                'Data Pagamento': data_pagamento_proj,
                'Categoria': row['Categoria'],
                'Valor': row['Valor']
            })

    df_final = pd.DataFrame(despesas_finais)
    print(f"{len(df_final)} eventos de despesas futuras foram criados.")
    return df_final


# --- FUNÇÃO DE PREPARAÇÃO DE FLUXOS (VERSÃO OTIMIZADA) ---
def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    """
    Cria uma lista de TODOS os eventos de fluxo de caixa futuros,
    usando operações vetorizadas para alta performance.
    """
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    lista_de_fluxos_df = []

    # Disponibilidades (da carteira)
    disponibilidade = next((a.get('Disponibilidade') for a in dados_carteira_recente['ativos'] if a.get('Disponibilidade')), None)
    if disponibilidade:
        df_cc = pd.DataFrame([{'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo']}])
        lista_de_fluxos_df.append(df_cc)

    cotas = next((a.get('Cotas') for a in dados_carteira_recente['ativos'] if a.get('Cotas')), None)
    if cotas:
        df_cotas = pd.DataFrame([{'data': data_base + timedelta(days=1), 'categoria': 'Fundo inv.', 'valor': c['valorBruto']} for c in cotas['ativos']])
        lista_de_fluxos_df.append(df_cotas)
            
    renda_fixa = next((a.get('RendaFixa') for a in dados_carteira_recente['ativos'] if a.get('RendaFixa')), None)
    if renda_fixa:
        df_rf = pd.DataFrame([{'data': datetime.strptime(t['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Títulos Renda Fixa', 'valor': t['mercadoAtual']} for t in renda_fixa['ativos']])
        lista_de_fluxos_df.append(df_rf)

    # Pagamentos do Estoque (OTIMIZAÇÃO APLICADA AQUI)
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Presente', 'Status'])
        status_a_incluir = ['A vencer', 'Previsto', 'Vencido']
        estoque_filtrado = estoque_valido[estoque_valido['Status'].isin(status_a_incluir)].copy()
        
        # Cria o DataFrame de fluxos do estoque diretamente, sem loop
        estoque_fluxos = estoque_filtrado[['Data Vencimento', 'Valor Presente']].rename(columns={'Data Vencimento': 'data', 'Valor Presente': 'valor'})
        estoque_fluxos['categoria'] = 'Pagamentos Estoque'
        lista_de_fluxos_df.append(estoque_fluxos)

    # Necessidades
    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base].copy()
        despesas_fluxos = despesas_futuras[['Data Pagamento', 'Valor']].rename(columns={'Data Pagamento': 'data', 'Valor': 'valor'})
        despesas_fluxos['categoria'] = 'Despesas'
        lista_de_fluxos_df.append(despesas_fluxos)
            
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        amort_fluxos_lista = []
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            amort_fluxos_lista.append({'data': data_pagamento, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao})
        df_amort = pd.DataFrame(amort_fluxos_lista)
        lista_de_fluxos_df.append(df_amort)
            
    # Concatena todos os DataFrames de uma só vez
    return pd.concat(lista_de_fluxos_df, ignore_index=True)


# --- DEMAIS FUNÇÕES (sem alterações) ---
# copiar dps

# --- BLOCO DE EXECUÇÃO ---
if __name__ == "__main__":
    # (O bloco de execução principal permanece o mesmo,
    # ele simplesmente chamará a nova e mais inteligente função 'processar_dados_despesas')
    
    # dados_carteira_recente, data_referencia = encontrar_e_carregar_carteira_recente(PATH_CARTEIRAS_HISTORICO)
    # df_estoque = processar_dados_estoque(PATH_ESTOQUE)
    # df_despesas = processar_dados_despesas(PATH_DESPESAS)
    
    df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)
    relatorio_final = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia)
    
    print(relatorio_final.to_string())


       Conta Corr.  Fundo inv. Títulos Renda Fixa Pagamentos Estoque Despesas Amort./ Resgat. Cota Disponibilidades     Necessidades Caixa Projetado
Prazo                                                                                                                                               
D+0       1,000.00        0.00               0.00      13,448,387.14     0.00                 0.00    13,449,387.14             0.00   13,449,387.14
D+1       1,000.00  440,304.72               0.00      13,500,473.05     0.00                 0.00    13,941,777.77             0.00   13,941,777.77
D+5       1,000.00  440,304.72               0.00      13,728,677.28     0.00                 0.00    14,169,982.00             0.00   14,169,982.00
D+10      1,000.00  440,304.72               0.00      14,060,107.05     0.00                 0.00    14,501,411.77             0.00   14,501,411.77
D+21      1,000.00  440,304.72               0.00      16,929,321.16     0.00                 0.00    17,3

C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\175266382.py:157: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_relatorio.applymap('{:,.2f}'.format)


In [31]:
import pandas as pd
import json
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS PARA AS FONTES DE DADOS FINAIS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÕES DE CARREGAMENTO FINAIS E CORRIGIDAS ---

def encontrar_e_carregar_carteira_recente(caminho_pasta):
    """Encontra o arquivo de carteira mais recente pela data interna e carrega seus dados."""
    print("Procurando o arquivo de carteira mais recente...")
    arquivos_json = glob.glob(os.path.join(caminho_pasta, "*.json"))
    if not arquivos_json: return None, None
    arquivo_mais_recente, data_mais_recente = None, None
    for arquivo in arquivos_json:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)
                data_atual_str = dados[0]['carteiras'][0]['dataAtual']
                data_atual_obj = datetime.strptime(data_atual_str.split('T')[0], '%Y-%m-%d')
                if data_mais_recente is None or data_atual_obj > data_mais_recente:
                    data_mais_recente, arquivo_mais_recente = data_atual_obj, arquivo
        except Exception: continue
    if arquivo_mais_recente:
        print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_mais_recente)}")
        with open(arquivo_mais_recente, 'r', encoding='utf-8') as f:
            return json.load(f)[0], data_mais_recente
    return None, None

def processar_dados_estoque(caminho_pasta):
    print("Carregando dados de estoque...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    all_dfs = []
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                df['Valor Presente'] = pd.to_numeric(df['Valor Presente'].astype(str).str.replace(',', '.'), errors='coerce')
                df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception: continue
    if not all_dfs: return pd.DataFrame()
    return pd.concat(all_dfs, ignore_index=True)

# --- FUNÇÃO DE PREPARAÇÃO DE FLUXOS (VERSÃO OTIMIZADA) ---
def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    """
    Cria uma lista de TODOS os eventos de fluxo de caixa futuros,
    usando operações vetorizadas para alta performance.
    """
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    lista_de_fluxos_df = []

    # Disponibilidades (da carteira)
    disponibilidade = next((a.get('Disponibilidade') for a in dados_carteira_recente['ativos'] if a.get('Disponibilidade')), None)
    if disponibilidade:
        df_cc = pd.DataFrame([{'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo']}])
        lista_de_fluxos_df.append(df_cc)

    cotas = next((a.get('Cotas') for a in dados_carteira_recente['ativos'] if a.get('Cotas')), None)
    if cotas:
        df_cotas = pd.DataFrame([{'data': data_base + timedelta(days=1), 'categoria': 'Fundo inv.', 'valor': c['valorBruto']} for c in cotas['ativos']])
        lista_de_fluxos_df.append(df_cotas)
            
    renda_fixa = next((a.get('RendaFixa') for a in dados_carteira_recente['ativos'] if a.get('RendaFixa')), None)
    if renda_fixa:
        df_rf = pd.DataFrame([{'data': datetime.strptime(t['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Títulos Renda Fixa', 'valor': t['mercadoAtual']} for t in renda_fixa['ativos']])
        lista_de_fluxos_df.append(df_rf)

    # Pagamentos do Estoque (OTIMIZAÇÃO APLICADA AQUI)
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Presente', 'Status'])
        status_a_incluir = ['A vencer', 'Previsto', 'Vencido']
        estoque_filtrado = estoque_valido[estoque_valido['Status'].isin(status_a_incluir)].copy()
        
        # Cria o DataFrame de fluxos do estoque diretamente, sem loop
        estoque_fluxos = estoque_filtrado[['Data Vencimento', 'Valor Presente']].rename(columns={'Data Vencimento': 'data', 'Valor Presente': 'valor'})
        estoque_fluxos['categoria'] = 'Pagamentos Estoque'
        lista_de_fluxos_df.append(estoque_fluxos)

    # Necessidades
    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base].copy()
        despesas_fluxos = despesas_futuras[['Data Pagamento', 'Valor']].rename(columns={'Data Pagamento': 'data', 'Valor': 'valor'})
        despesas_fluxos['categoria'] = 'Despesas'
        lista_de_fluxos_df.append(despesas_fluxos)
            
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        amort_fluxos_lista = []
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            amort_fluxos_lista.append({'data': data_pagamento, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao})
        df_amort = pd.DataFrame(amort_fluxos_lista)
        lista_de_fluxos_df.append(df_amort)
            
    # Concatena todos os DataFrames de uma só vez
    return pd.concat(lista_de_fluxos_df, ignore_index=True)


# --- DEMAIS FUNÇÕES FINAIS ---

def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    disponibilidade = next((a.get('Disponibilidade') for a in dados_carteira_recente['ativos'] if a.get('Disponibilidade')), None)
    if disponibilidade: fluxos.append({'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo']})
    cotas = next((a.get('Cotas') for a in dados_carteira_recente['ativos'] if a.get('Cotas')), None)
    if cotas:
        for cota in cotas['ativos']: fluxos.append({'data': data_base + timedelta(days=1), 'categoria': 'Fundo inv.', 'valor': cota['valorBruto']})
    renda_fixa = next((a.get('RendaFixa') for a in dados_carteira_recente['ativos'] if a.get('RendaFixa')), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']: fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Títulos Renda Fixa', 'valor': titulo['mercadoAtual']})
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Presente', 'Status'])
        status_a_incluir = ['A vencer', 'Previsto', 'Vencido']
        estoque_filtrado = estoque_valido[estoque_valido['Status'].isin(status_a_incluir)]
        for _, row in estoque_filtrado.iterrows(): fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Pagamentos Estoque', 'valor': row['Valor Presente']})
    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base]
        for _, despesa in despesas_futuras.iterrows(): fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas', 'valor': despesa['Valor']})
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            fluxos.append({'data': data_pagamento, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao})
    return pd.DataFrame(fluxos)

def gerar_relatorio_d_mais(df_fluxos, data_base):
    prazos = [0, 1, 5, 10, 21, 63]
    colunas_relatorio = ['Conta Corr.', 'Fundo inv.', 'Títulos Renda Fixa', 'Pagamentos Estoque', 'Despesas', 'Amort./ Resgat. Cota']
    linhas_relatorio = []
    for dias in prazos:
        data_limite = data_base + timedelta(days=dias)
        fluxos_ate_data = df_fluxos[df_fluxos['data'] <= data_limite]
        saldos_acumulados = fluxos_ate_data.groupby('categoria')['valor'].sum()
        linha = {'Prazo': f"D+{dias}"}
        for col in colunas_relatorio: linha[col] = saldos_acumulados.get(col, 0.0)
        linhas_relatorio.append(linha)
    df_relatorio = pd.DataFrame(linhas_relatorio).set_index('Prazo')
    df_relatorio['Disponibilidades'] = df_relatorio[colunas_relatorio[:4]].sum(axis=1)
    df_relatorio['Necessidades'] = df_relatorio[colunas_relatorio[4:]].sum(axis=1)
    df_relatorio['Caixa Projetado'] = df_relatorio['Disponibilidades'] + df_relatorio['Necessidades']
    return df_relatorio.applymap('{:,.2f}'.format)

# --- BLOCO DE EXECUÇÃO FINAL ---
if __name__ == "__main__":
    dados_carteira_recente, data_referencia = encontrar_e_carregar_carteira_recente(PATH_CARTEIRAS_HISTORICO)
    if dados_carteira_recente:
        df_estoque = processar_dados_estoque(PATH_ESTOQUE)
        df_despesas = processar_dados_despesas(PATH_DESPESAS, data_referencia)
        df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)
        relatorio_final = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia)
        print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (LÓGICA FINAL) ---")
        print(relatorio_final.to_string())

Procurando o arquivo de carteira mais recente...
Usando a carteira mais recente para a projeção: carteira_52203615000119_20250829.json
Carregando dados de estoque...

Carregando despesas com a lógica final...


C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\2392868282.py:34: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce')
C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\2392868282.py:34: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce')
C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\2392868282.py:34: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce')
C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\2392868282.py:34: UserWarning: Parsing dates in %d/%m/%Y format 

Usando o relatório de despesas gerado em: 08/12/2025
Projetando 3 categorias de despesas para os próximos 12 meses.
36 eventos de despesas futuras foram criados.

--- PROJEÇÃO DE FLUXO DE CAIXA (LÓGICA FINAL) ---
      Conta Corr.  Fundo inv. Títulos Renda Fixa Pagamentos Estoque    Despesas Amort./ Resgat. Cota Disponibilidades Necessidades Caixa Projetado
Prazo                                                                                                                                             
D+0      1,000.00        0.00               0.00      13,448,387.14        0.00                 0.00    13,449,387.14         0.00   13,449,387.14
D+1      1,000.00  440,304.72               0.00      13,500,473.05        0.00                 0.00    13,941,777.77         0.00   13,941,777.77
D+5      1,000.00  440,304.72               0.00      13,728,677.28        0.00                 0.00    14,169,982.00         0.00   14,169,982.00
D+10     1,000.00  440,304.72               0.00    

C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\1707903179.py:163: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_relatorio.applymap('{:,.2f}'.format)


In [32]:
import pandas as pd
import json
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS PARA AS FONTES DE DADOS FINAIS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÕES DE CARREGAMENTO FINAIS E CORRIGIDAS ---

def encontrar_e_carregar_carteira_recente(caminho_pasta):
    """Encontra o arquivo de carteira mais recente pela data interna e carrega seus dados."""
    print("Procurando o arquivo de carteira mais recente...")
    arquivos_json = glob.glob(os.path.join(caminho_pasta, "*.json"))
    if not arquivos_json: return None, None
    arquivo_mais_recente, data_mais_recente = None, None
    for arquivo in arquivos_json:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)
                data_atual_str = dados[0]['carteiras'][0]['dataAtual']
                data_atual_obj = datetime.strptime(data_atual_str.split('T')[0], '%Y-%m-%d')
                if data_mais_recente is None or data_atual_obj > data_mais_recente:
                    data_mais_recente, arquivo_mais_recente = data_atual_obj, arquivo
        except Exception: continue
    if arquivo_mais_recente:
        print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_mais_recente)}")
        with open(arquivo_mais_recente, 'r', encoding='utf-8') as f:
            return json.load(f)[0], data_mais_recente
    return None, None

def processar_dados_estoque(caminho_pasta):
    print("Carregando dados de estoque...")
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    all_dfs = []
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                df['Valor Presente'] = pd.to_numeric(df['Valor Presente'].astype(str).str.replace(',', '.'), errors='coerce')
                df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception: continue
    if not all_dfs: return pd.DataFrame()
    return pd.concat(all_dfs, ignore_index=True)

def processar_dados_despesas_final(caminho_pasta, data_base_projecao, meses_a_projetar=12):
    """Encontra o relatório de despesas mais recente e projeta suas provisões para o futuro."""
    print("\nCarregando despesas com a lógica final e corrigida...")
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame()
    df_mais_recente, data_relatorio_recente = None, None
    for arquivo in arquivos_excel:
        try:
            df_temp = pd.read_excel(arquivo, header=6)
            # ### A CORREÇÃO DEFINITIVA ESTÁ AQUI ###
            df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce', dayfirst=True)
            data_atual = df_temp['Data'].max()
            if data_relatorio_recente is None or data_atual > data_relatorio_recente:
                data_relatorio_recente, df_mais_recente = data_atual, df_temp
        except Exception: continue
    if df_mais_recente is None: return pd.DataFrame()
    print(f"Usando o relatório de despesas gerado em: {data_relatorio_recente.strftime('%d/%m/%Y')}")
    df_base = df_mais_recente.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'})
    df_base['Valor'] = -pd.to_numeric(df_base['Valor'], errors='coerce').abs()
    df_base['Data Pagamento'] = pd.to_datetime(df_base['Data Pagamento'], errors='coerce', dayfirst=True)
    df_base = df_base[['Categoria', 'Valor', 'Data Pagamento']].dropna()
    despesas_finais = []
    
    # Adiciona as despesas reais do arquivo base que são futuras
    despesas_reais_futuras = df_base[df_base['Data Pagamento'] >= data_base_projecao]
    despesas_finais.extend(despesas_reais_futuras.to_dict('records'))

    # Projeta as despesas para os meses seguintes
    mes_inicial_projecao = (data_base_projecao.replace(day=1) + relativedelta(months=1))
    for i in range(meses_a_projetar):
        mes_atual = mes_inicial_projecao + relativedelta(months=i)
        data_pagamento_proj = mes_atual.replace(day=5)
        while data_pagamento_proj.weekday() >= 5: data_pagamento_proj += timedelta(days=1)
        # Cria um "molde" das despesas do último relatório para projetar
        for _, row in df_base.iterrows():
            despesas_finais.append({'Data Pagamento': data_pagamento_proj, 'Categoria': row['Categoria'], 'Valor': row['Valor']})
            
    df_final = pd.DataFrame(despesas_finais)
    print(f"{len(df_final)} eventos de despesas futuras (reais + projetadas) foram criados.")
    return df_final

# --- DEMAIS FUNÇÕES FINAIS ---
def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    disponibilidade = next((a.get('Disponibilidade') for a in dados_carteira_recente['ativos'] if a.get('Disponibilidade')), None)
    if disponibilidade: fluxos.append({'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo']})
    cotas = next((a.get('Cotas') for a in dados_carteira_recente['ativos'] if a.get('Cotas')), None)
    if cotas:
        for cota in cotas['ativos']: fluxos.append({'data': data_base + timedelta(days=1), 'categoria': 'Fundo inv.', 'valor': cota['valorBruto']})
    renda_fixa = next((a.get('RendaFixa') for a in dados_carteira_recente['ativos'] if a.get('RendaFixa')), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']: fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Títulos Renda Fixa', 'valor': titulo['mercadoAtual']})
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Presente', 'Status'])
        status_a_incluir = ['A vencer', 'Previsto', 'Vencido']
        estoque_filtrado = estoque_valido[estoque_valido['Status'].isin(status_a_incluir)]
        for _, row in estoque_filtrado.iterrows(): fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Pagamentos Estoque', 'valor': row['Valor Presente']})
    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base]
        for _, despesa in despesas_futuras.iterrows(): fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas', 'valor': despesa['Valor']})
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            fluxos.append({'data': data_pagamento, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao})
    return pd.DataFrame(fluxos)

def gerar_relatorio_d_mais(df_fluxos, data_base):
    prazos = [0, 1, 5, 10, 21, 63, 365]
    colunas_relatorio = ['Conta Corr.', 'Fundo inv.', 'Títulos Renda Fixa', 'Pagamentos Estoque', 'Despesas', 'Amort./ Resgat. Cota']
    linhas_relatorio = []
    for dias in prazos:
        data_limite = data_base + timedelta(days=dias)
        fluxos_ate_data = df_fluxos[df_fluxos['data'] <= data_limite]
        saldos_acumulados = fluxos_ate_data.groupby('categoria')['valor'].sum()
        linha = {'Prazo': f"D+{dias}"}
        for col in colunas_relatorio: linha[col] = saldos_acumulados.get(col, 0.0)
        linhas_relatorio.append(linha)
    df_relatorio = pd.DataFrame(linhas_relatorio).set_index('Prazo')
    df_relatorio['Disponibilidades'] = df_relatorio[colunas_relatorio[:4]].sum(axis=1)
    df_relatorio['Necessidades'] = df_relatorio[colunas_relatorio[4:]].sum(axis=1)
    df_relatorio['Caixa Projetado'] = df_relatorio['Disponibilidades'] + df_relatorio['Necessidades']
    return df_relatorio.applymap('{:,.2f}'.format)

# --- BLOCO DE EXECUÇÃO FINAL ---
if __name__ == "__main__":
    dados_carteira_recente, data_referencia = encontrar_e_carregar_carteira_recente(PATH_CARTEIRAS_HISTORICO)
    if dados_carteira_recente:
        #df_estoque = processar_dados_estoque(PATH_ESTOQUE)
        df_despesas = processar_dados_despesas_final(PATH_DESPESAS, data_referencia)
        df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)
        relatorio_final = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia)
        print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (LÓGICA FINAL) ---")
        print(relatorio_final.to_string())

Procurando o arquivo de carteira mais recente...
Usando a carteira mais recente para a projeção: carteira_52203615000119_20250829.json

Carregando despesas com a lógica final e corrigida...
Usando o relatório de despesas gerado em: 01/09/2025
52 eventos de despesas futuras (reais + projetadas) foram criados.

--- PROJEÇÃO DE FLUXO DE CAIXA (LÓGICA FINAL) ---
      Conta Corr.  Fundo inv. Títulos Renda Fixa Pagamentos Estoque       Despesas Amort./ Resgat. Cota Disponibilidades    Necessidades Caixa Projetado
Prazo                                                                                                                                                   
D+0      1,000.00        0.00               0.00      13,448,387.14           0.00                 0.00    13,449,387.14            0.00   13,449,387.14
D+1      1,000.00  440,304.72               0.00      13,500,473.05           0.00                 0.00    13,941,777.77            0.00   13,941,777.77
D+5      1,000.00  440,304.

C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\2393327332.py:142: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_relatorio.applymap('{:,.2f}'.format)


In [33]:
import pandas as pd
import json
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS PARA AS FONTES DE DADOS FINAIS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\estoque_consolidado_agosto"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÕES DE CARREGAMENTO FINAIS E CORRIGIDAS ---

def encontrar_e_carregar_carteira_recente(caminho_pasta):
    """Encontra o arquivo de carteira mais recente pela data interna e carrega seus dados."""
    # ... (código da função sem alterações) ...
    arquivos_json = glob.glob(os.path.join(caminho_pasta, "*.json"))
    if not arquivos_json: return None, None
    arquivo_mais_recente, data_mais_recente = None, None
    for arquivo in arquivos_json:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)
                data_atual_str = dados[0]['carteiras'][0]['dataAtual']
                data_atual_obj = datetime.strptime(data_atual_str.split('T')[0], '%Y-%m-%d')
                if data_mais_recente is None or data_atual_obj > data_mais_recente:
                    data_mais_recente, arquivo_mais_recente = data_atual_obj, arquivo
        except Exception: continue
    if arquivo_mais_recente:
        with open(arquivo_mais_recente, 'r', encoding='utf-8') as f:
            return json.load(f)[0], data_mais_recente, os.path.basename(arquivo_mais_recente)
    return None, None, None

def processar_dados_estoque(caminho_pasta):
    # ... (código da função sem alterações) ...
    arquivos_csv = glob.glob(os.path.join(caminho_pasta, "*.csv"))
    if not arquivos_csv: return pd.DataFrame()
    all_dfs = []
    for file in arquivos_csv:
        try:
            df = pd.read_csv(file, encoding='utf-16', sep='\t', engine='python', on_bad_lines='warn', header=0)
            df.columns = df.columns.str.strip()
            if not df.empty:
                df['Valor Presente'] = pd.to_numeric(df['Valor Presente'].astype(str).str.replace(',', '.'), errors='coerce')
                df['Data Vencimento'] = pd.to_datetime(df['Data Vencimento'], errors='coerce', format='%Y-%m-%d', dayfirst=False)
                all_dfs.append(df)
        except Exception: continue
    if not all_dfs: return pd.DataFrame()
    return pd.concat(all_dfs, ignore_index=True)

def processar_dados_despesas_final(caminho_pasta, data_base_projecao, meses_a_projetar=12):
    """Encontra o relatório de despesas mais recente e projeta suas provisões para o futuro."""
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame(), None
    df_mais_recente, data_relatorio_recente = None, None
    for arquivo in arquivos_excel:
        try:
            df_temp = pd.read_excel(arquivo, header=6)
            df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce', dayfirst=True)
            data_atual = df_temp['Data'].max()
            if data_relatorio_recente is None or data_atual > data_relatorio_recente:
                data_relatorio_recente, df_mais_recente = data_atual, df_temp
        except Exception: continue
    if df_mais_recente is None: return pd.DataFrame(), None
    
    df_base = df_mais_recente.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'})
    df_base['Valor'] = -pd.to_numeric(df_base['Valor'], errors='coerce').abs()
    df_base['Data Pagamento'] = pd.to_datetime(df_base['Data Pagamento'], errors='coerce', dayfirst=True)
    df_base = df_base[['Categoria', 'Valor', 'Data Pagamento']].dropna()
    despesas_finais = []
    despesas_reais_futuras = df_base[df_base['Data Pagamento'] >= data_base_projecao]
    despesas_finais.extend(despesas_reais_futuras.to_dict('records'))
    mes_inicial_projecao = (data_base_projecao.replace(day=1) + relativedelta(months=1))
    for i in range(meses_a_projetar):
        mes_atual = mes_inicial_projecao + relativedelta(months=i)
        data_pagamento_proj = mes_atual.replace(day=5)
        while data_pagamento_proj.weekday() >= 5: data_pagamento_proj += timedelta(days=1)
        if data_pagamento_proj >= data_base_projecao:
            for _, row in df_base.iterrows():
                despesas_finais.append({'Data Pagamento': data_pagamento_proj, 'Categoria': row['Categoria'], 'Valor': row['Valor']})
    df_final = pd.DataFrame(despesas_finais)
    return df_final, data_relatorio_recente

# --- DEMAIS FUNÇÕES FINAIS ---
def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    # ... (código da função sem alterações) ...
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    disponibilidade = next((a.get('Disponibilidade') for a in dados_carteira_recente['ativos'] if a.get('Disponibilidade')), None)
    if disponibilidade: fluxos.append({'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo']})
    cotas = next((a.get('Cotas') for a in dados_carteira_recente['ativos'] if a.get('Cotas')), None)
    if cotas:
        for cota in cotas['ativos']: fluxos.append({'data': data_base + timedelta(days=1), 'categoria': 'Fundo inv.', 'valor': cota['valorBruto']})
    renda_fixa = next((a.get('RendaFixa') for a in dados_carteira_recente['ativos'] if a.get('RendaFixa')), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']: fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Títulos Renda Fixa', 'valor': titulo['mercadoAtual']})
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Presente', 'Status'])
        status_a_incluir = ['A vencer', 'Previsto', 'Vencido']
        estoque_filtrado = estoque_valido[estoque_valido['Status'].isin(status_a_incluir)]
        for _, row in estoque_filtrado.iterrows(): fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Pagamentos Estoque', 'valor': row['Valor Presente']})
    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base]
        for _, despesa in despesas_futuras.iterrows(): fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas', 'valor': despesa['Valor']})
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            fluxos.append({'data': data_pagamento, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao})
    return pd.DataFrame(fluxos)

def gerar_relatorio_d_mais(df_fluxos, data_base):
    # ... (código da função sem alterações) ...
    prazos = [0, 1, 5, 10, 21, 63, 365]
    colunas_relatorio = ['Conta Corr.', 'Fundo inv.', 'Títulos Renda Fixa', 'Pagamentos Estoque', 'Despesas', 'Amort./ Resgat. Cota']
    linhas_relatorio = []
    for dias in prazos:
        data_limite = data_base + timedelta(days=dias)
        fluxos_ate_data = df_fluxos[df_fluxos['data'] <= data_limite]
        saldos_acumulados = fluxos_ate_data.groupby('categoria')['valor'].sum()
        linha = {'Prazo': f"D+{dias}"}
        for col in colunas_relatorio: linha[col] = saldos_acumulados.get(col, 0.0)
        linhas_relatorio.append(linha)
    df_relatorio = pd.DataFrame(linhas_relatorio).set_index('Prazo')
    df_relatorio['Disponibilidades'] = df_relatorio[colunas_relatorio[:4]].sum(axis=1)
    df_relatorio['Necessidades'] = df_relatorio[colunas_relatorio[4:]].sum(axis=1)
    df_relatorio['Caixa Projetado'] = df_relatorio['Disponibilidades'] + df_relatorio['Necessidades']
    return df_relatorio.applymap('{:,.2f}'.format)

# --- BLOCO DE EXECUÇÃO FINAL ---
if __name__ == "__main__":
    
    # 1. Carrega todas as fontes de dados
    dados_carteira_recente, data_referencia, nome_arquivo_carteira = encontrar_e_carregar_carteira_recente(PATH_CARTEIRAS_HISTORICO)
    
    if dados_carteira_recente:
        df_estoque = processar_dados_estoque(PATH_ESTOQUE)
        df_despesas, data_relatorio_despesas = processar_dados_despesas_final(PATH_DESPESAS, data_referencia)
        
        # 2. **NOVO** Bloco de Sumário de Datas
        print("\n" + "="*50)
        print("--- SUMÁRIO DAS FONTES DE DADOS UTILIZADAS ---")
        print("="*50)
        print(f"1. PONTO DE PARTIDA (D+0):")
        print(f"   - Arquivo de Carteira: '{nome_arquivo_carteira}'")
        print(f"   - Data de Referência..: {data_referencia.strftime('%d/%m/%Y')}")
        
        print(f"\n2. ENTRADAS DE CAIXA:")
        if not df_estoque.empty:
            print(f"   - Arquivo de Estoque..: {df_estoque.shape[0]} parcelas encontradas.")
            data_min_estoque = df_estoque['Data Vencimento'].dropna().min()
            data_max_estoque = df_estoque['Data Vencimento'].dropna().max()
            print(f"   - Período Coberto.....: de {data_min_estoque.strftime('%d/%m/%Y')} a {data_max_estoque.strftime('%d/%m/%Y')}")
        
        print(f"\n3. SAÍDAS DE CAIXA:")
        if data_relatorio_despesas:
            print(f"   - Arquivo de Despesas.: Usando o relatório gerado em {data_relatorio_despesas.strftime('%d/%m/%Y')} como base.")
            print(f"   - Lógica Aplicada.....: Despesas provisionadas neste arquivo foram projetadas para pagamentos futuros (5º dia útil de cada mês).")
        print("="*50)
        
        # 3. Prepara e gera o relatório final
        df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)
        relatorio_final = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia)

        print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (LÓGICA FINAL) ---")
        print(relatorio_final.to_string())


--- SUMÁRIO DAS FONTES DE DADOS UTILIZADAS ---
1. PONTO DE PARTIDA (D+0):
   - Arquivo de Carteira: 'carteira_52203615000119_20250829.json'
   - Data de Referência..: 29/08/2025

2. ENTRADAS DE CAIXA:
   - Arquivo de Estoque..: 1644304 parcelas encontradas.
   - Período Coberto.....: de 02/03/2024 a 26/10/2033

3. SAÍDAS DE CAIXA:
   - Arquivo de Despesas.: Usando o relatório gerado em 01/09/2025 como base.
   - Lógica Aplicada.....: Despesas provisionadas neste arquivo foram projetadas para pagamentos futuros (5º dia útil de cada mês).

--- PROJEÇÃO DE FLUXO DE CAIXA (LÓGICA FINAL) ---
      Conta Corr.  Fundo inv. Títulos Renda Fixa Pagamentos Estoque       Despesas Amort./ Resgat. Cota Disponibilidades    Necessidades Caixa Projetado
Prazo                                                                                                                                                   
D+0      1,000.00        0.00               0.00      13,448,387.14           0.00                 

C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\3910402492.py:135: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_relatorio.applymap('{:,.2f}'.format)


In [40]:
import pandas as pd
import re
import json
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import numpy as np

# --- CAMINHOS PARA AS FONTES DE DADOS FINAIS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Estoques"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"


# --- FUNÇÕES DE CARREGAMENTO FINAIS E CORRIGIDAS ---

def encontrar_e_carregar_carteira_recente(caminho_pasta):
    """Encontra o arquivo de carteira mais recente pela data interna e carrega seus dados."""
    # ... (código da função sem alterações) ...
    arquivos_json = glob.glob(os.path.join(caminho_pasta, "*.json"))
    if not arquivos_json: return None, None
    arquivo_mais_recente, data_mais_recente = None, None
    for arquivo in arquivos_json:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)
                data_atual_str = dados[0]['carteiras'][0]['dataAtual']
                data_atual_obj = datetime.strptime(data_atual_str.split('T')[0], '%Y-%m-%d')
                if data_mais_recente is None or data_atual_obj > data_mais_recente:
                    data_mais_recente, arquivo_mais_recente = data_atual_obj, arquivo
        except Exception: continue
    if arquivo_mais_recente:
        with open(arquivo_mais_recente, 'r', encoding='utf-8') as f:
            return json.load(f)[0], data_mais_recente, os.path.basename(arquivo_mais_recente)
    return None, None, None




def processar_dados_estoque(caminho_pasta):
    """Lê o conjunto de arquivos de estoque mais recente da pasta e o prepara para a análise."""
    print("\nCarregando dados do novo estoque...")
    todos_arquivos = glob.glob(os.path.join(caminho_pasta, '**/*.csv'), recursive=True)
    if not todos_arquivos: return pd.DataFrame()
    data_recente = None
    regex_data = re.compile(r'(\d{2}\.\d{2}\.\d{2})')
    for arquivo in todos_arquivos:
        match = regex_data.search(os.path.basename(arquivo))
        if match:
            data_obj = datetime.strptime(match.group(1), '%d.%m.%y')
            if data_recente is None or data_obj > data_recente: data_recente = data_obj
    if data_recente is None: return pd.DataFrame()
    data_recente_str = data_recente.strftime('%d.%m.%y')
    print(f"Usando o conjunto de estoque da data: {data_recente_str}")
    arquivos_para_ler = [f for f in todos_arquivos if data_recente_str in os.path.basename(f)]
    lista_dfs_partes = []
    for arquivo in arquivos_para_ler:
        try:
            df_parte = pd.read_csv(arquivo, sep=';', decimal=',', encoding='utf-8', on_bad_lines='warn')
            lista_dfs_partes.append(df_parte)
        except Exception: continue
    if not lista_dfs_partes: return pd.DataFrame()
    df_consolidado = pd.concat(lista_dfs_partes, ignore_index=True)
    df_consolidado.rename(columns={'DataVencimento': 'Data Vencimento', 'ValorPresente': 'Valor Presente'}, inplace=True)
    df_consolidado['Data Vencimento'] = pd.to_datetime(df_consolidado['Data Vencimento'], dayfirst=True, errors='coerce')
    df_consolidado['Valor Presente'] = pd.to_numeric(df_consolidado['Valor Presente'], errors='coerce')
    print(f"Dados do novo estoque consolidados: {len(df_consolidado)} linhas.")
    return df_consolidado

def processar_dados_despesas_final(caminho_pasta, data_base_projecao, meses_a_projetar=12):
    """Encontra o relatório de despesas mais recente e projeta suas provisões para o futuro."""
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame(), None
    df_mais_recente, data_relatorio_recente = None, None
    for arquivo in arquivos_excel:
        try:
            df_temp = pd.read_excel(arquivo, header=6)
            df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce', dayfirst=True)
            data_atual = df_temp['Data'].max()
            if data_relatorio_recente is None or data_atual > data_relatorio_recente:
                data_relatorio_recente, df_mais_recente = data_atual, df_temp
        except Exception: continue
    if df_mais_recente is None: return pd.DataFrame(), None
    
    df_base = df_mais_recente.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'})
    df_base['Valor'] = -pd.to_numeric(df_base['Valor'], errors='coerce').abs()
    df_base['Data Pagamento'] = pd.to_datetime(df_base['Data Pagamento'], errors='coerce', dayfirst=True)
    df_base = df_base[['Categoria', 'Valor', 'Data Pagamento']].dropna()
    despesas_finais = []
    despesas_reais_futuras = df_base[df_base['Data Pagamento'] >= data_base_projecao]
    despesas_finais.extend(despesas_reais_futuras.to_dict('records'))
    mes_inicial_projecao = (data_base_projecao.replace(day=1) + relativedelta(months=1))
    for i in range(meses_a_projetar):
        mes_atual = mes_inicial_projecao + relativedelta(months=i)
        data_pagamento_proj = mes_atual.replace(day=5)
        while data_pagamento_proj.weekday() >= 5: data_pagamento_proj += timedelta(days=1)
        if data_pagamento_proj >= data_base_projecao:
            for _, row in df_base.iterrows():
                despesas_finais.append({'Data Pagamento': data_pagamento_proj, 'Categoria': row['Categoria'], 'Valor': row['Valor']})
    df_final = pd.DataFrame(despesas_finais)
    return df_final, data_relatorio_recente

# --- DEMAIS FUNÇÕES FINAIS ---
def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # Disponibilidades
    disponibilidade = next((a.get('Disponibilidade') for a in dados_carteira_recente['ativos'] if a.get('Disponibilidade')), None)
    if disponibilidade: fluxos.append({'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo']})

    cotas = next((a.get('Cotas') for a in dados_carteira_recente['ativos'] if a.get('Cotas')), None)
    if cotas:
        for cota in cotas['ativos']: fluxos.append({'data': data_base + timedelta(days=1), 'categoria': 'Fundo inv.', 'valor': cota['valorBruto']})

    renda_fixa = next((a.get('RendaFixa') for a in dados_carteira_recente['ativos'] if a.get('RendaFixa')), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']: fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Títulos Renda Fixa', 'valor': titulo['mercadoAtual']})

    # Pagamentos do Estoque (COM FILTRO APENAS POR DATA)
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Presente'])
        
        # ### A LÓGICA CORRIGIDA E SIMPLIFICADA ESTÁ AQUI ###
        estoque_futuro = estoque_valido[estoque_valido['Data Vencimento'] >= data_base]
        
        print(f"\n{len(estoque_futuro)} parcelas do estoque com vencimento futuro foram incluídas na projeção.")
        
        for _, row in estoque_futuro.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Pagamentos Estoque', 'valor': row['Valor Presente']})

    # Necessidades
    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base]
        for _, despesa in despesas_futuras.iterrows(): fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas', 'valor': despesa['Valor']})

    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            fluxos.append({'data': data_pagamento, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao})
            
    return pd.DataFrame(fluxos)

def gerar_relatorio_d_mais(df_fluxos, data_base):
    # ... (código da função sem alterações) ...
    prazos = [0, 1, 5, 10, 21, 63, 365]
    colunas_relatorio = ['Conta Corr.', 'Fundo inv.', 'Títulos Renda Fixa', 'Pagamentos Estoque', 'Despesas', 'Amort./ Resgat. Cota']
    linhas_relatorio = []
    for dias in prazos:
        data_limite = data_base + timedelta(days=dias)
        fluxos_ate_data = df_fluxos[df_fluxos['data'] <= data_limite]
        saldos_acumulados = fluxos_ate_data.groupby('categoria')['valor'].sum()
        linha = {'Prazo': f"D+{dias}"}
        for col in colunas_relatorio: linha[col] = saldos_acumulados.get(col, 0.0)
        linhas_relatorio.append(linha)
    df_relatorio = pd.DataFrame(linhas_relatorio).set_index('Prazo')
    df_relatorio['Disponibilidades'] = df_relatorio[colunas_relatorio[:4]].sum(axis=1)
    df_relatorio['Necessidades'] = df_relatorio[colunas_relatorio[4:]].sum(axis=1)
    df_relatorio['Caixa Projetado'] = df_relatorio['Disponibilidades'] + df_relatorio['Necessidades']
    return df_relatorio.applymap('{:,.2f}'.format)

# --- BLOCO DE EXECUÇÃO FINAL ---
if __name__ == "__main__":
    
    # 1. Carrega todas as fontes de dados
    dados_carteira_recente, data_referencia, nome_arquivo_carteira = encontrar_e_carregar_carteira_recente(PATH_CARTEIRAS_HISTORICO)
    
    if dados_carteira_recente:
        df_estoque = processar_dados_estoque(PATH_ESTOQUE)
        df_despesas, data_relatorio_despesas = processar_dados_despesas_final(PATH_DESPESAS, data_referencia)
        
        # 2. **NOVO** Bloco de Sumário de Datas
        print("\n" + "="*50)
        print("--- SUMÁRIO DAS FONTES DE DADOS UTILIZADAS ---")
        print("="*50)
        print(f"1. PONTO DE PARTIDA (D+0):")
        print(f"   - Arquivo de Carteira: '{nome_arquivo_carteira}'")
        print(f"   - Data de Referência..: {data_referencia.strftime('%d/%m/%Y')}")
        
        print(f"\n2. ENTRADAS DE CAIXA:")
        if not df_estoque.empty:
            print(f"   - Arquivo de Estoque..: {df_estoque.shape[0]} parcelas encontradas.")
            data_min_estoque = df_estoque['Data Vencimento'].dropna().min()
            data_max_estoque = df_estoque['Data Vencimento'].dropna().max()
            print(f"   - Período Coberto.....: de {data_min_estoque.strftime('%d/%m/%Y')} a {data_max_estoque.strftime('%d/%m/%Y')}")
        
        print(f"\n3. SAÍDAS DE CAIXA:")
        if data_relatorio_despesas:
            print(f"   - Arquivo de Despesas.: Usando o relatório gerado em {data_relatorio_despesas.strftime('%d/%m/%Y')} como base.")
            print(f"   - Lógica Aplicada.....: Despesas provisionadas neste arquivo foram projetadas para pagamentos futuros (5º dia útil de cada mês).")
        print("="*50)
        
        # 3. Prepara e gera o relatório final
        df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)
        relatorio_final = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia)

        print("\n--- PROJEÇÃO DE FLUXO DE CAIXA (LÓGICA FINAL) ---")
        print(relatorio_final.to_string())


Carregando dados do novo estoque...
Usando o conjunto de estoque da data: 29.08.25
Dados do novo estoque consolidados: 1644304 linhas.

--- SUMÁRIO DAS FONTES DE DADOS UTILIZADAS ---
1. PONTO DE PARTIDA (D+0):
   - Arquivo de Carteira: 'carteira_52203615000119_20250829.json'
   - Data de Referência..: 29/08/2025

2. ENTRADAS DE CAIXA:
   - Arquivo de Estoque..: 1644304 parcelas encontradas.
   - Período Coberto.....: de 02/03/2024 a 26/10/2033

3. SAÍDAS DE CAIXA:
   - Arquivo de Despesas.: Usando o relatório gerado em 01/09/2025 como base.
   - Lógica Aplicada.....: Despesas provisionadas neste arquivo foram projetadas para pagamentos futuros (5º dia útil de cada mês).

1589899 parcelas do estoque com vencimento futuro foram incluídas na projeção.

--- PROJEÇÃO DE FLUXO DE CAIXA (LÓGICA FINAL) ---
      Conta Corr.  Fundo inv. Títulos Renda Fixa Pagamentos Estoque       Despesas Amort./ Resgat. Cota Disponibilidades    Necessidades Caixa Projetado
Prazo                               

C:\Users\Leo\AppData\Local\Temp\ipykernel_16300\514727119.py:166: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return df_relatorio.applymap('{:,.2f}'.format)


In [41]:
import pandas as pd
import base64
from datetime import datetime

# --- CONFIGURAÇÕES ---

# !! IMPORTANTE !!
# 1. Coloque o caminho para o arquivo da sua logo aqui.
#    Se o logo estiver na mesma pasta do script, basta o nome do arquivo (ex: "logo.png")
CAMINHO_DO_LOGO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\images\logo_inv.png"

# 2. Informações para o cabeçalho do relatório
NOME_DO_FUNDO = "FIDC FCT CONSIGNADO II"
NOME_ARQUIVO_SAIDA = "relatorio_fluxo_de_caixa.html"

# (Assumindo que o DataFrame 'relatorio_final' e 'data_referencia' já existem no seu notebook)
# Se não existirem, descomente e ajuste as linhas abaixo para teste:
# data_referencia = datetime(2025, 8, 29)
# relatorio_final = pd.DataFrame({
#     'Conta Corr.': ['1,000.00', '1,000.00'], 'Fundo inv.': ['0.00', '440,304.72'],
#     'Títulos Renda Fixa': ['0.00', '0.00'], 'Pagamentos Estoque': ['42,252.11', '94,338.02'],
#     'Despesas': ['0.00', '0.00'], 'Amort./ Resgat. Cota': ['0.00', '0.00'],
#     'Disponibilidades': ['43,252.11', '535,642.74'], 'Necessidades': ['0.00', '0.00'],
#     'Caixa Projetado': ['43,252.11', '535,642.74']
# }, index=pd.Index(['D+0', 'D+1'], name='Prazo'))


# --- FUNÇÃO PARA CODIFICAR A LOGO ---

def codificar_imagem_base64(caminho_imagem):
    """Lê um arquivo de imagem e o converte para o formato base64."""
    try:
        with open(caminho_imagem, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"AVISO: Arquivo de logo não encontrado em '{caminho_imagem}'. Um placeholder será usado.")
        # Retorna um pixel transparente como placeholder
        return "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mNkYAAAAAYAAjCB0C8AAAAASUVORK5CYII="


# --- GERAÇÃO DO HTML ---

def gerar_relatorio_html(df, nome_fundo, data_ref, caminho_logo, arquivo_saida):
    """Gera o arquivo HTML completo com base no DataFrame e nas especificações de estilo."""
    
    logo_base64 = codificar_imagem_base64(caminho_logo)
    
    # Converte o DataFrame para uma tabela HTML, adicionando classes para estilização
    tabela_html = df.to_html(classes="display compact", table_id="relatorioTabela", border=0)

    # Template HTML completo com CSS e JavaScript
    html_template = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Fluxo de Caixa</title>
        
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css">
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/fixedcolumns/4.3.0/css/fixedColumns.dataTables.min.css">

        <style>
            body {{
                font-family: "Gill Sans MT", Arial, sans-serif;
                background-color: #FFFFFF;
                color: #313131;
                font-size: 1.1em;
                padding: 20px;
            }}
            h1 {{
                font-size: 2.2em;
                color: #163f3f;
                margin: 0;
            }}
            h2 {{
                font-size: 1.5em;
                color: #163f3f;
                border-bottom: 2px solid #163f3f;
                padding-bottom: 5px;
            }}
            .header-container {{
                display: flex;
                align-items: center;
                margin-bottom: 20px;
            }}
            .info-box {{
                border: 1px solid #e0e0e0;
                background-color: #f5f5f5;
                padding: 15px;
                margin-bottom: 30px;
            }}
            
            /* Estilos da Tabela 📊 */
            table.dataTable thead th {{
                background-color: #163f3f !important;
                color: white !important;
                border-bottom: 2px solid #0e3030;
            }}
            table.dataTable.display tbody tr.odd {{
                background-color: #f5f5f5;
            }}
            table.dataTable.display tbody tr.even {{
                background-color: #e0e0e0;
            }}
            table.dataTable tbody tr:hover {{
                background-color: #FFD8A7 !important;
            }}
            
            /* Coluna Fixa */
            table.dataTable tbody th {{
                font-weight: bold;
                background-color: #0e3030;
                color: white;
            }}

            /* Corte de texto no cabeçalho */
            th.truncate {{
                white-space: nowrap;
                overflow: hidden;
                text-overflow: ellipsis;
                max-width: 150px; /* Ajuste conforme necessário */
            }}
        </style>
    </head>
    <body>

        <div class="header-container">
            <img src="data:image/png;base64,{logo_base64}" style="max-height:80px; margin-right:20px;">
            <h1>Relatório de Fluxo de Caixa</h1>
        </div>
        
        <div class="info-box">
            <strong>Fundo:</strong> {nome_fundo}<br>
            <strong>Data do Relatório:</strong> {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}<br>
            <strong>Data de Referência (D+0):</strong> {data_ref.strftime('%d/%m/%Y')}
        </div>

        <h2>Projeção de Caixa por Prazo</h2>
        
        {tabela_html}

        <script type="text/javascript" src="https://code.jquery.com/jquery-3.7.0.js"></script>
        <script type="text/javascript" src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
        <script type="text/javascript" src="https://cdn.datatables.net/fixedcolumns/4.3.0/js/dataTables.fixedColumns.min.js"></script>
        
        <script type="text/javascript">
            jQuery.fn.dataTable.ext.type.order['brazilian-number-pre'] = function ( a ) {{
                var x = a.replace( /[.\\sR$]/g, '' ).replace( ',', '.' );
                return parseFloat( x );
            }};
        </script>

        <script>
            $(document).ready(function() {{
                $('#relatorioTabela').DataTable({{
                    "scrollX": true,
                    "fixedColumns": {{
                        left: 1 // Fixa a primeira coluna (Prazo)
                    }},
                    "paging": false,
                    "searching": false,
                    "info": false,
                    "columnDefs": [
                        {{ "type": "brazilian-number", "targets": "_all" }}
                    ]
                }});

                // Adiciona tooltip aos cabeçalhos com texto cortado
                $('th.truncate').each(function(){{
                    $(this).attr('title', $(this).text());
                }});
            }});
        </script>

    </body>
    </html>
    """

    # Salva o conteúdo em um arquivo .html
    try:
        with open(arquivo_saida, 'w', encoding='utf-8') as f:
            f.write(html_template)
        print(f"✅ Relatório HTML gerado com sucesso: '{os.path.abspath(arquivo_saida)}'")
    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo HTML: {e}")

# --- BLOCO DE EXECUÇÃO ---
if __name__ == "__main__":
    gerar_relatorio_html(
        df=relatorio_final, 
        nome_fundo=NOME_DO_FUNDO, 
        data_ref=data_referencia, 
        caminho_logo=CAMINHO_DO_LOGO,
        arquivo_saida=NOME_ARQUIVO_SAIDA
    )

✅ Relatório HTML gerado com sucesso: 'c:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\relatorio_fluxo_de_caixa.html'


In [42]:
# --- FUNÇÃO PARA CODIFICAR A LOGO ---
def codificar_imagem_base64(caminho_imagem):
    try:
        with open(caminho_imagem, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        print(f"AVISO: Arquivo de logo não encontrado. Um placeholder será usado.")
        return "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mNkYAAAAAYAAjCB0C8AAAAASUVORK5CYII="

# --- NOVA GERAÇÃO DO HTML ---

def gerar_relatorio_html_interativo(df_summary, df_details, nome_fundo, data_ref, caminho_logo, arquivo_saida):
    """Gera o arquivo HTML interativo com funcionalidade de drill-down."""
    
    logo_base64 = codificar_imagem_base64(caminho_logo)
    
    # Prepara os dados detalhados para serem embutidos como JSON no HTML
    df_details['data'] = df_details['data'].dt.strftime('%d/%m/%Y')
    dados_detalhados_json = df_details.to_json(orient='records')

    # ---- Construção manual da tabela principal para adicionar atributos de dados ----
    tabela_html = '<table id="relatorioTabela" class="display compact"><thead><tr><th>Prazo</th>'
    for col in df_summary.columns:
        tabela_html += f'<th>{col}</th>'
    tabela_html += '</tr></thead><tbody>'
    
    for prazo, row in df_summary.iterrows():
        tabela_html += f'<tr><th>{prazo}</th>'
        for col, value in row.items():
            # Adiciona os atributos data-* nas células para identificar o que foi clicado
            tabela_html += f'<td data-prazo="{prazo}" data-categoria="{col}">{value}</td>'
        tabela_html += '</tr>'
        
    tabela_html += '</tbody></table>'

    # Template HTML completo com a nova estrutura do modal e JavaScript
    html_template = f"""
    <!DOCTYPE html>
    <html lang="pt-BR">
    <head>
        <meta charset="UTF-8">
        <title>Relatório de Fluxo de Caixa Interativo</title>
        
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css">
        <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/fixedcolumns/4.3.0/css/fixedColumns.dataTables.min.css">

        <style>
            /* (Estilos CSS da versão anterior, omitidos para brevidade) */
            body {{ font-family: "Gill Sans MT", Arial, sans-serif; background-color: #FFFFFF; color: #313131; font-size: 1.1em; padding: 20px; }}
            h1 {{ font-size: 2.2em; color: #163f3f; margin: 0; }}
            .header-container {{ display: flex; align-items: center; margin-bottom: 20px; }}
            .info-box {{ border: 1px solid #e0e0e0; background-color: #f5f5f5; padding: 15px; margin-bottom: 30px; }}
            table.dataTable thead th {{ background-color: #163f3f !important; color: white !important; }}
            table.dataTable.display tbody tr.odd {{ background-color: #f5f5f5; }}
            table.dataTable.display tbody tr.even {{ background-color: #e0e0e0; }}
            table.dataTable tbody tr:hover {{ background-color: #FFD8A7 !important; }}
            table.dataTable tbody th {{ font-weight: bold; background-color: #0e3030; color: white; }}

            /* Estilos para células clicáveis e modal */
            #relatorioTabela tbody td:not(:empty) {{ cursor: pointer; }}
            .modal {{ display: none; position: fixed; z-index: 1000; left: 0; top: 0; width: 100%; height: 100%; overflow: auto; background-color: rgba(0,0,0,0.5); }}
            .modal-content {{ background-color: #fefefe; margin: 5% auto; padding: 20px; border: 1px solid #888; width: 80%; max-width: 900px; }}
            .close-button {{ color: #aaa; float: right; font-size: 28px; font-weight: bold; cursor: pointer; }}
            #modal-table-container table {{ width: 100%; border-collapse: collapse; }}
            #modal-table-container th, #modal-table-container td {{ border: 1px solid #ddd; padding: 8px; }}
            #modal-table-container th {{ background-color: #f2f2f2; }}
        </style>
    </head>
    <body>

        <div class="header-container">
            <img src="data:image/png;base64,{logo_base64}" style="max-height:80px; margin-right:20px;">
            <h1>Relatório de Fluxo de Caixa Interativo</h1>
        </div>
        <div class="info-box">
            <strong>Fundo:</strong> {nome_fundo}<br>
            <strong>Data do Relatório:</strong> {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}<br>
            <strong>Data de Referência (D+0):</strong> {data_ref.strftime('%d/%m/%Y')}
        </div>
        
        {tabela_html}

        <div id="detailModal" class="modal">
            <div class="modal-content">
                <span class="close-button">&times;</span>
                <h2 id="modal-title">Detalhamento</h2>
                <div id="modal-table-container" style="max-height: 400px; overflow-y: auto;"></div>
            </div>
        </div>

        <script src="https://code.jquery.com/jquery-3.7.0.js"></script>
        <script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
        <script src="https://cdn.datatables.net/fixedcolumns/4.3.0/js/dataTables.fixedColumns.min.js"></script>

        <script>
            var dadosDetalhados = {dados_detalhados_json};
            var dataReferencia = new Date('{data_ref.strftime("%Y-%m-%d")}T00:00:00');
        </script>

        <script>
        $(document).ready(function() {{
            // Inicializa a tabela principal
            $('#relatorioTabela').DataTable({{ "scrollX": true, "fixedColumns": {{ left: 1 }}, "paging": false, "searching": false, "info": false, "ordering": false }});

            var modal = $('#detailModal');
            
            // Lógica para ABRIR o modal
            $('#relatorioTabela tbody').on('click', 'td', function() {{
                var cell = $(this);
                var prazo = cell.data('prazo');
                var categoria = cell.data('categoria');
                
                // Ignora cliques em colunas de resumo ou células vazias
                if (!prazo || !categoria || ['Disponibilidades', 'Necessidades', 'Caixa Projetado'].includes(categoria) || cell.text() === '0.00') {{
                    return;
                }}

                var dias = parseInt(prazo.replace('D+', ''));
                var dataLimite = new Date(dataReferencia);
                dataLimite.setDate(dataLimite.getDate() + dias);

                // Filtra os dados detalhados
                var itensFiltrados = dadosDetalhados.filter(function(item) {{
                    var dataItem = new Date(item.data.split('/').reverse().join('-'));
                    return item.categoria === categoria && dataItem <= dataLimite;
                }});

                // Cria a tabela de detalhes
                var tableHtml = '<table><thead><tr><th>Data</th><th>Valor</th></tr></thead><tbody>';
                var total = 0;
                itensFiltrados.forEach(function(item) {{
                    total += item.valor;
                    tableHtml += '<tr><td>' + item.data + '</td><td>' + item.valor.toLocaleString('pt-BR', {{style: 'currency', currency: 'BRL'}}) + '</td></tr>';
                }});
                tableHtml += '</tbody><tfoot><tr><th>Total</th><th>' + total.toLocaleString('pt-BR', {{style: 'currency', currency: 'BRL'}}) + '</th></tr></tfoot></table>';

                // Preenche e exibe o modal
                $('#modal-title').text('Detalhamento de: ' + categoria + ' (até ' + prazo + ')');
                $('#modal-table-container').html(tableHtml);
                modal.show();
            }});

            // Lógica para FECHAR o modal
            $('.close-button').on('click', function() {{
                modal.hide();
            }});
            $(window).on('click', function(event) {{
                if (event.target == modal[0]) {{
                    modal.hide();
                }}
            }});
        }});
        </script>

    </body>
    </html>
    """

    try:
        with open(arquivo_saida, 'w', encoding='utf-8') as f:
            f.write(html_template)
        print(f"✅ Relatório HTML interativo gerado com sucesso: '{os.path.abspath(arquivo_saida)}'")
    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo HTML: {e}")

# --- BLOCO DE EXECUÇÃO ---
# (Certifique-se de que os DataFrames 'relatorio_final' e 'df_fluxos_detalhados'
#  e a variável 'data_referencia' existam no seu ambiente Jupyter)
if 'relatorio_final' in locals() and 'df_fluxos_detalhados' in locals():
    gerar_relatorio_html_interativo(
        df_summary=relatorio_final, 
        df_details=df_fluxos_detalhados,
        nome_fundo=NOME_DO_FUNDO, 
        data_ref=data_referencia, 
        caminho_logo=CAMINHO_DO_LOGO,
        arquivo_saida=NOME_ARQUIVO_SAIDA
    )
else:
    print("AVISO: DataFrame 'relatorio_final' ou 'df_fluxos_detalhados' não encontrado. Execute o script principal primeiro.")


✅ Relatório HTML interativo gerado com sucesso: 'c:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\relatorio_fluxo_de_caixa.html'


In [ ]:

# depuracao apenas 
import pandas as pd
import os
import glob
from datetime import datetime
import re

# --- CAMINHO PARA A PASTA COM OS NOVOS ARQUIVOS DE ESTOQUE ---
PATH_ESTOQUE_NOVO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Estoques"

# --- BLOCO DE DEPURAÇÃO FOCADO NO ESTOQUE ---

print("--- DEPURAÇÃO: Inspecionando o carregamento do Novo Estoque ---")

todos_arquivos = glob.glob(os.path.join(PATH_ESTOQUE_NOVO, '**/*.csv'), recursive=True)
if not todos_arquivos:
    print("ERRO: Nenhum arquivo de estoque (.csv) encontrado.")
else:
    # 1. Encontra a data mais recente nos nomes dos arquivos
    data_recente = None
    regex_data = re.compile(r'(\d{2}\.\d{2}\.\d{2})')
    for arquivo in todos_arquivos:
        match = regex_data.search(os.path.basename(arquivo))
        if match:
            data_str = match.group(1)
            data_obj = datetime.strptime(data_str, '%d.%m.%y')
            if data_recente is None or data_obj > data_recente:
                data_recente = data_obj

    if data_recente is None:
        print("ERRO: Não foi possível determinar a data mais recente dos arquivos.")
    else:
        data_recente_str = data_recente.strftime('%d.%m.%y')
        print(f"1. Usando o conjunto de estoque da data: {data_recente_str}")
        arquivos_para_ler = [f for f in todos_arquivos if data_recente_str in os.path.basename(f)]
        
        # 2. Lê e concatena as partes
        lista_dfs_partes = []
        for arquivo in arquivos_para_ler:
            try:
                df_parte = pd.read_csv(arquivo, sep=';', decimal=',', encoding='utf-8', on_bad_lines='warn')
                lista_dfs_partes.append(df_parte)
            except Exception as e:
                print(f"AVISO ao ler '{os.path.basename(arquivo)}': {e}")
        
        if lista_dfs_partes:
            df_consolidado = pd.concat(lista_dfs_partes, ignore_index=True)
            print(f"\n2. Arquivos lidos e consolidados. Total de {len(df_consolidado)} linhas.")
            
            print("\n3. Amostra dos dados brutos (primeiras 5 linhas):")
            print(df_consolidado.head())
            
            print("\n4. Nomes das colunas encontrados no arquivo:")
            print(df_consolidado.columns.tolist())

            # 5. Inspeciona a coluna 'Situacao'
            if 'Situacao' in df_consolidado.columns:
                print("\n5. Análise da coluna 'Situacao' (que será o nosso 'Status'):")
                print("Valores únicos encontrados:")
                print(df_consolidado['Situacao'].unique())
            else:
                print("\n5. ALERTA: A coluna 'Situacao' não foi encontrada!")

            # 6. Simula o filtro do script principal
            print("\n6. Simulando o filtro que o script principal aplica...")
            
            # Renomeia Situacao -> Status para o teste
            df_teste = df_consolidado.rename(columns={'Situacao': 'Status'})
            
            status_a_incluir = ['A vencer', 'Previsto', 'Vencido']
            print(f"   - O script procura por status em: {status_a_incluir}")
            
            df_filtrado = df_teste[df_teste['Status'].isin(status_a_incluir)]
            
            print(f"   - Resultado: Após o filtro, restaram {len(df_filtrado)} linhas.")
            
            print("\n------------------------------------------------------------------")
            if len(df_filtrado) == 0:
                print(">>> DIAGNÓSTICO: O filtro de 'Status' está removendo todas as linhas.")
                print("Os valores na coluna 'Situacao' do novo arquivo não correspondem a 'A vencer', 'Previsto' ou 'Vencido'.")
                print("A solução é ajustar a lista 'status_a_incluir' no script principal para usar os valores corretos (ex: ['Sem cobrança']).")
            else:
                 print(">>> DIAGNÓSTICO: O filtro de 'Status' está funcionando. O problema pode estar na conversão das datas ou valores.")

--- DEPURAÇÃO: Inspecionando o carregamento do Novo Estoque ---
1. Usando o conjunto de estoque da data: 29.08.25

2. Arquivos lidos e consolidados. Total de 1644304 linhas.

3. Amostra dos dados brutos (primeiras 5 linhas):
       Situacao PES_TIPO_PESSOA      CedenteCnpjCpf  TIT_CEDENTE_ENT_CODIGO  \
0  Sem cobrança               J  34.337.707/0001-00                  318853   
1  Sem cobrança               J  34.337.707/0001-00                  318853   
2  Sem cobrança               J  34.337.707/0001-00                  318853   
3  Sem cobrança               J  34.337.707/0001-00                  318853   
4  Sem cobrança               J  34.337.707/0001-00                  318853   

                                      CedenteNome     Cnae  \
0  BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A  6499999   
1  BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A  6499999   
2  BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A  6499999   
3  BMP MONEY PLUS SOCIEDADE DE CREDITO DIRETO S.A  6

In [28]:
display(relatorio_final)

,Conta Corr.,Fundo inv.,Títulos Renda Fixa,Pagamentos Estoque,Despesas,Amort./ Resgat. Cota,Disponibilidades,Necessidades,Caixa Projetado
Prazo,,,,,,,,,
D+0,"1,000.00",0.00,0.00,"13,448,387.14",0.00,0.00,"13,449,387.14",0.00,"13,449,387.14"
D+1,"1,000.00","440,304.72",0.00,"13,500,473.05",0.00,0.00,"13,941,777.77",0.00,"13,941,777.77"
D+5,"1,000.00","440,304.72",0.00,"13,728,677.28",0.00,0.00,"14,169,982.00",0.00,"14,169,982.00"
D+10,"1,000.00","440,304.72",0.00,"14,060,107.05",0.00,0.00,"14,501,411.77",0.00,"14,501,411.77"
D+21,"1,000.00","440,304.72",0.00,"16,929,321.16",0.00,0.00,"17,370,625.88",0.00,"17,370,625.88"
D+63,"1,000.00","440,304.72",0.00,"27,287,355.18",0.00,0.00,"27,728,659.90",0.00,"27,728,659.90"
D+365,"1,000.00","440,304.72",0.00,"87,195,087.53",0.00,"-24,845,090.89","87,636,392.25","-24,845,090.89","62,791,301.36"
D+730,"1,000.00","440,304.72",0.00,"135,929,948.76",0.00,"-62,112,727.22","136,371,253.48","-62,112,727.22","74,258,526.26"
D+1200,"1,000.00","440,304.72",0.00,"174,807,393.82",0.00,"-108,697,272.64","175,248,698.54","-108,697,272.64","66,551,425.90"


In [44]:

# --- FUNÇÃO DE PREPARAÇÃO DE FLUXOS (COM DESCRIÇÃO DETALHADA) ---
def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    """Cria uma lista de TODOS os eventos de fluxo de caixa, com descrição detalhada para cada um."""
    fluxos = []
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')

    # Disponibilidades
    disponibilidade = next((a.get('Disponibilidade') for a in dados_carteira_recente['ativos'] if a.get('Disponibilidade')), None)
    if disponibilidade:
        fluxos.append({'data': data_base, 'categoria': 'Conta Corr.', 'valor': disponibilidade['ativos'][0]['saldo'], 'descricao': 'Saldo Inicial'})

    cotas = next((a.get('Cotas') for a in dados_carteira_recente['ativos'] if a.get('Cotas')), None)
    if cotas:
        for cota in cotas['ativos']:
            fluxos.append({'data': data_base + timedelta(days=1), 'categoria': 'Fundo inv.', 'valor': cota['valorBruto'], 'descricao': f"Resgate {cota.get('titulo', 'Fundo')}"})
            
    renda_fixa = next((a.get('RendaFixa') for a in dados_carteira_recente['ativos'] if a.get('RendaFixa')), None)
    if renda_fixa:
        for titulo in renda_fixa['ativos']:
            fluxos.append({'data': datetime.strptime(titulo['dataVencimento'], '%Y-%m-%d'), 'categoria': 'Títulos Renda Fixa', 'valor': titulo['mercadoAtual'], 'descricao': f"Venc. {titulo.get('codigoCustodia', 'Título RF')}"})

    # Pagamentos do Estoque
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Presente'])
        estoque_futuro = estoque_valido[estoque_valido['Data Vencimento'] >= data_base]
        for _, row in estoque_futuro.iterrows():
            fluxos.append({'data': row['Data Vencimento'], 'categoria': 'Pagamentos Estoque', 'valor': row['Valor Presente'], 'descricao': f"Título Nº {row.get('NumeroTitulo', 'N/A')}"})

    # Necessidades
    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base]
        for _, despesa in despesas_futuras.iterrows():
            fluxos.append({'data': despesa['Data Pagamento'], 'categoria': 'Despesas', 'valor': despesa['Valor'], 'descricao': despesa['Categoria']})
            
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        data_amortizacao = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_amortizacao + relativedelta(months=i)
            while data_pagamento.weekday() >= 5: data_pagamento += pd.Timedelta(days=1)
            fluxos.append({'data': data_pagamento, 'categoria': 'Amort./ Resgat. Cota', 'valor': valor_amortizacao, 'descricao': f"Parcela {i+1}/36"})
            
    return pd.DataFrame(fluxos)


df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)


In [1]:
import pandas as pd
import json
import os
import glob
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import re
import base64

# --- CAMINHOS E CONFIGURAÇÕES FINAIS ---
PATH_CARTEIRAS_HISTORICO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Carteiras"
PATH_ESTOQUE_NOVO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Estoques"
PATH_DESPESAS = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\data\Despesas"
CAMINHO_DO_LOGO = r"C:\Users\Leo\Desktop\Porto_Real\portoauto\images\logo_inv.png" 
NOME_DO_FUNDO = "FIDC FCT CONSIGNADO II"
NOME_ARQUIVO_SAIDA = "relatorio_fluxo_caixa_final.html"

# --- FUNÇÕES DE CARREGAMENTO FINAIS ---

def encontrar_e_carregar_carteira_recente(caminho_pasta):
    """Encontra o arquivo de carteira mais recente pela data interna e carrega seus dados."""
    print("Procurando o arquivo de carteira mais recente...")
    arquivos_json = glob.glob(os.path.join(caminho_pasta, "*.json"))
    if not arquivos_json: return None, None, None
    arquivo_mais_recente, data_mais_recente = None, None
    for arquivo in arquivos_json:
        try:
            with open(arquivo, 'r', encoding='utf-8') as f:
                dados = json.load(f)
                data_atual_str = dados[0]['carteiras'][0]['dataAtual']
                data_atual_obj = datetime.strptime(data_atual_str.split('T')[0], '%Y-%m-%d')
                if data_mais_recente is None or data_atual_obj > data_mais_recente:
                    data_mais_recente, arquivo_mais_recente = data_atual_obj, arquivo
        except Exception: continue
    if arquivo_mais_recente:
        print(f"Usando a carteira mais recente para a projeção: {os.path.basename(arquivo_mais_recente)}")
        with open(arquivo_mais_recente, 'r', encoding='utf-8') as f:
            return json.load(f)[0], data_mais_recente, os.path.basename(arquivo_mais_recente)
    return None, None, None

def processar_dados_estoque_novo(caminho_pasta):
    """Lê o conjunto de arquivos de estoque mais recente da pasta e o prepara para a análise."""
    print("\nCarregando dados do novo estoque...")
    todos_arquivos = glob.glob(os.path.join(caminho_pasta, '**/*.csv'), recursive=True)
    if not todos_arquivos: return pd.DataFrame()
    data_recente = None
    regex_data = re.compile(r'(\d{2}\.\d{2}\.\d{2})')
    for arquivo in todos_arquivos:
        match = regex_data.search(os.path.basename(arquivo))
        if match:
            data_obj = datetime.strptime(match.group(1), '%d.%m.%y')
            if data_recente is None or data_obj > data_recente: data_recente = data_obj
    if data_recente is None: return pd.DataFrame()
    data_recente_str = data_recente.strftime('%d.%m.%y')
    print(f"Usando o conjunto de estoque da data: {data_recente_str}")
    arquivos_para_ler = [f for f in todos_arquivos if data_recente_str in os.path.basename(f)]
    lista_dfs_partes = []
    for arquivo in arquivos_para_ler:
        try:
            df_parte = pd.read_csv(arquivo, sep=';', decimal=',', encoding='utf-8', on_bad_lines='warn')
            lista_dfs_partes.append(df_parte)
        except Exception: continue
    if not lista_dfs_partes: return pd.DataFrame()
    df_consolidado = pd.concat(lista_dfs_partes, ignore_index=True)
    df_consolidado.rename(columns={
        'DataVencimento': 'Data Vencimento', 
        'ValorPresente': 'Valor Presente',
        'ValorNominal': 'Valor Nominal'  # Adiciona o Valor Nominal
    }, inplace=True)
    
    # Converte as colunas de data e valor
    df_consolidado['Data Vencimento'] = pd.to_datetime(df_consolidado['Data Vencimento'], dayfirst=True, errors='coerce')
    df_consolidado['Valor Presente'] = pd.to_numeric(df_consolidado['Valor Presente'], errors='coerce')
    
    # ### ALTERAÇÃO AQUI: Converte também o Valor Nominal ###
    df_consolidado['Valor Nominal'] = pd.to_numeric(df_consolidado['Valor Nominal'], errors='coerce')
    
    print(f"Dados do novo estoque consolidados: {len(df_consolidado)} linhas.")
    return df_consolidado

def processar_dados_despesas_final(caminho_pasta, data_base_projecao, meses_a_projetar=12):
    """Encontra o relatório de despesas mais recente e projeta suas provisões para o futuro."""
    print("\nCarregando despesas com a lógica final...")
    arquivos_excel = glob.glob(os.path.join(caminho_pasta, '**/Despesas_Consolidadas.xlsx'), recursive=True)
    if not arquivos_excel: return pd.DataFrame(), None
    df_mais_recente, data_relatorio_recente = None, None
    for arquivo in arquivos_excel:
        try:
            df_temp = pd.read_excel(arquivo, header=6)
            df_temp['Data'] = pd.to_datetime(df_temp['Data'], errors='coerce', dayfirst=True)
            data_atual = df_temp['Data'].max()
            if data_relatorio_recente is None or data_atual > data_relatorio_recente:
                data_relatorio_recente, df_mais_recente = data_atual, df_temp
        except Exception: continue
    if df_mais_recente is None: return pd.DataFrame(), None
    print(f"Usando o relatório de despesas gerado em: {data_relatorio_recente.strftime('%d/%m/%Y')}")
    df_base = df_mais_recente.rename(columns={'Título': 'Categoria', 'Valor Mercado': 'Valor'})
    df_base['Valor'] = -pd.to_numeric(df_base['Valor'], errors='coerce').abs()
    df_base['Data Pagamento'] = pd.to_datetime(df_base['Data Pagamento'], errors='coerce', dayfirst=True)
    df_base = df_base[['Categoria', 'Valor', 'Data Pagamento']].dropna()
    despesas_finais = []
    despesas_reais_futuras = df_base[df_base['Data Pagamento'] >= data_base_projecao]
    despesas_finais.extend(despesas_reais_futuras.to_dict('records'))
    mes_inicial_projecao = (data_base_projecao.replace(day=1) + relativedelta(months=1))
    for i in range(meses_a_projetar):
        mes_atual = mes_inicial_projecao + relativedelta(months=i)
        data_pagamento_proj = mes_atual.replace(day=5)
        while data_pagamento_proj.weekday() >= 5: data_pagamento_proj += timedelta(days=1)
        if data_pagamento_proj >= data_base_projecao:
            for _, row in df_base.iterrows():
                despesas_finais.append({'Data Pagamento': data_pagamento_proj, 'Categoria': row['Categoria'], 'Valor': row['Valor']})
    df_final = pd.DataFrame(despesas_finais)
    print(f"{len(df_final)} eventos de despesas futuras (reais + projetadas) foram criados.")
    return df_final, data_relatorio_recente

def preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas_datadas):
    """
    Cria uma lista de TODOS os eventos de fluxo de caixa, com descrição detalhada para cada um.
    VERSÃO VETORIZADA para maior eficiência.
    """
    data_base = datetime.strptime(dados_carteira_recente['carteiras'][0]['dataAtual'].split('T')[0], '%Y-%m-%d')
    
    # Lista para armazenar os DataFrames de cada categoria de fluxo
    lista_de_fluxos_df = []

    # --- Disponibilidades ---
    
    # 1. Conta Corrente e Fundo Investido (geralmente poucos itens, vetorização simples)
    disponibilidade = next((a.get('Disponibilidade') for a in dados_carteira_recente['ativos'] if a.get('Disponibilidade')), None)
    if disponibilidade:
        df_cc = pd.DataFrame([{
            'data': data_base, 'categoria': 'Conta Corr.', 
            'valor': disponibilidade['ativos'][0]['saldo'], 'descricao': 'Saldo Inicial'
        }])
        lista_de_fluxos_df.append(df_cc)

    cotas = next((a.get('Cotas') for a in dados_carteira_recente['ativos'] if a.get('Cotas')), None)
    if cotas:
        df_fundos = pd.DataFrame(cotas['ativos'])
        df_fundos['data'] = data_base # Alterado para D+0 conforme solicitado
        df_fundos['categoria'] = 'Fundo inv.'
        df_fundos['valor'] = df_fundos['valorBruto']
        df_fundos['descricao'] = "Resgate " + df_fundos['titulo']
        lista_de_fluxos_df.append(df_fundos[['data', 'categoria', 'valor', 'descricao']])

    # 2. Renda Fixa
    renda_fixa = next((a.get('RendaFixa') for a in dados_carteira_recente['ativos'] if a.get('RendaFixa')), None)
    if renda_fixa:
        df_rf = pd.DataFrame(renda_fixa['ativos'])
        df_rf['data'] = pd.to_datetime(df_rf['dataVencimento'])
        df_rf['categoria'] = 'Títulos Renda Fixa'
        df_rf['valor'] = df_rf['mercadoAtual']
        df_rf['descricao'] = "Venc. " + df_rf['codigoCustodia']
        lista_de_fluxos_df.append(df_rf[['data', 'categoria', 'valor', 'descricao']])

    # 3. Pagamentos do Estoque (onde a vetorização faz mais diferença)
    if not df_estoque.empty:
        estoque_valido = df_estoque.dropna(subset=['Data Vencimento', 'Valor Nominal'])
        estoque_futuro = estoque_valido[estoque_valido['Data Vencimento'] >= data_base].copy()
        
        print(f"\n{len(estoque_futuro)} parcelas do estoque com vencimento futuro foram incluídas na projeção.")
        
        # Operações vetorizadas: criar e modificar colunas inteiras de uma vez
        estoque_futuro['categoria'] = 'Pagamentos Estoque'
        estoque_futuro['descricao'] = "Título Nº " + estoque_futuro['NumeroTitulo'].astype(str)
        
        # Renomeia as colunas para o formato final, selecionando apenas o que precisamos
        df_estoque_final = estoque_futuro.rename(columns={'Data Vencimento': 'data', 'Valor Nominal': 'valor'})
        lista_de_fluxos_df.append(df_estoque_final[['data', 'categoria', 'valor', 'descricao']])

    # --- Necessidades ---

    # 4. Despesas
    if not df_despesas_datadas.empty:
        despesas_futuras = df_despesas_datadas[df_despesas_datadas['Data Pagamento'] >= data_base].copy()
        despesas_futuras['categoria'] = 'Despesas'
        despesas_futuras['descricao'] = despesas_futuras['Categoria'] # Usa a categoria original como descrição
        df_despesas_final = despesas_futuras.rename(columns={'Data Pagamento': 'data', 'Valor': 'valor'})
        lista_de_fluxos_df.append(df_despesas_final[['data', 'categoria', 'valor', 'descricao']])

    # 5. Amortização de Cotas
    carteira_senior = next((c for c in dados_carteira_recente['carteiras'] if 'SR2' in c['nome']), None)
    if carteira_senior:
        pl_senior = carteira_senior['pl']
        valor_amortizacao = (pl_senior / 36) * -1
        
        # Gera a lista de datas e descrições primeiro
        datas_amortizacao = []
        descricoes_amortizacao = []
        data_base_amort = datetime(2026, 1, 16)
        for i in range(36):
            data_pagamento = data_base_amort + relativedelta(months=i)
            while data_pagamento.weekday() >= 5:
                data_pagamento += pd.Timedelta(days=1)
            datas_amortizacao.append(data_pagamento)
            descricoes_amortizacao.append(f"Parcela {i+1}/36")
            
        # Cria o DataFrame de uma só vez
        df_amort = pd.DataFrame({
            'data': datas_amortizacao,
            'categoria': 'Amort./ Resgat. Cota',
            'valor': valor_amortizacao,
            'descricao': descricoes_amortizacao
        })
        lista_de_fluxos_df.append(df_amort)
    
    # Concatena todos os DataFrames de uma vez no final
    if not lista_de_fluxos_df:
        return pd.DataFrame()
        
    return pd.concat(lista_de_fluxos_df, ignore_index=True)

def gerar_relatorio_d_mais(df_fluxos, data_base):
    """Gera a tabela de resumo no formato D+N."""
    prazos = [0, 1, 5, 10, 21, 63, 365]
    colunas_relatorio = ['Conta Corr.', 'Fundo inv.', 'Títulos Renda Fixa', 'Pagamentos Estoque', 'Despesas', 'Amort./ Resgat. Cota']
    linhas_relatorio = []
    for dias in prazos:
        data_limite = data_base + timedelta(days=dias)
        fluxos_ate_data = df_fluxos[df_fluxos['data'] <= data_limite]
        saldos_acumulados = fluxos_ate_data.groupby('categoria')['valor'].sum()
        linha = {'Prazo': f"D+{dias}"}
        for col in colunas_relatorio: linha[col] = saldos_acumulados.get(col, 0.0)
        linhas_relatorio.append(linha)
    df_relatorio = pd.DataFrame(linhas_relatorio).set_index('Prazo')
    df_relatorio['Disponibilidades'] = df_relatorio[colunas_relatorio[:4]].sum(axis=1)
    df_relatorio['Necessidades'] = df_relatorio[colunas_relatorio[4:]].sum(axis=1)
    df_relatorio['Caixa Projetado'] = df_relatorio['Disponibilidades'] + df_relatorio['Necessidades']
    return df_relatorio # Retorna o DataFrame com números para a função de HTML

def codificar_imagem_base64(caminho_imagem):
    try:
        with open(caminho_imagem, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except FileNotFoundError:
        return "iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVR42mNkYAAAAAYAAjCB0C8AAAAASUVORK5CYII="

def gerar_relatorio_html_final(df_summary, df_details, nome_fundo, data_ref, caminho_logo, arquivo_saida):
    """
    Gera o arquivo HTML final, com detalhamento para todas as categorias,
    EXCETO para 'Pagamentos Estoque' para manter o arquivo leve.
    """
    logo_base64 = codificar_imagem_base64(caminho_logo)
    
    # Prepara os dados detalhados (excluindo o estoque)
    df_details_leve = df_details[df_details['categoria'] != 'Pagamentos Estoque'].copy()
    df_details_leve['data'] = pd.to_datetime(df_details_leve['data'], errors='coerce')
    df_details_leve.dropna(subset=['data'], inplace=True)
    df_details_leve['data'] = df_details_leve['data'].dt.strftime('%d/%m/%Y')
    dados_detalhados_json = df_details_leve.to_json(orient='records', force_ascii=False)

    df_summary_formatted = df_summary.applymap('{:,.2f}'.format)
    tabela_html = '<table id="relatorioTabela" class="display compact"><thead><tr><th>Prazo</th>'
    for col in df_summary_formatted.columns:
        tabela_html += f'<th>{col}</th>'
    tabela_html += '</tr></thead><tbody>'
    for prazo, row in df_summary_formatted.iterrows():
        tabela_html += f'<tr><th>{prazo}</th>'
        for col, value in row.items():
            tabela_html += f'<td data-prazo="{prazo}" data-categoria="{col}">{value}</td>'
        tabela_html += '</tr>'
    tabela_html += '</tbody></table>'

    html_template = f"""
    <!DOCTYPE html><html lang="pt-BR"><head><meta charset="UTF-8"><title>Relatório de Fluxo de Caixa Interativo</title>
    <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.13.6/css/jquery.dataTables.min.css">
    <style>
        body {{ font-family: "Gill Sans MT", Arial, sans-serif; background-color: #FFFFFF; color: #313131; font-size: 1.1em; padding: 20px; }}
        h1 {{ font-size: 2.2em; color: #163f3f; margin: 0; }} .header-container {{ display: flex; align-items: center; margin-bottom: 20px; }}
        .info-box {{ border: 1px solid #e0e0e0; background-color: #f5f5f5; padding: 15px; margin-bottom: 30px; }}
        table.dataTable thead th {{ background-color: #163f3f !important; color: white !important; }}
        table.dataTable.display tbody tr.odd {{ background-color: #f5f5f5; }} table.dataTable.display tbody tr.even {{ background-color: #e0e0e0; }}
        table.dataTable tbody tr:hover {{ background-color: #FFD8A7 !important; }} table.dataTable tbody th {{ font-weight: bold; background-color: #0e3030; color: white; }}
        
        /* Modificação: O cursor só vira 'pointer' para células que não são do estoque */
        #relatorioTabela tbody td[data-categoria="Pagamentos Estoque"] {{ cursor: default; }}
        #relatorioTabela tbody td:not([data-categoria="Pagamentos Estoque"]):not(:empty) {{ cursor: pointer; }}

        .modal {{ display: none; position: fixed; z-index: 1000; left: 0; top: 0; width: 100%; height: 100%; overflow: auto; background-color: rgba(0,0,0,0.5); }}
        .modal-content {{ background-color: #fefefe; margin: 5% auto; padding: 20px; border: 1px solid #888; width: 80%; max-width: 900px; }}
        .close-button {{ color: #aaa; float: right; font-size: 28px; font-weight: bold; cursor: pointer; }}
        #modal-table-container table {{ width: 100%; border-collapse: collapse; }} #modal-table-container th, #modal-table-container td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
        #modal-table-container th {{ background-color: #f2f2f2; }}
    </style></head><body>
    <div class="header-container"><img src="data:image/png;base64,{logo_base64}" style="max-height:80px; margin-right:20px;"><h1>Relatório de Fluxo de Caixa Interativo</h1></div>
    <div class="info-box"><strong>Fundo:</strong> {nome_fundo}<br><strong>Data do Relatório:</strong> {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}<br><strong>Data de Referência (D+0):</strong> {data_ref.strftime('%d/%m/%Y')}</div>
    {tabela_html}
    <div id="detailModal" class="modal"><div class="modal-content"><span class="close-button">&times;</span><h2 id="modal-title">Detalhamento</h2><div id="modal-table-container" style="max-height: 400px; overflow-y: auto;"></div></div></div>
    <script src="https://code.jquery.com/jquery-3.7.0.js"></script><script src="https://cdn.datatables.net/1.13.6/js/jquery.dataTables.min.js"></script>
    <script>var dadosDetalhados = {dados_detalhados_json}; var dataReferencia = new Date('{data_ref.strftime("%Y-%m-%d")}T00:00:00');</script>
    <script>
    $(document).ready(function() {{
        $('#relatorioTabela').DataTable({{ "paging": false, "searching": false, "info": false, "ordering": false }});
        var modal = $('#detailModal');
        $('#relatorioTabela tbody').on('click', 'td', function() {{
            var cell = $(this); var prazo = cell.data('prazo'); var categoria = cell.data('categoria');

            // ### A CORREÇÃO ESTÁ AQUI ###
            // Ignora o clique se for uma coluna de resumo, vazia, ou a coluna de Estoque
            if (!prazo || !categoria || 
                ['Disponibilidades', 'Necessidades', 'Caixa Projetado', 'Pagamentos Estoque'].includes(categoria) || 
                cell.text().trim() === '0.00' || cell.text().trim() === '0,00') 
            {{
                return;
            }}

            var dias = parseInt(prazo.replace('D+', '')); var dataLimite = new Date(dataReferencia); dataLimite.setDate(dataLimite.getDate() + dias);
            var itensFiltrados = dadosDetalhados.filter(function(item) {{
                var dataItem = new Date(item.data.split('/').reverse().join('-') + 'T00:00:00');
                return item.categoria === categoria && dataItem <= dataLimite;
            }});
            var tableHtml = '<table><thead><tr><th>Data</th><th>Descrição</th><th style="text-align:right;">Valor</th></tr></thead><tbody>';
            var total = 0;
            itensFiltrados.forEach(function(item) {{
                total += item.valor;
                tableHtml += '<tr><td>' + item.data + '</td><td>' + (item.descricao || '') + '</td><td style="text-align:right;">' + item.valor.toLocaleString('pt-BR', {{style: 'currency', currency: 'BRL'}}) + '</td></tr>';
            }});
            tableHtml += '</tbody><tfoot><tr><th colspan="2">Total</th><th style="text-align:right;">' + total.toLocaleString('pt-BR', {{style: 'currency', currency: 'BRL'}}) + '</th></tr></tfoot></table>';
            $('#modal-title').text('Detalhamento de: ' + categoria + ' (até ' + prazo + ')');
            $('#modal-table-container').html(tableHtml);
            modal.show();
        }});
        $('.close-button').on('click', function() {{ modal.hide(); }});
        $(window).on('click', function(event) {{ if (event.target == modal[0]) modal.hide(); }});
    }});
    </script></body></html>
    """

    try:
        with open(arquivo_saida, 'w', encoding='utf-8') as f:
            f.write(html_template)
        print(f"✅ Relatório HTML interativo (versão leve) gerado com sucesso: '{os.path.abspath(arquivo_saida)}'")
    except Exception as e:
        print(f"❌ Erro ao salvar o arquivo HTML: {e}")


# --- BLOCO DE EXECUÇÃO FINAL ---
if __name__ == "__main__":
    dados_carteira_recente, data_referencia, _ = encontrar_e_carregar_carteira_recente(PATH_CARTEIRAS_HISTORICO)
    if dados_carteira_recente:
        df_estoque = processar_dados_estoque_novo(PATH_ESTOQUE_NOVO)
        df_despesas, _ = processar_dados_despesas_final(PATH_DESPESAS, data_referencia)
        df_fluxos_detalhados = preparar_fluxos_detalhados(dados_carteira_recente, df_estoque, df_despesas)
        relatorio_final = gerar_relatorio_d_mais(df_fluxos_detalhados, data_referencia)
        gerar_relatorio_html_final(relatorio_final, df_fluxos_detalhados, NOME_DO_FUNDO, data_referencia, CAMINHO_DO_LOGO, NOME_ARQUIVO_SAIDA)

Procurando o arquivo de carteira mais recente...
Usando a carteira mais recente para a projeção: carteira_52203615000119_20250930.json

Carregando dados do novo estoque...
Usando o conjunto de estoque da data: 30.09.25
Dados do novo estoque consolidados: 1724011 linhas.

Carregando despesas com a lógica final...
Usando o relatório de despesas gerado em: 30/09/2025
52 eventos de despesas futuras (reais + projetadas) foram criados.

1669666 parcelas do estoque com vencimento futuro foram incluídas na projeção.
✅ Relatório HTML interativo (versão leve) gerado com sucesso: 'c:\Users\Leo\Desktop\Porto_Real\portoauto\src\vortx_estoques\relatorio_fluxo_caixa_final.html'


C:\Users\Leo\AppData\Local\Temp\ipykernel_14628\3610487332.py:252: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_summary_formatted = df_summary.applymap('{:,.2f}'.format)


In [55]:
df_estoque.columns

Index(['Situacao', 'PES_TIPO_PESSOA', 'CedenteCnpjCpf',
       'TIT_CEDENTE_ENT_CODIGO', 'CedenteNome', 'Cnae', 'SecaoCNAEDescricao',
       'NotaPdd', 'SAC_TIPO_PESSOA', 'SacadoCnpjCpf', 'SacadoNome',
       'IdTituloVortx', 'TipoAtivo', 'DataEmissao', 'DataAquisicao',
       'Data Vencimento', 'NumeroBoleto', 'NumeroTitulo', 'CampoChave',
       'ValorAquisicao', 'Valor Nominal', 'Valor Presente', 'PDDNota',
       'PDDVencido', 'PagamentoParcial', 'Coobricacao', 'DataGeracao',
       'PDDTotal', 'CampoAdicional1', 'CampoAdicional2', 'CampoAdicional3',
       'CampoAdicional4', 'CampoAdicional5', 'PDDEfeitoVagao',
       'PercentagemEfeitoVagao', 'IdTituloVortxOriginador', 'Registradora',
       'IdContratoRegistradora', 'IdTituloRegistradora'],
      dtype='object')

In [56]:
df_estoque["PDDTotal"]

0          424.0903
1          504.5277
2          315.1219
3            0.0000
4            0.0000
             ...   
1644299      0.0000
1644300      0.0000
1644301      0.0000
1644302      0.0000
1644303      0.0000
Name: PDDTotal, Length: 1644304, dtype: float64